In [2]:
import string
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

POS Tagging demonstration:

In [3]:
def preprocessing(str):
    #stopword removal
    Stop = stopwords.words('english')
    #remove punctuation
    no_punc = [char for char in str if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    #add word lemmatization.
    
    #return ' '.join([word for word in no_punc.split() if (word.lower() not in Stop)])
    return ' '.join([word for word in no_punc.split()])

In [4]:
def pos_tagging(s):
    nopunc_sentence = preprocessing(s)
    print(nopunc_sentence)
    lemmatizer = WordNetLemmatizer()
    tags = pos_tag(nltk.word_tokenize(nopunc_sentence))
    dict_tag = {'NN':[], 'VB':[], 'OTH':[]} #only three categories at the moment, can add all categories coming from pos_tag programmatically.
    for tag in tags:
        if tag[1] in ["NN","NNS","NNP","NNPS"]: #NN-singular noun, NNS- noun plural, NNP-proper noun singular, NNPS- proper noun plural.
            dict_tag['NN'].append(tag[0])
        elif tag[1] in ["VB","VBD","VBG","VBN","VBP","VBZ"]: #VB: verb in base form, VBG: verb in present participle, VBN: past participle taken, VBZ: verb singular, present, non-3rd person take, VBP:verb, singular present
            dict_tag['VB'].append(tag[0])
        else:
            dict_tag['OTH'].append(tag[0])
    return dict_tag

In [5]:
string2 = "You should be ashamed of yourself! for slandering Newcastle United"
tags2 = pos_tagging(string2)
print(tags2)

You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}


In [6]:
print(preprocessing(string2))

You should be ashamed of yourself for slandering Newcastle United


In [7]:
lemmatizer = WordNetLemmatizer()

list2= nltk.word_tokenize(preprocessing(string2))
lemmatized = ' '.join([lemmatizer.lemmatize(w) for w in list2])
print(list2)
print(lemmatized)
print(pos_tagging(lemmatized))
print(pos_tagging(preprocessing(string2)))
print(pos_tagging(string2))

['You', 'should', 'be', 'ashamed', 'of', 'yourself', 'for', 'slandering', 'Newcastle', 'United']
You should be ashamed of yourself for slandering Newcastle United
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}


Generating transition and emission matrices -> tarini_HMM


In [8]:
import numpy as np
import math
import string
from collections import defaultdict
import pandas as pd

In [9]:
#Initialization

setPunctuation = set(string.punctuation)
suffixNoun = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
suffixVerb = ["ate", "ify", "ise", "ize"]
suffixAdjective = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
suffixAdverb = ["ward", "wards", "wise"]

In [10]:
# preprocessing datasets

def testingDataSet_preProcessing(vocabulary, file):
    preprossed = []
    
    with open(file, "r") as f:
        for index, word in enumerate(f):
            if not word.split():
                word = "--n--"
                preprossed.append(word)

            elif word.strip() not in vocabulary:
                word = tagAllotement(word)
                preprossed.append(word)

            else:
                preprossed.append(word.strip())

    return preprossed
def tagAllotement(word):
    if any(char.isdigit() for char in word):
        return "--unk_digit--"

    elif any(char in setPunctuation for char in word):
        return "--unk_punct--"

    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    elif any(word.endswith(suffix) for suffix in suffixNoun):
        return "--unk_noun--"

    elif any(word.endswith(suffix) for suffix in suffixVerb):
        return "--unk_verb--"

    elif any(word.endswith(suffix) for suffix in suffixAdjective):
        return "--unk_adj--"

    elif any(word.endswith(suffix) for suffix in suffixAdverb):
        return "--unk_adv--"
    return "--unk--"

def preprocessWord(wordTagPair, vocabulary): 
    if not wordTagPair.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word,tag = wordTagPair.split()
        if word not in vocabulary: 
            word = tagAllotement(word)
        return word, tag
    return None 

In [11]:
vocabulary = {}  

with open("WSJ_02-21.pos", 'r') as file:
    trainingDataset = file.readlines()

with open("WSJ_24.pos", 'r') as file:
    testingDataset = file.readlines()
    
with open("hmm_vocab.txt", 'r') as file:
    voc = file.read().split('\n')

i = 0
for word in voc:
    vocabulary[word] = i
    i+=1
    
#preprocess testingDataSet
preprocessedTestingData = testingDataSet_preProcessing(vocabulary, "test.words") 

In [ ]:
vocabulary

In [13]:
def calcCounts(trainingDataset, vocabulary):
    #countEmission - #(tag,word) pairs
    #countTransition - #(tag,tag) pairs
    #countTag - #tags
    countEmission = defaultdict(int)
    countTransition = defaultdict(int)
    countTag = defaultdict(int)
    
    previousTag = '--s--' 
    index = 0 
    
    for wordTagPair in trainingDataset:
        index+=1

        word, tag = preprocessWord(wordTagPair, vocabulary)
        
        countTransition[(previousTag, tag)] += 1
        countEmission[(tag, word)] += 1
        countTag[tag] += 1
        
        previousTag = tag
        
    return countEmission, countTransition, countTag

In [14]:
countEmission, countTransition, countTag = calcCounts(trainingDataset, vocabulary)

In [15]:
markovStates = sorted(countTag.keys())
len(markovStates)

46

In [16]:
def calcTransitionMatrix(countTransition, countTag, epsilon, markovStates):
    
    numberStates = len(markovStates)
    transitionMatrix = np.zeros((numberStates, numberStates))
    tagTagPair = set(countTransition.keys())

    for i in range(numberStates):
        for j in range(numberStates):
            countNextTag = 0
            key = (markovStates[i],markovStates[j])
            
            if key in countTransition: 
                countNextTag = countTransition[key]
                
            countPreviousTag = countTag[key[0]]
            transitionMatrix[i][j] = (countNextTag+epsilon)/(countPreviousTag+(numberStates*epsilon))

    return transitionMatrix
epsilon = 0.001
transitionMatrix = calcTransitionMatrix(countTransition, countTag, epsilon, markovStates)
def calcEmissionMatrix(countEmission, countTag, epsilon, markovStates, vocabulary):

    numberStates = len(markovStates)
    numberWords = len(vocabulary)
    emissionMatrix = np.zeros((numberStates, numberWords))
    tagWordPair = set(list(countEmission.keys()))
    
    for i in range(numberStates):
        for j in range(numberWords):
            countWord = 0
            key =  (markovStates[i],vocabulary[j])

            if key in countEmission: 
                countWord = countEmission[key]

            countTags = countTag[key[0]]

            emissionMatrix[i][j] = (countWord+epsilon)/(countTags+(epsilon*numberWords))

    return emissionMatrix
emissionMatrix = calcEmissionMatrix(countEmission, countTag, epsilon, markovStates,  list(vocabulary))


Viterbi Algorithm.


In [17]:
def viterbi_initialise(states, tag_counts, transitionMatrix, emissionMatrix, corpus, vocab):
    n_tags = len(tag_counts)
    best_probs = np.zeros((n_tags, len(corpus)),dtype = float)
    best_paths = np.zeros((n_tags, len(corpus)), dtype = float)
    #initialising best_probs and best_paths.

    start_index = states.index("--s--")
    for i in range(n_tags):
        if transitionMatrix[start_index, i]==0:
            best_probs[i,0] = float('-inf')
        else:
            best_probs[i,0] = (transitionMatrix[start_index,i])*(emissionMatrix[i,vocab[corpus[0]]])
    
    return best_probs, best_paths

In [18]:
def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = tagAllotement(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep

In [19]:
orig, t = preprocess(vocabulary, "test.words")
best_probs, best_paths = viterbi_initialise(markovStates, countTag, transitionMatrix = transitionMatrix, emissionMatrix = emissionMatrix, corpus = t, vocab = vocabulary )


In [20]:
print(best_probs)



[[1.51592315e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.50510938e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.78603817e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.31040319e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.88505608e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.05949895e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [21]:
print(len(markovStates))

46


In [22]:
print(best_paths.shape)

(46, 34199)


In [23]:
print(best_paths)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [26]:
#a

array([[ 1.51592315e-10, -1.86320649e+01, -3.00482069e+01, ...,
        -2.08549676e+05, -2.08566966e+05, -2.08571263e+05],
       [ 9.50510938e-11, -1.75936734e+01, -3.10134850e+01, ...,
        -2.08549629e+05, -2.08562247e+05, -2.08575061e+05],
       [ 5.78603817e-11, -1.85364714e+01, -2.79457885e+01, ...,
        -2.08555186e+05, -2.08561596e+05, -2.08560331e+05],
       ...,
       [ 1.31040319e-10, -1.83233393e+01, -2.77094482e+01, ...,
        -2.08553206e+05, -2.08567112e+05, -2.08571409e+05],
       [ 2.88505608e-09, -1.90734414e+01, -2.75470123e+01, ...,
        -2.08551249e+05, -2.08559584e+05, -2.08566925e+05],
       [ 1.05949895e-08, -1.83628846e+01, -2.87423753e+01, ...,
        -2.08551210e+05, -2.08559707e+05, -2.08565224e+05]])

In [27]:
#b

array([[ 0., 33., 20., ..., 14., 26.,  7.],
       [ 0.,  3., 20., ..., 14., 26.,  7.],
       [ 0., 34., 20., ..., 14., 26.,  7.],
       ...,
       [ 0.,  5., 20., ..., 14., 26.,  7.],
       [ 0., 45., 20., ..., 14., 26.,  7.],
       [ 0.,  6., 20., ..., 14., 26.,  7.]])

In [30]:
#print(prediction_array)


['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NN', 'VBZ', 'DT', 'NN', ',', 'IN', 'NNS', 'IN', 'NN', ',', 'NN', ',', 'NN', 'CC', 'NN', '.', '--s--', 'DT', 'RBS', 'JJ', 'NN', 'MD', 'VB', 'DT', 'NNP', 'NN', 'NN', 'NN', 'JJ', 'IN', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'TO', 'RB', '$', 'CD', 'CD', 'IN', 'NNP', 'POS', '$', 'CD', 'CD', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NNP', '.', '--s--', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'NN', 'NN', 'VBZ', 'VBN', 'TO', 'VB', ',', 'IN', 'RB', 'RB', 'RB', 'IN', 'DT', 'CD', 'NN', 'NN', 'VBD', 'NNP', 'IN', 'DT', 'NN', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'VBG', 'VBN', 'IN', 'DT', 'NN', 'DT', 'NN', 'NN', 'VBD', 'RB', 'RB', 'IN', 'NNP', 'POS', 'NN', ',', 'IN', 'PRP', 'VBD', 'VBN', 'IN', 'PRP$', 'JJ', 'JJ', 'NN', '.', '--s--', 'NNS', 'VBP', 'VBN', 'RB', 'TO', 'WRB', 'JJ', 'NN', 'NN', 'PRP', 'VBP', 'TO', 'VB', 'IN', 'NNP', 'NNS', 'IN', 'JJ', 'N

In [51]:
def viterbi(transitionMatrix, emissionMatrix, t, best_probs, best_paths, states, vocab):
    #best_probs, best_paths = viterbi_forward(transitionMatrix, emissionMatrix, t, best_probs, best_paths, vocab)
    n_tags = best_probs.shape[0]
    for i in range(1, len(t)):
        for j in range(n_tags):
            best_prob_i = float('-inf')
            best_path_i = None
            for k in range(n_tags):
                probability = best_probs[k,i-1]+math.log(emissionMatrix[j,vocab[t[i]]] * transitionMatrix[k,j])
                if probability > best_prob_i:
                    best_prob_i = probability
                    best_path_i = k
            best_probs[j,i]=best_prob_i
            best_paths[j,i] = best_path_i

    #prediction_array = viterbi_backward(best_probs, best_paths, t, states)
    corpus_words_length = len(best_paths[0]) #number of words in the corpus  = number of columns in the best_probs and best_paths matrix
    array_z = [None]* corpus_words_length
    unique_pos_tag_length = len(best_probs)  #number of unique pos tags
    last_word_best_prob = float("-inf")
    prediction_array = [None]*corpus_words_length
    for i in range(unique_pos_tag_length):
        if best_probs[i, -1]> last_word_best_prob:
            last_word_best_prob = best_probs[i, -1]
            array_z[corpus_words_length - 1] = i
    prediction_array[corpus_words_length - 1] = states[array_z[corpus_words_length - 1]]
    
    for j in range(corpus_words_length - 1, 0, -1):
        pos_tag_for_word = array_z[j]
        if(pos_tag_for_word!=None):
            pos_tag_for_word = (int)(pos_tag_for_word) #typecasting vector to int, later used for indexing
            #print(pos_tag_for_word)
        array_z[j - 1] = best_paths[int(pos_tag_for_word), j] 
        prediction_array[j -1] = states[int(array_z[j - 1])]
    return prediction_array

In [52]:
prediction_array2 = viterbi(transitionMatrix, emissionMatrix, t, best_probs, best_paths, markovStates, vocab = vocabulary)

In [50]:
prediction = ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NN', 'VBZ', 'DT', 'NN', ',', 'IN', 'NNS', 'IN', 'NN', ',', 'NN', ',', 'NN', 'CC', 'NN', '.', '--s--', 'DT', 'RBS', 'JJ', 'NN', 'MD', 'VB', 'DT', 'NNP', 'NN', 'NN', 'NN', 'JJ', 'IN', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'TO', 'RB', '$', 'CD', 'CD', 'IN', 'NNP', 'POS', '$', 'CD', 'CD', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NNP', '.', '--s--', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'NN', 'NN', 'VBZ', 'VBN', 'TO', 'VB', ',', 'IN', 'RB', 'RB', 'RB', 'IN', 'DT', 'CD', 'NN', 'NN', 'VBD', 'NNP', 'IN', 'DT', 'NN', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'VBG', 'VBN', 'IN', 'DT', 'NN', 'DT', 'NN', 'NN', 'VBD', 'RB', 'RB', 'IN', 'NNP', 'POS', 'NN', ',', 'IN', 'PRP', 'VBD', 'VBN', 'IN', 'PRP$', 'JJ', 'JJ', 'NN', '.', '--s--', 'NNS', 'VBP', 'VBN', 'RB', 'TO', 'WRB', 'JJ', 'NN', 'NN', 'PRP', 'VBP', 'TO', 'VB', 'IN', 'NNP', 'NNS', 'IN', 'JJ', 'NN', 'CC', 'NN', 'NN', ',', 'RB', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'NNP', 'NN', 'NNS', ',', 'JJ', 'NNP', ',', 'VBP', 'VBN', 'TO', 'VB', 'VBD', 'RB', '.', '--s--', '``', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN', "''", 'IN', 'DT', 'NN', 'NN', ',', 'VBD', 'NNP', 'NNP', ',', 'NN', 'IN', 'NN', 'IN', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'IN', 'PRP', 'VBD', 'VBN', 'IN', 'DT', 'RB', 'JJ', 'NN', ',', 'MD', 'VB', 'NN', 'NNS', ':', 'CC', 'DT', 'NN', 'VBD', 'NNP', 'IN', 'NNS', 'VBD', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'NNP', 'CC', 'JJ', 'NNS', 'VBP', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'VBG', 'DT', 'NN', 'NN', 'VBP', 'RB', 'VBN', 'VBN', '.', '--s--', '``', 'NNP', 'VBZ', 'RB', 'VBG', 'TO', 'VB', 'RBR', 'RB', 'JJ', 'IN', 'DT', 'JJ', 'CD', 'CC', 'CD', 'NNS', 'IN', 'NN', 'VBZ', 'TO', 'VB', ',', "''", 'PRP', 'VBD', '.', '--s--', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'VBN', 'RB', 'IN', 'NNP', ',', 'CC', 'RB', 'RB', 'RB', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNS', ',', 'NNS', 'VBD', '.', '--s--', 'NNP', 'IN', 'VBG', 'NN', 'VBP', 'VBN', 'IN', 'NNS', 'WP', 'VBD', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'NN', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'CC', 'DT', 'WP', 'VBP', 'DT', 'RB', 'RBR', 'JJ', 'JJ', 'NN', 'NNS', 'IN', 'PRP$', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'DT', 'NNS', ':', 'DT', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'IN', 'CD', 'NN', 'TO', 'DT', 'NN', 'IN', 'CD', 'NN', ',', 'VBG', 'TO', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'NNS', ',', 'WDT', 'VBD', 'VB', 'RP', 'DT', 'NN', 'NN', 'NN', ',', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'IN', 'DT', 'NN', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NNP', 'IN', 'NNP', 'NNP', ',', 'VBZ', 'IN', 'DT', 'VBG', 'DT', 'RBR', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'IN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', '--s--', '``', 'NN', 'NNS', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', ',', 'CC', 'IN', 'DT', 'NNP', 'NN', 'PRP', 'MD', 'RB', ',', "''", 'PRP', 'VBD', '.', '--s--', 'NNP', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'JJ', ',', 'CC', 'NN', 'NNS', 'VBD', 'RB', 'RB', 'IN', 'CD', 'NN', ',', 'VBD', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNP', 'NNP', 'NNP', '.', '--s--', 'PRP', 'RB', 'VBZ', 'PRP', 'VBZ', '``', 'JJ', 'NN', ',', "''", 'WDT', 'VBZ', 'DT', 'JJ', 'NN', 'CC', 'NN', 'NNS', ',', 'VBD', 'JJ', 'JJ', 'NN', '.', '--s--', 'PRP', 'VBZ', 'DT', 'NN', 'IN', 'RB', 'RB', 'IN', 'CD', 'NN', 'IN', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'RB', 'JJR', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'CD', 'NNS', '.', '--s--', 'NNS', 'VBP', 'DT', 'NNP', 'NN', 'IN', 'NNS', 'IN', 'JJ', 'NNS', 'VBD', 'DT', 'NN', 'IN', 'NNP', 'VBZ', ',', 'IN', 'NN', 'VBZ', 'RB', 'JJ', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', '--s--', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'POS', 'JJ', ',', 'WRB', 'MD', 'PRP', 'RB', 'RB', 'VB', 'DT', 'NN', 'TO', 'VB', 'IN', 'DT', 'NNS', 'VBP', 'JJ', 'NN', 'IN', 'PRP', 'VBD', '.', '--s--', ':', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', '--s--', 'WRB', 'NN', 'NNS', 'VBP', 'VBN', 'JJ', 'TO', 'NN', 'NN', 'NNS', ',', 'VBP', 'PRP', '``', 'VBN', 'IN', 'PRP$', 'NNS', 'RB', "''", '.', '--s--', ':', 'NNP', 'NNP', '.', '--s--', 'VBG', 'DT', 'NN', 'NN', 'VBZ', 'RB', 'VB', 'RB', 'CC', 'RB', 'JJ', 'NNS', 'CC', 'NN', 'NNS', ',', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NNS', 'MD', 'VB', 'VBG', 'NNP', 'POS', 'NN', 'IN', 'DT', '.', '--s--', '``', 'NNP', 'NNP', "''", 'VBZ', 'DT', 'JJ', 'NN', 'WDT', 'VBZ', 'RB', 'CD', 'NNS', 'RB', 'IN', 'JJ', 'NN', 'IN', 'PRP$', 'NN', ',', 'NNP', 'NNP', '.', '--s--', 'DT', 'JJ', 'NNS', 'VBP', 'PRP', 'RB', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'CD', '.', '--s--', 'NNP', 'POS', 'NN', 'IN', 'CD', ',', 'RB', 'TO', 'VB', 'JJ', '.', '--s--', 'VBG', 'TO', 'DT', 'NN', 'POS', 'NN', 'NN', ',', '``', 'NNP', 'NNP', ',', "''", 'VBD', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NNS', 'NN', 'IN', '#', 'CD', 'CD', 'NN', ',', '``', 'VBZ', 'DT', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'JJ', ',', 'JJ', 'NNS', ',', 'VBN', 'IN', 'PRP$', 'JJ', 'NNS', 'IN', 'NN', ',', 'NN', 'CC', 'NN', '.', '--s--', 'CC', 'IN', 'PDT', 'DT', 'NN', 'CC', 'NN', 'IN', 'JJ', 'NN', 'VBN', 'DT', 'JJ', 'NNS', 'TO', 'VB', 'RB', ':', 'NN', 'TO', 'VB', 'RP', 'DT', 'CD', 'IN', 'NN', ':', 'NN', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', ':', "''", '--s--', 'PRP', 'VBP', 'RB', 'VB', 'DT', 'JJ', 'NNS', 'NN', 'TO', 'VB', 'DT', 'NN', 'VBZ', 'JJ', 'IN', 'JJR', 'IN', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'NN', 'VBZ', 'VBN', 'VBN', 'IN', 'NNP', 'POS', 'JJ', 'NNS', 'IN', 'JJ', 'IN', 'NN', ',', 'CC', 'PRP$', 'JJ', 'NN', 'VBZ', 'PRP$', 'JJ', 'NNS', 'IN', 'DT', 'NNS', 'POS', 'JJ', 'CC', 'JJ', 'NNS', '.', '--s--', 'NNP', 'RB', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'NN', ',', 'DT', 'NN', 'NNS', 'VBP', 'JJ', 'NN', '.', '--s--', 'VBG', 'IN', 'DT', 'NN', 'VBN', 'CC', 'NNS', 'VBN', 'IN', 'DT', 'JJ', 'NNS', ',', 'NNP', 'NNP', 'POS', 'NN', 'MD', 'VB', 'RB', 'RB', 'VBN', 'IN', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'NN', 'PRP', 'VBP', 'IN', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', ',', 'VBZ', 'RB', 'VBN', 'RP', 'DT', 'CD', 'IN', 'NN', '.', '--s--', 'NN', 'VBZ', ',', 'PRP', 'VBZ', 'VBN', 'PRP', 'RB', 'RB', 'RB', '.', '--s--', 'RB', 'IN', 'VB', 'DT', 'NN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'NNP', 'NN', 'IN', 'PRP', 'TO', 'DT', 'NN', 'NN', 'IN', 'VBG', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'VBZ', 'VBG', 'TO', 'VB', 'NN', '.', '--s--', 'NNS', 'VBP', 'DT', 'NN', 'RB', 'VBG', 'NNS', ',', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'JJ', 'NN', 'NNS', 'MD', 'VB', 'JJ', '.', '--s--', 'NNP', ',', 'VBG', 'NNP', '.', '--s--', 'DT', 'NN', 'NNS', 'VBP', 'RP', 'RB', ',', 'CC', 'PDT', 'DT', 'RB', 'JJ', 'NNS', 'NN', 'IN', 'NN', '.', '--s--', 'RB', 'RB', 'IN', 'VBG', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', '.', '--s--', 'CC', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBZ', 'VBN', 'TO', 'NNS', 'TO', 'VB', 'VBN', 'IN', 'DT', 'RB', 'JJ', 'JJ', 'NNS', '(', 'IN', 'WRB', 'VBZ', 'NN', 'NNS', 'RB', 'JJ', 'RB', '.', ')', '.', '--s--', 'NNP', ',', 'NNP', 'NNP', 'VBZ', 'DT', 'NN', ',', 'RB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'NNS', 'VBP', 'PRP$', 'NN', 'IN', 'VBG', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'NN', '.', '--s--', 'RB', 'IN', 'VBG', 'NNS', 'TO', 'VB', 'DT', 'NN', 'POS', 'NN', ',', 'DT', 'NN', 'POS', 'JJ', 'NNS', ',', 'VBN', 'IN', 'DT', 'JJ', 'NNP', ',', 'VB', 'PRP$', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBN', 'WRB', 'DT', 'NN', 'POS', 'JJ', 'NN', ',', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', ',', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'CC', 'DT', 'NN', 'VBZ', '.', '--s--', 'IN', 'PRP', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'JJ', 'NN', 'VBP', 'JJ', 'NNS', ',', 'PRP', 'RB', 'VBP', 'VBN', 'VBN', ',', 'IN', 'JJ', 'IN', 'DT', 'NNS', 'VBP', 'RB', 'VBN', '.', '--s--', 'IN', 'NN', ',', 'JJ', 'VBP', 'RB', 'IN', 'NN', '.', '--s--', 'CD', 'NN', 'DT', 'NN', 'RB', 'VBN', 'IN', 'NN', '.', '--s--', 'NNP', 'NNS', 'IN', 'DT', 'NN', 'NN', ',', 'CC', 'IN', 'PRP', 'VBZ', 'DT', 'JJ', 'NNP', ',', 'PRP', 'VBZ', 'RB', 'NN', 'TO', 'VB', 'CC', 'VB', 'PRP', ':', 'DT', 'JJ', 'NN', 'MD', 'VB', 'VBN', 'RB', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'VBZ', 'VBN', 'RP', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'PRP', 'VBZ', 'NNP', ',', 'DT', 'JJ', 'NNS', 'POS', 'NN', 'IN', 'DT', 'NN', ',', 'IN', 'DT', 'JJ', 'NN', 'TO', 'VB', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'IN', 'PRP$', 'NN', '.', '--s--', 'RB', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'IN', 'NNP', 'NNP', 'VBZ', 'RB', 'JJ', 'TO', 'VB', 'TO', 'NNS', 'IN', 'DT', 'NN', 'VBN', 'IN', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', ',', 'DT', 'VBZ', 'RB', 'JJ', 'NN', ',', 'CC', 'JJ', 'NN', ',', 'CC', 'IN', 'JJ', 'NNS', ',', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'DT', 'NN', 'NN', 'POS', 'RBS', 'JJ', 'NN', '.', '--s--', 'NNP', 'RB', ',', 'PRP', 'VBZ', 'IN', 'DT', 'NN', 'VBN', 'VBN', 'NN', ',', 'VBZ', 'NN', 'NNS', 'CC', 'VBZ', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'RB', 'TO', 'VB', 'PRP', 'VBZ', ',', 'RB', 'JJ', ',', 'TO', 'DT', 'NNS', ',', 'DT', 'NN', 'VBZ', 'JJ', 'NNP', 'IN', 'DT', 'JJ', 'NN', ',', 'NN', 'IN', 'CD', 'JJ', 'NNS', '.', '--s--', 'PRP', 'VBZ', 'PRP$', 'NN', 'TO', 'DT', 'NN', ',', 'WRB', 'PRP', 'VBZ', 'DT', 'JJ', 'JJ', 'NN', 'WP', 'VBD', 'PRP', 'IN', 'VBG', 'DT', 'NN', 'IN', 'PRP$', 'JJ', 'NNS', 'CC', 'RB', 'RB', 'VB', 'PRP$', 'NN', 'CC', 'VBD', 'PRP', 'IN', 'NNP', '.', '--s--', 'IN', 'VBG', 'CC', 'VBG', '``', 'NNP', 'NNP', ',', "''", 'NNP', 'VBZ', 'VBN', 'IN', 'RB', 'RB', 'IN', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'NNS', 'VBP', 'DT', 'RB', 'VB', '#', 'CD', 'CD', '(', '$', 'CD', 'CD', ')', ',', 'CC', 'MD', 'VB', 'VBN', 'RB', 'JJR', 'VBD', 'RB', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'VBD', 'VBN', 'VBN', 'IN', 'DT', 'NN', 'IN', '``', 'NNS', ',', "''", 'DT', 'RBR', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'CD', 'NNS', 'VBD', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'CD', 'JJ', 'JJ', 'NNS', 'CC', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NNP', 'NN', '.', '--s--', 'DT', 'NNS', 'VBP', 'DT', 'JJ', 'NN', 'IN', '``', 'NNP', 'NNP', 'POS', "''", 'NNS', 'TO', 'VB', 'PRP', ',', 'CC', 'DT', 'JJ', 'NN', 'IN', 'PRP', ',', 'IN', 'NNP', '.', '--s--', 'NNP', 'POS', 'NNP', 'NN', 'NN', ',', 'NNP', 'NNP', ',', 'VBZ', 'VBG', 'TO', 'VB', 'RB', 'IN', '.', '--s--', 'PRP', 'VBZ', 'VBG', 'JJ', ',', 'VBG', 'CD', 'NNP', 'NN', 'NN', ':', '``', 'NNP', 'POS', 'NN', ',', "''", '``', 'RB', 'RB', 'IN', 'NNP', "''", 'CC', '``', 'VBG', 'PRP', 'RB', "''", ':', 'WDT', 'VBD', 'JJ', 'NNS', '.', '--s--', 'RB', 'IN', 'VBG', 'PRP', ',', 'NNP', 'NNP', 'RB', 'VBZ', 'VBN', 'PRP$', 'NN', 'IN', 'DT', 'NN', 'CC', 'DT', 'JJ', 'NN', ':', '``', 'NNP', 'NNP', "''", 'MD', 'VB', 'VBN', 'DT', 'NN', ',', 'DT', 'JJ', 'NN', 'TO', 'DT', 'JJ', 'JJ', '``', 'NNP', 'NNP', "''", 'NNS', 'IN', 'DT', 'NNS', '.', '--s--', 'DT', 'NNS', 'RB', 'VBP', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'WDT', 'DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', ',', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'JJ', 'IN', 'DT', 'JJ', 'NN', 'NN', 'JJ', 'NNS', 'TO', 'NNP', 'NNP', 'CC', 'NN', 'NNS', 'VBN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'JJ', 'NNP', 'NNP', 'NNP', ':', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'NNP', ',', 'MD', 'VB', 'IN', 'NNP', 'NNP', ',', 'CC', 'NNP', 'NNP', '(', 'NNP', ')', 'IN', 'NNP', 'NNP', ':', 'NNP', 'NNP', 'VBZ', 'DT', 'JJ', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'MD', 'RB', 'VB', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNPS', 'NNS', '.', '--s--', 'RB', ',', 'JJ', 'NNS', ',', 'NN', 'IN', 'NN', 'IN', '``', 'NNP', 'IN', 'NN', "''", ':', 'DT', 'NN', 'VBZ', 'RB', 'VBG', 'DT', 'NN', 'IN', 'JJ', 'JJ', 'NNS', 'WDT', 'RB', 'VBP', 'VBN', 'IN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'VBZ', 'RB', 'VBD', 'DT', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'RB', 'IN', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'DT', 'NN', 'NN', '.', '--s--', 'PRP', 'VBZ', 'IN', 'DT', 'JJ', 'NN', 'NN', ',', 'WDT', 'VBZ', ',', 'VBG', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'NNS', '.', '--s--', 'CC', 'PDT', 'DT', 'NN', ',', 'DT', 'NNS', 'VBP', 'IN', 'DT', 'NN', 'POS', 'NN', '.', '--s--', 'PRP', 'MD', 'VB', 'RB', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'POS', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', 'IN', 'NNP', 'VBD', 'NNS', 'IN', 'PRP$', 'NN', 'NNS', 'IN', 'PRP$', 'JJ', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'IN', 'VBN', 'NNS', 'IN', 'NN', 'NNS', 'IN', 'DT', 'CD', 'NNP', 'NNS', 'WRB', 'NNP', 'VBZ', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'POS', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'NNPS', 'NNP', ',', 'DT', 'JJ', 'NN', ',', 'NN', ',', 'NN', 'CC', 'JJ', 'NNS', 'NN', 'WDT', 'VBZ', 'NN', 'IN', 'NNP', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'TO', 'VB', 'NNP', 'NNS', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNP', ',', 'NNP', 'NNP', 'IN', 'NNP', 'VBD', 'TO', 'VB', 'NNS', 'TO', 'NNP', 'IN', 'PRP', 'VBZ', 'JJ', 'IN', 'VBG', 'NNP', '.', '--s--', 'IN', 'NN', 'IN', 'DT', 'NN', ',', 'NNP', 'VBD', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'POS', 'NN', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', '$', 'CD', 'CD', 'TO', 'VB', 'VBN', 'TO', 'NNP', 'IN', 'NNP', ',', 'IN', '$', 'CD', 'CD', 'MD', 'VB', 'IN', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'NN', ',', 'NNP', 'NNP', ',', '$', 'CD', 'CD', 'MD', 'VB', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBN', 'IN', 'NNP', ',', 'CC', 'DT', 'VBG', '$', 'CD', 'CD', 'MD', 'VB', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'NNP', ',', 'NNP', 'NNP', ',', 'NN', 'CC', 'JJ', 'NN', 'NN', 'IN', 'NNP', ',', 'VBD', 'PRP$', 'NN', 'VBZ', 'RB', 'VBN', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBN', 'IN', 'NNP', '.', 'NNP', 'NNP', 'NNP', 'IN', 'PRP', 'MD', 'VB', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'NN', 'VBP', 'NNP', 'NNP', ',', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', ',', 'PRP', 'VBD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', 'VBZ', '``', 'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'JJ', 'NNS', 'IN', 'PRP', 'MD', 'CC', 'MD', 'VB', "''", 'TO', 'VB', 'VB', 'RP', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'PRP', 'VBD', 'DT', 'NNS', 'TO', 'VB', 'VBN', 'MD', 'VB', '``', 'JJ', "''", 'NNS', ',', 'VBG', 'DT', 'NN', 'NN', 'CC', 'DT', 'JJ', 'NN', 'NN', ',', 'CC', 'MD', 'RB', 'VB', 'DT', 'NNS', 'IN', 'NNP', '.', '--s--', '``', 'PRP', 'MD', 'RB', 'VB', 'DT', 'NN', 'IN', 'NNP', ',', "''", 'PRP', 'VBD', '.', '--s--', 'DT', 'VBN', 'NNS', 'RB', 'VB', 'RP', 'IN', 'IN', 'DT', 'JJ', 'NN', ',', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', 'TO', 'VB', 'NNS', 'TO', 'NNP', 'IN', 'DT', 'NN', 'IN', 'NNP', '.', '--s--', '``', 'PRP', 'VBP', 'IN', 'IN', 'WP', 'PRP', 'VBD', ',', 'DT', 'NNS', 'MD', 'RB', 'VBP', 'TO', 'VB', 'NNP', ',', 'CC', 'PRP', 'MD', 'RB', 'VB', 'IN', 'DT', 'NN', 'IN', 'PRP', ',', 'IN', 'NNP', 'MD', 'RB', 'VB', 'DT', 'NN', 'IN', 'NNP', 'IN', 'RB', ',', "''", 'NNP', 'NNP', 'VBD', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'IN', 'NNP', 'VBZ', 'NN', 'IN', 'DT', 'NN', 'NNS', 'IN', 'DT', 'CD', 'NNS', 'WRB', 'NNP', 'CC', 'PRP$', 'JJ', 'NNS', 'VBP', 'VBN', '.', '--s--', 'DT', 'VBN', 'NNS', 'VBD', 'VBN', 'IN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', ',', 'CC', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'VBD', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'TO', 'VB', 'PRP$', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBD', 'PRP$', 'JJ', '#', 'CD', 'CD', '(', '$', 'CD', 'CD', ')', 'NN', 'NN', 'IN', 'PRP', 'VBD', 'DT', 'NN', ',', 'CC', 'VBD', 'PRP', 'MD', 'VB', 'DT', 'JJ', 'NN', 'IN', 'CC', 'WRB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'TO', 'NNP', 'VBZ', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'NNP', 'VBD', 'IN', 'DT', 'VBN', 'NNS', 'WDT', ',', '``', 'PRP', 'MD', 'VB', 'DT', 'NN', 'JJ', 'VBZ', 'VBN', '.', '--s--', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'RB', 'DT', 'JJ', 'JJ', 'CD', ',', 'CC', 'NNP', 'RB', 'VBZ', 'TO', 'VB', 'RP', 'CD', 'NN', 'IN', 'NNP', 'POS', 'NNS', 'IN', 'NNS', 'TO', 'VB', 'PRP$', 'NN', '.', "''", '--s--', 'DT', 'NN', 'VBZ', 'RB', 'VB', 'DT', 'CD', 'NN', 'RB', 'VBN', 'RP', 'IN', 'NNP', 'IN', 'NNP', ',', 'DT', 'NN', 'VBD', '.', '--s--', '``', 'PRP', 'MD', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'NN', 'NNS', '.', "''", '--s--', 'TO', 'VB', 'RP', 'NNP', 'NNP', 'POS', 'NNS', ',', 'NNP', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'WDT', 'MD', 'VB', 'PRP', 'TO', 'DT', 'NN', 'CC', 'JJ', 'NNS', 'NN', '.', '--s--', 'NNP', 'NNS', 'IN', 'NNP', 'NNPS', 'NNP', 'VBD', 'DT', 'NNP', 'NN', 'CC', 'NN', 'NN', 'RB', 'IN', 'JJ', 'NNS', 'IN', 'JJ', 'NNP', ',', 'DT', 'JJ', 'NNS', 'NN', 'IN', 'DT', 'CD', 'NN', 'NN', '.', '--s--', 'DT', 'CD', 'JJ', 'JJ', 'NN', 'NNS', 'VBD', 'CD', 'JJ', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'IN', 'NNP', ',', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', 'RBR', '.', '--s--', 'NNP', 'VBD', 'NN', 'NNS', 'VBD', 'RB', 'CD', 'NN', 'TO', 'CD', 'NNS', ',', 'NNS', 'CC', 'NN', 'NN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'TO', 'VB', 'RP', 'CD', 'NNS', 'VBZ', 'TO', 'VB', 'VBN', 'DT', 'NN', ',', 'IN', 'JJS', 'RB', ',', 'RB', 'IN', 'CD', 'NN', 'NNS', ',', 'WDT', 'VBD', 'RB', 'NNP', 'CD', '.', '--s--', 'DT', 'VBZ', 'RB', 'JJ', 'IN', 'NNP', ',', 'WDT', 'VBD', 'JJ', 'NNS', 'IN', 'NNP', 'CC', 'NNP', 'CC', 'VBD', 'PRP$', 'JJ', 'NNP', 'NN', 'CC', 'NN', 'NNS', 'VBP', 'CD', 'NN', 'IN', 'JJ', 'NN', 'VBZ', 'RB', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'NNS', 'IN', 'DT', 'JJ', 'NNS', 'VBD', 'CD', 'NN', 'IN', 'DT', 'NN', 'RB', '.', '--s--', 'IN', 'NNP', ',', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NNP', 'NNS', 'VBD', 'RB', 'JJ', 'IN', 'CD', 'NNS', '.', '--s--', 'DT', 'IN', 'DT', 'NNP', 'NN', 'NNS', 'VBP', 'RB', 'VBN', 'NNS', 'IN', 'JJ', 'CD', 'NNS', ',', 'CC', 'PRP', 'MD', 'VB', 'TO', 'VB', 'PRP$', 'NNS', 'TO', 'VB', 'NNS', 'RB', '.', '--s--', '``', 'PRP', 'VBP', 'VBN', 'DT', 'NN', 'WRB', ',', 'IN', 'NNS', ',', 'PRP', 'VBZ', 'DT', 'JJ', 'NN', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'NNS', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ',', 'NNP', '.', '--s--', 'NNP', 'NNS', 'VBD', 'TO', 'DT', 'RB', 'VBN', 'JJ', 'NN', 'NN', 'IN', 'CD', 'CD', 'NNS', ',', 'DT', 'JJS', 'IN', 'NNP', 'CD', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'RB', 'CD', 'CD', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'NNS', 'VBD', 'RB', 'JJ', 'NN', 'VBD', 'PRP', 'MD', 'RB', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'NN', '.', '--s--', 'NNP', 'POS', 'JJ', 'NN', 'NNS', 'VBD', 'CD', 'NN', 'CC', 'PRP$', 'JJ', 'NNS', 'VBD', 'RP', 'DT', 'RB', 'JJR', 'CD', 'NN', 'IN', 'DT', 'JJ', 'NN', 'DT', 'NN', 'RB', '.', '--s--', 'DT', 'IN', 'DT', 'NNP', 'NNS', 'IN', 'NNP', 'VBD', 'JJ', 'NNS', '.', '--s--', 'NNP', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'NN', 'IN', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNS', 'VBD', 'RB', 'JJ', ':', 'DT', 'NNS', 'VBP', 'VBN', 'CC', 'NNP', 'VBZ', 'PRP$', 'NNS', 'RB', 'IN', 'JJ', '.', '--s--', 'DT', 'NNS', 'NN', 'IN', 'DT', 'NN', 'CD', 'NN', 'NN', 'MD', 'VB', 'VBN', 'VBN', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'VBD', 'IN', 'NN', 'TO', 'NN', 'NNS', 'CC', 'JJ', 'NN', ',', 'DT', 'NN', 'NN', 'VBD', '.', '--s--', 'JJ', 'NN', ',', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'NN', 'WDT', 'VBD', 'JJ', 'NNS', 'IN', 'PDT', 'DT', 'CD', 'NNS', 'VBD', 'VBN', 'VBN', '.', '--s--', 'RB', 'IN', 'NNP', ',', 'JJ', 'NN', 'NNS', 'VBD', 'RB', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'CC', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'NN', 'NNS', ',', 'VBG', 'PRP$', 'RB', 'JJ', 'NN', 'DT', 'NN', '.', '--s--', 'RB', ',', 'NNS', 'IN', 'NNS', ',', 'VBG', 'DT', 'NN', 'POS', 'JJ', 'NNS', ',', 'VBD', 'CD', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'POS', 'NNS', 'IN', 'RB', 'VBN', 'NNS', 'VBD', 'CD', 'NN', 'IN', 'DT', 'NN', 'RBR', '.', '--s--', 'NNP', 'POS', 'NN', 'IN', 'NNP', ',', 'NNP', ',', 'VBD', 'VBG', 'RP', 'TO', 'VB', 'CD', 'NN', 'NNP', ',', 'DT', 'NNP', 'NN', 'VBD', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'JJ', 'NN', 'MD', 'VB', 'RP', 'TO', 'JJ', ',', "''", 'PRP', 'VBD', '.', '--s--', 'RB', ',', 'NNP', 'VBD', 'JJ', 'NNS', 'IN', 'PRP$', 'JJ', 'JJ', 'NNS', 'CC', 'NNS', 'MD', 'VB', 'DT', 'NN', 'IN', 'CD', 'NN', 'IN', 'NNS', 'VBD', 'CD', 'NNS', '.', '--s--', 'NN', 'NNS', 'VBD', 'RB', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'VBD', 'RBR', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'VBD', 'JJ', 'NNS', ',', 'WDT', 'VBP', 'RB', 'VBN', 'IN', 'NN', 'NNS', ',', 'MD', 'VB', 'IN', 'CD', 'NN', 'CC', 'CD', 'NN', 'IN', 'JJS', 'NN', '.', '--s--', 'SYM', ':', 'NNP', 'VBP', 'JJ', 'NN', 'NNS', 'VBD', 'IN', 'NN', '.', '--s--', 'SYM', ':', 'JJ', 'NN', '--s--', 'SYM', ':', 'NN', 'NN', 'VBZ', 'JJR', 'IN', 'CD', 'NN', '.', '--s--', 'SYM', ':', 'EX', 'VBD', 'CD', 'NN', 'NNS', 'IN', 'DT', 'RBS', 'JJ', 'NN', 'CC', 'CD', 'DT', 'NN', 'RBR', '.', '--s--', 'NN', 'NNS', 'VBN', 'IN', 'JJ', 'NNS', 'NN', 'RB', 'IN', 'NNS', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'VBD', 'VBN', 'NN', 'CC', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'VBN', 'DT', 'NN', 'TO', 'DT', 'NN', 'POS', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBD', 'TO', 'VB', 'JJ', 'NNS', '.', '--s--', 'NNP', 'VBZ', 'CC', 'NNS', 'NN', 'NNS', '.', '--s--', 'PRP', 'RB', 'VBZ', 'NNS', 'IN', 'JJ', 'NNS', 'CC', 'JJ', ',', 'JJ', 'NN', 'CC', 'NN', 'VBN', 'TO', 'VB', 'NN', 'CC', 'NN', 'NNS', '.', '--s--', 'PRP', 'VBD', 'RB', 'RB', 'RB', 'RB', 'IN', 'DT', 'NN', 'NN', 'VBN', 'IN', 'DT', 'NNP', 'NNP', ':', 'CC', 'RB', 'IN', 'DT', 'NNP', 'NNP', 'NNP', ':', 'VBD', 'VBN', 'IN', 'NNS', 'RB', 'IN', 'VBG', 'NNS', ',', 'NNS', 'CC', 'NNS', '.', '--s--', 'RB', ',', 'DT', 'NN', 'VBZ', 'VBN', 'DT', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'TO', 'VB', 'CC', 'RB', 'VBN', 'RP', 'DT', 'NN', ':', '``', 'NNP', 'NNP', 'NNP', "''", ':', 'NNP', 'NNP', 'NNP', '.', '--s--', '``', 'PRP', 'VBP', 'DT', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'JJ', 'NN', 'WP', 'VBZ', 'IN', 'DT', 'NNP', ',', 'NNP', 'NNP', ',', 'NN', 'IN', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'IN', 'NN', ',', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'RP', 'NNS', 'IN', 'NNP', 'CC', 'RB', 'RB', 'IN', 'DT', 'NNP', 'NNP', 'WDT', 'MD', 'VB', 'PRP', '.', '--s--', 'CC', 'DT', 'JJ', 'NNS', 'VBN', 'IN', 'IN', 'NN', 'CC', 'JJ', 'NNS', 'VBP', 'VBG', 'DT', 'NN', 'POS', 'NN', 'IN', 'JJR', 'NNS', 'IN', 'CD', '.', '--s--', 'IN', 'NN', ',', 'NNP', 'NNP', 'NNP', 'VBZ', 'IN', 'NN', 'IN', 'VBG', 'IN', 'PRP$', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'RB', 'MD', 'VB', 'RB', 'IN', 'NNP', ',', 'RB', 'MD', 'NNS', '.', '--s--', 'IN', 'NN', ',', 'NN', 'IN', 'NNS', 'VBZ', 'VBG', 'JJR', 'IN', 'JJ', 'NNS', 'IN', 'WRB', 'VBG', 'RB', 'VBD', 'RB', 'VBN', '.', '--s--', 'RB', 'IN', 'VBG', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', ',', 'NNP', 'NNP', 'NNP', 'VBZ', 'RBR', 'JJ', 'TO', 'VB', 'DT', 'NN', 'IN', 'PRP$', 'NNS', 'VBP', 'RB', 'JJ', ',', 'RB', 'JJ', '.', '--s--', '``', 'PRP', 'VBP', 'DT', 'NN', 'TO', 'VB', 'DT', 'NNS', 'TO', 'VB', 'IN', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', '``', 'DT', 'JJ', '(', 'NN', ')', 'NNS', 'VBP', 'RB', 'VBN', 'CC', 'RBR', 'JJ', 'IN', 'IN', 'WP', 'VBZ', 'VBG', 'IN', 'RB', '.', "''", '--s--', 'RB', ',', 'NNP', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'NNPS', 'IN', 'JJ', 'NN', 'NNS', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NNS', '.', '--s--', 'NNP', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', '.', '--s--', 'NNP', 'CC', 'NN', 'VBP', 'VBG', 'NNS', 'CC', 'NNS', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'CC', 'DT', 'NNP', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'NNP', 'NN', 'IN', 'JJ', 'NNS', ',', 'RB', 'RB', 'DT', 'JJ', 'NNS', 'DT', 'NN', 'DT', 'IN', 'JJ', '.', '--s--', 'JJ', 'NN', 'NN', 'NNP', 'NNP', 'VBZ', 'VBN', 'CD', 'NN', 'NNS', 'IN', 'CD', 'JJ', 'CC', 'JJ', 'JJ', 'NNS', ',', 'IN', 'NNP', 'POS', 'NNP', 'NNP', 'VBZ', 'VB', 'PRP', 'VB', 'VBN', 'RB', 'IN', 'PRP', 'VBZ', 'VBG', 'IN', 'JJ', 'NNS', '.', '--s--', 'CC', 'NNP', 'NNP', 'NNP', 'VBZ', 'RB', 'VBP', 'TO', 'VB', 'RP', '.', '--s--', 'IN', 'PRP$', 'NN', 'IN', 'JJ', 'NN', 'CC', 'DT', 'JJ', 'NN', ',', 'DT', 'NN', 'POS', 'NN', 'VBZ', 'PRP', 'RB', 'VBZ', 'RB', 'TO', 'VB', ':', 'IN', 'NNP', 'CC', 'IN', 'DT', '``', 'NNP', 'NNP', 'NNP', '.', "''", '--s--', 'NNP', 'NNP', 'NNP', 'CC', 'PRP$', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', ',', 'NNP', 'NNP', ',', 'VBP', 'PRP', 'MD', 'RB', 'VB', 'RP', 'PRP$', 'JJR', 'IN', 'CD', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'PRP', 'VBP', 'RB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'IN', 'CD', 'CD', 'NNS', 'IN', 'CD', 'JJ', 'NNS', ':', 'IN', 'JJ', 'CC', 'NNP', 'TO', 'NNP', 'CC', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'IN', 'DT', 'NNS', 'IN', 'NNP', 'NNP', '.', '--s--', 'NNS', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NNP', 'NNP', 'TO', 'VB', 'PRP', 'JJ', 'CC', 'VB', 'TO', 'VB', 'RBR', 'JJ', 'TO', 'NNS', ',', 'DT', 'NN', 'POS', 'NNS', 'VBD', 'VBG', 'TO', 'VB', 'RBR', 'JJ', '.', '--s--', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'TO', 'VB', 'IN', 'NN', 'TO', 'VB', 'RP', 'NN', '.', '--s--', 'RB', 'RB', 'VBD', 'DT', 'JJ', 'NNP', 'IN', 'NNP', 'NNPS', 'VBP', 'NNP', 'NNP', 'NNP', 'POS', 'JJ', 'NN', ',', 'CC', 'DT', 'NNP', 'IN', 'NNP', 'VBD', 'NN', 'RB', 'RBR', 'JJ', ':', '``', 'PRP', 'VBD', 'PRP', 'CD', 'NN', 'NNS', 'IN', 'JJ', 'NNP', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'WP', 'VBD', 'VB', 'DT', 'NNP', 'NN', '.', '--s--', '``', 'DT', 'VBZ', 'DT', 'NN', '.', "''", '--s--', 'PRP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', ',', 'WDT', 'VBD', 'TO', 'VB', 'VBN', 'IN', 'PRP$', 'NNS', 'CC', 'NNS', 'IN', 'JJ', 'NNS', '(', 'WP', 'VBD', 'NNS', 'TO', 'DT', 'NN', 'IN', 'NN', ',', 'JJ', 'NNS', 'CC', 'NNS', ')', '.', '--s--', 'IN', 'NN', ',', 'DT', 'IN', 'DT', 'NN', 'POS', 'JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'PRP', 'VBD', 'JJ', 'IN', 'JJ', 'NNS', 'IN', 'VBG', 'NN', 'JJ', 'IN', 'WP', 'CD', 'NN', 'RB', 'VBZ', 'DT', '``', 'NNP', 'NNP', 'IN', 'DT', 'JJ', 'NNP', '.', "''", '--s--', '``', 'IN', 'DT', 'JJ', 'CD', 'NNS', ',', 'JJ', 'IN', 'PRP', 'VBP', 'VBN', 'RP', 'IN', 'JJ', 'IN', 'NN', 'VBG', 'TO', 'PRP$', 'NNS', ',', "''", 'VBZ', 'DT', 'NN', '.', '--s--', '``', 'EX', 'VBD', 'DT', 'RB', 'JJ', 'NNS', '.', "''", '--s--', 'DT', 'NNS', ',', 'RB', ',', 'VBD', 'TO', 'VB', 'JJ', '.', '--s--', 'IN', 'JJ', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', ',', 'VBG', 'TO', 'JJ', 'NNS', 'VBD', 'DT', 'NN', 'IN', 'NNP', '.', '--s--', '``', 'WRB', 'PRP', 'VBP', 'TO', 'DT', 'NNP', 'NN', ',', 'PRP$', 'NN', 'RB', 'VBZ', 'JJ', ',', "''", 'VBZ', 'DT', 'NNP', 'NN', '.', '--s--', '``', 'PRP', 'VBZ', 'TO', 'VB', 'RP', 'DT', 'NN', 'CC', 'VB', 'DT', 'NN', '.', "''", '--s--', 'RB', ',', 'DT', 'JJS', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'JJ', 'NN', 'RB', 'VBZ', 'PRP$', 'RBS', 'RB', 'JJ', 'NNS', 'IN', 'IN', 'NN', '.', '--s--', 'NNP', ',', 'WDT', 'JJS', 'IN', 'DT', 'NN', 'VBZ', 'VBN', 'RB', 'JJ', ',', 'VBZ', 'VBN', 'DT', 'VBG', 'IN', 'DT', 'JJ', 'NN', ',', '``', 'DT', 'NN', 'IN', 'DT', 'NNP', ',', "''", 'WDT', 'VBZ', 'IN', 'IN', 'CD', 'NN', '.', '--s--', 'PRP', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'NN', 'NNS', 'CC', 'JJ', 'NNS', 'WP', 'VB', 'NNP', 'POS', 'JJ', 'NNS', 'IN', 'RB', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'WP', 'VBP', 'TO', 'VB', 'RB', 'RBR', 'JJ', 'NNS', 'VBP', 'TO', 'VB', 'RP', 'IN', 'CD', 'IN', 'NNP', 'NN', 'IN', '``', 'NNP', 'NNP', ',', "''", 'IN', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'JJ', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'VBZ', 'VBG', 'TO', 'VB', 'RP', 'NNS', '.', '--s--', 'DT', 'NNP', 'NN', 'VBZ', 'TO', 'VB', 'JJ', 'NNS', 'TO', 'VB', 'DT', 'JJS', 'NNS', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'NNP', 'IN', 'DT', 'NN', 'TO', 'VB', 'IN', 'DT', 'JJ', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'DT', 'JJ', 'CD', 'NN', 'NN', 'VBD', 'NNP', ',', 'IN', 'JJ', 'CC', 'JJ', 'NN', ',', 'CC', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NNS', 'IN', 'IN', 'DT', 'NN', '.', '--s--', 'EX', 'VBZ', 'RB', 'DT', 'NN', 'IN', 'JJ', 'NNS', ',', 'VBG', 'TO', 'VB', 'RP', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', '.', '--s--', 'DT', 'JJ', 'CD', 'NN', 'VBD', '``', 'DT', 'NNP', 'IN', 'NNP', "''", 'VBZ', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'JJ', 'NN', 'CC', 'NN', 'IN', 'IN', 'DT', 'NN', '.', '--s--', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'NNS', 'CC', 'DT', 'NN', 'NN', '.', '--s--', 'CC', 'TO', 'VB', 'JJR', 'NNS', ',', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'JJS', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNPS', 'VBP', 'JJ', 'DT', 'NN', 'IN', 'NNP', '.', '--s--', '``', 'PRP', 'VBP', 'VBG', 'IN', 'PDT', 'DT', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'NN', '.', '--s--', '``', 'DT', 'NN', 'PRP', 'VBP', 'TO', 'VB', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'IN', 'JJ', '``', 'JJ', 'NN', ',', "''", 'IN', 'NNS', ',', 'VBG', 'RP', 'IN', 'NN', 'TO', 'NN', ',', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'NNS', ',', 'VB', 'PRP$', 'NN', ',', 'CC', 'VBP', 'TO', 'VB', ',', 'RB', 'TO', 'VB', 'CC', 'JJ', 'JJ', 'NN', 'RB', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'TO', 'VB', ',', 'IN', 'NN', ',', 'RB', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'PRP', 'VBZ', 'VBN', 'IN', 'NN', 'IN', 'VBG', 'TO', 'VB', 'RB', 'VBG', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', 'NN', 'VBD', 'VBN', 'VBN', 'IN', 'DT', 'JJ', 'NNS', ',', 'RB', 'IN', 'NNS', '.', '--s--', 'PRP', 'VBZ', 'RB', 'IN', 'PRP', 'RB', ',', 'IN', 'DT', 'NN', 'IN', 'NN', 'TO', 'VB', 'NNS', 'IN', 'DT', '``', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', ',', "''", 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'DT', 'NN', '.', '--s--', 'WRB', 'VBZ', 'DT', 'JJ', 'NN', 'VBP', 'VBG', 'RP', '.', '--s--', 'RB', 'PRP', 'VBZ', 'IN', 'JJ', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'CC', 'DT', 'JJ', 'NN', '.', '--s--', 'IN', 'RB', 'PRP', 'MD', 'VB', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'IN', 'JJS', 'DT', 'JJ', 'NN', 'PRP', 'MD', 'VB', 'JJ', 'TO', 'VB', 'JJ', 'RB', 'CC', 'VB', 'DT', 'NN', 'IN', 'VBG', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'VBG', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NN', ':', 'IN', ',', 'IN', 'DT', 'NNS', ',', 'VBG', 'DT', 'NN', '.', '--s--', 'DT', 'IN', 'PRP', 'WP', 'VBD', 'DT', 'NN', 'RB', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'VBZ', 'RB', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'JJ', 'JJ', 'NNS', ',', 'IN', 'PRP$', 'NNS', 'IN', 'JJ', 'NNS', 'VBP', 'DT', 'NN', '.', '--s--', 'NNP', 'IN', 'DT', 'NN', 'NN', 'IN', 'DT', 'NNS', 'VBD', 'TO', 'VB', 'JJ', 'NN', 'IN', 'JJ', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'DT', 'NN', 'VBD', 'VBN', 'JJ', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNS', 'RB', ',', 'WRB', 'PRP', 'RB', 'VBD', 'VB', 'JJ', '.', '--s--', 'DT', 'NN', 'PRP', 'VBD', 'TO', 'VB', 'CD', 'JJ', 'NNS', ',', 'DT', 'VBG', 'NN', 'IN', 'JJR', 'NN', 'CC', 'DT', 'VBG', 'NN', 'TO', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'CC', 'NNS', '.', '--s--', 'WRB', 'RB', 'VB', 'CC', 'RB', 'VB', 'DT', 'NNS', 'CC', 'NNS', ',', 'DT', 'NNS', 'VBD', ',', 'CC', 'VB', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'JJ', 'NN', ',', 'RB', ',', 'MD', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'RB', 'PRP', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'MD', 'VB', 'JJ', ',', 'VBG', 'JJ', 'NNS', ',', 'JJ', 'NNS', ',', 'CC', 'JJ', 'NNS', '.', '--s--', 'DT', 'JJ', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'VBG', 'DT', 'NN', ',', 'RB', 'VBD', 'PRP', 'MD', 'VB', 'DT', 'NN', 'TO', 'VB', 'NN', ',', 'RB', 'IN', 'NN', 'NNS', 'CC', 'PRP$', 'NNS', '.', '--s--', 'DT', 'NN', 'VBG', 'NNS', 'TO', 'VB', 'VBN', 'TO', 'JJ', 'NNS', 'VBD', 'VBN', 'TO', 'VB', 'TO', 'DT', 'WP', 'VBD', 'VBN', 'RB', 'IN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', '``', 'NNS', "''", 'VBG', 'NNS', ',', 'VBG', 'NNS', ',', 'VBG', 'NNS', ',', 'CC', 'VBG', 'NNS', 'IN', '$', 'CD', 'DT', 'NN', ',', 'NN', 'JJ', ',', 'CC', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', '$', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'DT', 'NNS', 'MD', 'VB', 'VBG', 'DT', 'NN', 'JJ', 'TO', '$', 'CD', 'DT', 'NN', '.', '--s--', 'NNP', 'PRP', ',', 'JJS', 'IN', '``', 'DT', 'NNS', "''", 'MD', 'VB', 'VBN', 'CD', ':', 'TO', 'CD', ',', 'DT', 'RB', 'RB', 'JJ', 'NN', 'NNS', ',', 'CC', 'JJ', 'NN', 'NN', 'IN', 'VBG', 'IN', 'NN', '.', '--s--', 'PRP', 'MD', 'VB', 'VBG', 'RBR', 'RB', 'IN', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'VBG', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'NNS', 'CC', 'RB', 'VBG', 'NNS', '.', '--s--', 'IN', 'PRP', 'VBD', ',', 'JJ', 'NN', 'VBN', 'IN', 'JJ', 'CC', 'NN', 'NNS', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'NN', ',', 'RB', 'DT', 'JJ', 'NNS', 'VBD', 'TO', 'VB', 'DT', 'RB', 'JJ', 'NN', 'IN', '``', 'VB', 'IN', 'NN', 'TO', 'VB', '.', "''", '--s--', 'NNP', 'TO', 'JJ', 'NN', 'IN', 'DT', 'NNP', ',', 'WDT', 'VBZ', 'TO', 'VB', 'PRP$', 'JJ', 'NN', 'NN', ',', 'RB', 'VBN', 'TO', 'DT', 'JJ', 'NN', 'VBG', 'VBN', '.', '--s--', 'RB', ',', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'VBD', 'TO', 'VB', 'VBN', 'RP', '.', '--s--', 'WP', 'JJR', 'NN', 'TO', 'VB', 'IN', 'NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NNS', 'IN', 'NN', 'NNS', ',', 'WRB', 'PDT', 'DT', 'JJ', 'NN', 'IN', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'PRP', 'MD', 'VB', 'VBN', 'RP', 'CC', 'VBN', 'IN', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'VBN', 'IN', 'VBG', 'JJ', 'JJ', 'JJ', 'NNS', 'IN', 'CD', 'NNS', ':', 'DT', 'NN', 'NN', 'IN', 'JJ', 'NN', '(', 'RB', 'VBG', 'TO', 'DT', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', ')', ',', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NN', 'NN', '(', 'FW', 'FW', 'NNP', 'NNP', 'NNP', ')', ',', 'DT', 'JJ', 'NNS', 'NN', 'TO', 'NNS', 'TO', 'VB', 'NN', 'CC', 'JJ', 'NN', 'NN', 'NNS', '(', 'DT', 'NNP', 'NN', ')', ',', 'DT', 'NN', 'NN', 'JJ', 'NN', 'IN', 'NNS', '(', 'VBG', 'TO', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'IN', 'NNS', ')', ',', 'CC', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'CC', 'DT', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', ',', 'CC', 'NNP', 'NNP', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'VBZ', 'VBN', ',', 'DT', 'NNP', 'MD', 'VB', 'JJR', 'NNS', ',', 'RB', 'VBG', 'DT', 'JJ', 'NN', ',', 'RB', 'VBD', 'IN', 'DT', '$', 'CD', 'CD', '.', '--s--', 'PRP', 'VBZ', 'RB', 'VBN', 'IN', '``', 'RB', 'JJ', 'NNS', 'VBD', 'DT', 'NN', ',', "''", 'CC', 'DT', 'NN', 'VBZ', 'RB', 'RB', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'RB', 'IN', 'PRP', 'VBP', 'NNS', '.', '--s--', 'DT', '``', 'JJ', "''", 'NN', 'VBG', 'RP', 'IN', 'NNP', 'MD', 'VB', 'JJ', 'NNS', ',', 'CC', 'DT', 'JJ', 'NNS', 'VBP', 'JJ', 'CC', 'DT', 'NN', 'VBZ', 'VBG', 'DT', 'NN', 'RB', 'JJ', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'VBZ', 'IN', 'DT', 'NN', '.', '--s--', 'VBZ', 'DT', 'JJ', 'NN', 'RB', 'JJ', 'IN', 'PRP', 'VBZ', 'JJ', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'PRP', 'VBZ', 'RB', 'JJ', 'IN', 'RB', '.', '--s--', 'VBG', 'TO', 'DT', 'NNP', 'NNP', ',', 'JJ', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'CD', 'NNS', 'DT', 'NN', 'IN', 'NN', ',', 'IN', 'JJ', 'NNS', 'TO', 'NN', 'IN', 'DT', 'NNS', 'VBP', 'VBN', 'CD', 'NN', '(', 'VBN', 'IN', 'NN', ')', '.', '--s--', 'RB', 'IN', 'NN', 'VBZ', 'VB', 'JJ', '``', 'JJ', 'NNS', ',', "''", 'JJ', 'NN', 'VBZ', 'RB', 'DT', 'NN', 'TO', 'VB', 'PRP', '.', '--s--', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'NNS', ',', 'PRP', 'MD', 'VB', 'DT', 'NN', 'VBN', 'IN', 'JJ', 'NNS', 'IN', 'VBG', 'JJR', 'NNS', 'VBN', 'IN', 'NN', 'VBD', '``', 'NN', ',', "''", 'RB', 'IN', 'IN', 'DT', 'NN', 'POS', 'NN', 'IN', '``', 'NN', '.', "''", '--s--', 'CC', 'PRP', 'MD', 'VB', 'DT', 'NN', 'NN', 'IN', 'VBG', 'NNS', '.', '--s--', 'WP', 'PRP', 'VBP', 'RB', 'VBP', 'TO', 'VB', 'VBZ', 'VB', 'DT', 'NN', ',', 'CC', 'RB', 'VB', 'TO', 'VB', 'PRP', 'IN', 'VBG', 'DT', 'NNP', 'NNP', '.', '--s--', 'TO', 'DT', 'NN', 'PRP', 'VBP', 'NN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNS', ',', 'IN', 'NN', ',', 'PRP', 'MD', 'VB', 'NN', ',', 'VB', 'JJ', 'NN', ',', 'CC', 'VB', 'RBR', 'JJ', 'NN', ',', 'RB', 'IN', 'JJ', 'JJ', 'NN', 'NNS', 'IN', 'JJ', 'NNS', 'CC', 'VB', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'JJR', 'NNS', 'TO', 'VB', 'DT', 'JJ', 'NNS', '.', '--s--', 'NNP', 'JJ', 'NN', ',', 'IN', 'DT', 'JJ', 'RB', 'VBN', 'CC', 'JJ', 'NN', 'VBZ', 'RB', 'VB', 'TO', 'VB', 'VBN', '.', '--s--', 'RB', 'RB', '.', '--s--', 'NNS', 'VBP', 'JJ', ',', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', ',', 'CC', 'DT', 'NN', 'RB', 'IN', 'NN', 'VBZ', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'NNP', 'NNPS', 'VBP', 'VBG', 'DT', 'NN', 'IN', 'VBG', '``', 'NNS', ',', "''", '``', 'NNS', "''", 'CC', '``', 'NNS', ',', "''", 'DT', 'IN', 'WDT', 'MD', 'VB', 'TO', 'VB', 'DT', 'VBN', '``', 'NNS', '.', "''", '--s--', 'RB', 'RB', ',', 'DT', 'NN', 'VBZ', 'JJ', 'TO', 'VB', 'DT', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'JJ', 'JJ', 'NN', 'TO', 'VB', 'DT', '``', 'NNP', 'IN', 'NNP', "''", 'NN', 'IN', '$', 'CD', 'CD', 'TO', 'VB', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'VBD', 'PRP', 'TO', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'RB', 'DT', 'JJ', 'NN', 'MD', 'VB', 'JJ', 'IN', 'DT', 'NNP', 'NNP', 'VBZ', 'IN', 'TO', 'JJ', 'JJ', 'NN', 'TO', 'VB', 'TO', 'DT', 'NN', 'IN', 'DT', 'NN', 'POS', 'NN', 'CC', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', '``', 'FW', "''", 'JJ', 'NNS', 'IN', 'NN', 'NNS', '.', '--s--', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'VBN', ',', 'DT', 'NN', 'VBN', ',', 'CC', 'JJ', 'NN', 'VBN', '.', '--s--', 'NNP', 'NN', ',', 'DT', 'JJ', 'NN', ',', 'RB', 'MD', 'RB', 'VB', 'RP', 'RB', ',', 'IN', 'NN', ',', 'CC', 'DT', 'NNS', 'IN', 'JJ', 'NN', 'VBG', 'NNS', 'IN', '``', 'JJ', 'NN', "''", 'MD', 'VB', 'VBN', 'RB', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBN', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'JJ', 'NN', ',', 'IN', 'WDT', 'NNP', 'NNP', 'RB', 'VBD', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'WP', 'VBD', 'VBN', 'NN', 'CC', 'JJ', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'VBD', 'VBN', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'PRP', 'VBZ', 'PRP$', 'NN', ',', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBD', 'DT', 'NN', 'CC', 'VBZ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'CD', ',', 'WP', 'VBD', 'DT', 'NN', 'POS', 'NNP', 'NN', ',', 'VBD', 'VBN', 'NN', 'CC', 'JJ', 'NN', 'NN', ',', 'VBG', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'IN', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', 'NNP', 'IN', 'IN', '$', 'CD', 'CD', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'RB', 'VBD', 'CD', 'NNP', 'NNP', 'NNS', 'IN', 'NNP', 'POS', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBZ', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'TO', 'NNS', 'CC', 'NNS', '.', '--s--', 'NNP', ',', 'VBN', 'IN', 'NNP', ',', 'NNP', ',', 'VBZ', 'NN', 'NNS', 'TO', 'JJ', 'NN', 'NNS', 'CC', 'JJ', 'NN', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'VBD', 'NNP', 'CD', ',', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'NNS', 'IN', '$', 'CD', 'CD', '.', '--s--', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'VBN', 'RB', 'TO', 'CD', 'NN', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', 'NN', ',', 'DT', 'NNP', 'NNPS', 'NNP', 'VBD', '.', '--s--', 'DT', 'NN', 'NN', 'VBD', 'DT', 'JJS', 'NN', 'VBD', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', 'VBN', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'JJ', 'IN', 'CD', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'IN', 'CD', 'NN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'PRP', 'MD', 'VB', 'NNP', 'NN', 'IN', 'DT', 'IN', 'CD', 'IN', 'CD', 'NN', ',', 'DT', 'NN', 'VBD', '.', '--s--', 'DT', 'MD', 'VB', 'RP', 'IN', 'DT', 'CD', 'NN', 'NN', 'VBD', 'IN', 'CD', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'DT', 'JJ', ',', 'JJ', 'NNP', 'NNP', 'NNP', 'NN', 'WDT', 'MD', 'VB', 'NNS', 'CD', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'NN', 'IN', 'DT', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'MD', 'VB', 'VBN', 'DT', 'NN', ',', 'WRB', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'VBP', 'VBN', '.', '--s--', 'NNP', 'NNP', 'NNPS', 'VBP', 'JJ', 'NN', 'NNS', 'IN', 'VBG', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'CD', 'CD', 'JJ', 'NNS', '(', '$', 'CD', 'CD', ')', 'IN', 'JJ', 'NNS', 'RB', 'JJ', '.', '--s--', 'RB', 'JJ', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'NNP', 'NNP', 'NNPS', ',', 'WDT', 'MD', 'VB', 'VBN', 'DT', 'NN', 'IN', 'NN', 'NN', '.', '--s--', 'DT', 'NNS', 'VBP', 'IN', 'NN', 'NNP', 'CD', '.', '--s--', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'VBZ', 'VBN', 'RB', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'WDT', 'JJR', 'IN', 'CD', 'NNS', 'VBD', 'PRP$', 'JJ', 'NN', 'CC', 'NN', 'NN', 'CC', 'NNS', 'IN', 'NN', 'NNS', 'VBN', 'NN', 'NN', 'DT', 'NN', 'TO', 'JJ', 'NN', ',', 'DT', 'NN', 'VBZ', 'VBN', 'TO', 'NNP', '.', '--s--', 'IN', 'JJS', 'CD', 'NNS', 'VBG', 'CD', 'CC', 'DT', 'JJ', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'DT', 'NN', '.', '--s--', 'CD', ',', 'VBN', 'IN', 'NNP', 'NNP', 'NNP', '(', 'NNP', ',', 'NNP', '.', ')', 'CC', 'NNP', 'NNP', 'NNP', '(', 'NNP', ',', 'NNP', ')', ',', 'MD', 'VB', 'VBN', 'JJ', 'NN', 'NNS', 'TO', 'NNS', 'WP', 'VBD', 'VBN', '.', '--s--', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'NNP', 'NNP', 'NNP', '(', 'NNP', ',', 'NNP', ')', ',', 'CC', 'VBG', 'DT', 'JJ', 'JJ', 'NNS', 'IN', 'IN', 'NNS', 'IN', 'JJ', 'JJ', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'VBZ', 'VBN', 'VBN', 'RP', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'PRP', 'MD', 'RB', 'VB', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'VBN', 'PRP$', 'JJ', 'NNP', 'NNP', 'NNP', '(', 'NNP', ')', 'NN', ',', 'IN', 'PRP$', 'NNS', 'VBP', 'TO', 'VB', 'VBN', '.', '--s--', 'WP', 'VBZ', 'CD', 'TO', 'VB', 'IN', 'PDT', 'DT', '.', '--s--', 'NNP', 'CC', 'JJ', 'NNS', 'VBP', 'DT', 'NNS', '.', '--s--', 'NNS', 'JJ', 'TO', 'VB', 'NN', '``', 'VB', 'PRP$', 'NNS', 'TO', 'NN', "''", 'NN', 'NN', 'NNS', ':', 'RB', 'JJ', 'NNS', '.', '--s--', 'RB', 'VB', 'DT', 'WP', 'VBP', 'DT', '``', 'JJ', 'NN', 'IN', 'NNS', ',', "''", 'DT', 'NN', 'VBG', 'JJ', 'NNS', 'IN', 'RB', 'IN', 'VBG', 'NNS', '.', '--s--', 'RB', 'EX', 'VBP', 'JJ', 'NNS', '.', '--s--', 'TO', 'VB', ',', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'CC', 'JJ', 'NNS', 'VBP', 'JJ', 'NNS', 'IN', 'NN', 'NN', '.', '--s--', 'NNP', 'IN', 'DT', 'NN', 'NN', 'DT', 'NN', 'IN', 'DT', 'NN', 'JJ', 'NN', 'MD', 'VB', ':', '--s--', 'IN', 'DT', 'NN', 'MD', 'RB', 'VB', 'IN', 'PRP', ',', 'PRP', 'VBP', ',', 'PRP', 'MD', 'RB', 'VB', 'IN', 'PRP$', 'NN', '.', '--s--', 'NNP', 'IN', 'DT', 'NN', 'VBP', 'RB', 'VBZ', 'VBN', ':', 'PRP', 'VBP', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'CC', 'VBP', 'DT', 'NN', 'IN', 'NN', ',', 'CC', 'VBP', 'TO', 'VB', 'NNS', 'NNS', ',', 'RB', 'NNS', '.', '--s--', 'CC', 'WP', 'IN', 'DT', 'IN', 'PRP', 'WP$', 'NNS', 'VBP', 'RB', 'VBN', 'IN', 'NN', 'CC', 'NN', '.', '--s--', 'WRB', 'MD', 'PRP', 'VBP', 'RB', 'JJ', 'NN', '.', '--s--', 'NNP', 'POS', 'VB', 'IN', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'NN', ':', '``', 'JJ', 'NN', "''", 'VBZ', 'RB', 'VBN', 'NN', '.', '--s--', 'MD', 'VB', 'VB', 'JJ', 'CC', 'NNS', '.', '--s--', 'NNP', 'CC', 'RB', '.', '--s--', 'NNP', 'CC', 'JJ', '.', '--s--', 'NNP', 'CC', 'JJ', '.', '--s--', '``', 'NNS', 'VBP', 'IN', 'NN', 'CC', 'VB', 'JJ', 'CC', 'VB', 'IN', 'NNS', 'CC', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'WP', 'NNS', 'IN', 'NN', 'MD', 'PRP', 'VBP', '.', '--s--', 'WP', 'VBZ', '``', 'JJ', "''", 'NN', '.', '--s--', 'PDT', 'DT', 'NN', 'VB', 'VBN', 'IN', 'DT', 'JJ', 'NN', ',', 'IN', 'JJ', 'NNS', ',', 'CC', 'IN', 'JJ', 'JJ', 'NNS', '.', '--s--', 'CC', 'WP', 'MD', 'VB', '.', '--s--', 'JJ', 'NNS', ',', 'RB', 'IN', 'DT', 'NN', ',', 'CC', 'DT', 'NNS', '.', '--s--', 'NNP', 'RB', 'CC', 'DT', 'NNS', '.', '--s--', 'JJ', 'NNS', ',', 'CC', 'JJ', 'NNS', ',', 'CC', 'DT', 'JJ', 'NN', '.', '--s--', 'JJ', 'CC', 'JJ', '.', '--s--', 'DT', 'VBP', 'RB', 'JJ', 'NNS', ',', 'CC', 'DT', 'NN', '``', 'JJ', 'NN', "''", 'VBZ', 'NN', 'IN', 'PRP', '.', '--s--', 'RB', 'WRB', 'MD', 'PRP', 'VBP', 'RB', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'RB', 'VBP', 'CD', 'NNS', ':', 'CD', '.', 'VB', 'DT', 'NNS', ',', 'RB', 'DT', 'NN', '.', '--s--', 'NNP', '``', 'JJ', 'NN', "''", 'IN', 'DT', 'JJ', ':', 'VB', 'JJ', 'NNS', '.', '--s--', 'PRP', 'MD', 'VB', 'IN', 'JJ', 'NNS', '.', '--s--', 'LS', '.', '``', 'NNP', "''", 'MD', 'VB', 'NN', '.', '--s--', 'IN', 'RB', 'VBN', ',', 'NN', 'VBZ', 'NN', '.', '--s--', 'PRP', 'VBZ', 'VBG', 'NN', ',', 'CC', 'VBG', 'RP', 'NN', ',', 'CC', 'VBG', 'DT', 'NN', '.', '--s--', 'PRP', 'VBZ', 'IN', 'NNS', 'IN', 'JJ', ',', 'WP$', 'NNS', 'TO', 'NNS', 'VBP', 'IN', 'DT', '$', 'CD', 'DT', 'NN', ',', 'VBP', 'RB', 'VB', '.', '--s--', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'JJ', 'NNS', ':', 'NNP', 'NNS', 'TO', 'NN', 'NNS', 'NN', 'TO', '``', 'DT', 'NNP', 'NNP', 'IN', 'DT', 'NNP', "''", ':', 'VBG', 'DT', 'NNS', 'MD', 'VB', 'VBN', ',', 'RB', 'VBN', '.', '--s--', 'CC', 'DT', 'NNS', 'VBP', 'IN', 'CD', 'NN', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'WRB', 'NN', 'IN', 'NN', '.', '--s--', 'LS', '.', 'NNP', 'VBZ', 'JJ', ':', 'NN', 'VBZ', 'RB', '.', '--s--', 'NNP', 'NN', 'VBZ', 'JJ', '.', '--s--', 'PRP', 'VBZ', 'RB', 'JJ', 'CC', 'NNS', '.', '--s--', '(', 'WP', 'MD', 'VB', 'JJ', 'CD', 'CD', 'NNS', 'IN', 'NN', 'DT', 'NN', '.', ')', '--s--', 'CC', 'IN', 'NN', 'NN', 'CC', 'IN', 'JJ', 'NNS', 'DT', 'JJ', 'NN', 'VBZ', 'JJ', 'NNS', 'IN', 'NN', '.', '--s--', 'PRP', 'MD', 'RB', 'VB', 'NN', ':', 'RB', 'IN', 'DT', 'NNS', 'CC', 'DT', 'NNS', '.', '--s--', 'PRP$', 'NN', 'MD', 'VB', 'CC', 'RB', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NNP', ',', 'MD', 'VB', 'IN', 'DT', 'NN', 'NNS', 'RB', 'IN', 'NN', ',', 'CC', 'MD', 'VB', 'JJ', 'NNS', 'IN', 'WDT', 'NNS', 'RBS', 'VBP', 'VBG', '.', '--s--', 'LS', '.', 'JJ', 'NNS', 'VBP', 'RB', 'JJ', '.', '--s--', 'RB', 'VB', 'DT', 'JJ', 'NN', 'MD', 'VB', 'JJ', 'NN', 'VBN', 'RB', ':', 'NN', 'NNS', 'VBD', ',', 'NN', 'NNS', ',', 'NNS', 'NNS', ',', 'JJ', 'NNS', 'NNS', '.', '--s--', 'EX', 'VBZ', 'JJ', 'NN', 'TO', 'VB', 'VBN', ',', 'CC', 'JJ', 'NN', 'CC', 'NN', 'NN', 'VBP', 'VBN', 'IN', 'RB', 'VB', 'WP', 'VBP', 'IN', 'JJ', 'NNS', 'MD', 'VB', 'RB', 'IN', 'PRP', 'RB', ':', 'CC', 'RB', 'RB', '.', '--s--', 'JJ', 'NN', 'NNS', 'VBP', 'VBN', ',', 'NN', ',', 'NN', 'CC', 'NN', ':', 'DT', 'IN', 'JJ', 'NN', '.', '--s--', 'PRP', 'VBP', 'VBN', 'TO', 'NNS', '.', '--s--', 'JJ', 'JJ', 'NNS', 'RB', 'VB', 'NN', 'CC', 'JJ', 'NN', ':', 'PRP', 'VBP', 'RB', 'JJ', ':', 'JJR', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'CC', 'NNP', '.', '--s--', 'JJ', 'NNS', 'VBP', 'JJR', ',', 'CC', 'JJ', 'NNS', 'RB', 'VB', 'TO', 'DT', '$', 'CD', 'DT', 'NN', '.', '--s--', 'VBP', 'PRP', 'JJ', 'IN', '.', '--s--', 'NNP', 'VBP', 'IN', 'JJ', 'NNS', 'VBP', ':', 'RB', 'RB', 'IN', 'DT', 'NNS', 'IN', 'NNS', 'VBP', 'VBN', '.', '--s--', 'CC', 'DT', 'NNS', 'VBP', 'VBN', '.', '--s--', 'CD', '.', 'NNP', 'NN', 'VBP', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'JJ', 'NNS', 'JJ', ',', 'RB', '.', '--s--', 'UH', ',', 'IN', 'VBN', '.', '--s--', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'DT', 'NNS', 'MD', 'VB', 'VBN', 'TO', 'VB', ',', 'CC', 'EX', 'VBP', 'JJ', 'NNS', 'IN', 'NN', 'NNS', ',', 'CC', 'PRP$', 'NNS', ',', 'TO', 'VB', 'IN', 'JJ', 'JJ', 'NN', '.', '--s--', 'PRP', 'MD', 'VB', 'IN', 'DT', 'IN', 'NNS', 'IN', 'JJ', 'NNS', ',', 'CC', 'VB', 'IN', 'NN', 'NNS', 'VBN', 'IN', 'JJ', 'NNS', 'CC', 'VBN', 'IN', 'NNS', 'CC', 'NNS', '.', '--s--', 'NNP', 'NN', 'VBP', 'RB', 'VB', 'DT', 'NNS', '.', '--s--', 'PRP', 'VBP', 'RB', 'VBN', 'IN', 'JJ', 'NN', 'CC', 'NN', '.', '--s--', 'PRP', 'VBP', 'JJ', 'TO', 'VB', 'VBN', '.', '--s--', 'CC', 'PRP', 'VBP', 'VBN', 'RP', 'IN', 'JJ', 'JJ', 'NNS', ',', 'IN', 'NN', 'NNS', 'VBG', 'RP', ',', 'VBN', 'IN', 'JJ', 'NNS', 'CC', 'JJ', 'NNS', '.', '--s--', 'CC', 'JJ', 'IN', 'PRP', 'MD', 'VB', 'RB', 'RB', 'VBN', 'IN', 'JJ', 'CC', 'JJ', 'NN', '.', '--s--', 'DT', 'MD', 'RB', 'VB', 'DT', 'NN', ':', 'NNS', 'VB', 'RP', 'IN', 'JJ', 'NNS', '.', '--s--', 'CC', 'DT', 'WP', 'NN', 'RB', 'VBZ', 'VBG', 'VBP', 'VBN', '.', '--s--', 'NNP', 'NNS', ',', 'VBG', 'NN', 'NNS', ',', 'VBG', 'JJ', ',', 'PRP', 'VBP', 'RB', 'RBR', 'JJ', ':', 'NNS', 'RB', 'VB', 'IN', 'PRP', '.', '--s--', 'RB', 'IN', 'PRP', 'VBZ', 'JJR', 'TO', 'VB', 'NNS', 'CC', 'NN', 'NNS', 'CC', 'NN', 'NN', 'IN', 'VBN', 'NNS', ',', 'DT', 'NNS', 'IN', 'DT', 'JJ', 'NNS', 'VBG', 'DT', 'NNS', 'VBP', 'DT', 'NN', '.', '--s--', 'NNP', 'NN', ',', 'DT', 'NN', 'VBP', 'RB', 'VBG', 'NN', '.', '--s--', 'PRP', 'VBP', 'VBG', 'RP', 'DT', 'NN', ',', 'VBG', 'DT', 'NNS', ',', 'VBG', 'DT', 'NN', '.', '--s--', 'CC', 'PRP', 'VBP', 'PRP', 'TO', 'VB', 'VBG', ',', 'CC', 'PRP', 'VBP', 'TO', 'VB', 'PRP', '(', 'CC', 'NNS', ')', ',', 'TO', 'VB', 'NN', 'IN', 'PRP$', 'NNS', ',', 'TO', 'VB', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'VBZ', 'DT', 'NN', 'TO', 'DT', 'NN', '.', '--s--', 'PRP', 'VBZ', 'WP', 'JJ', 'NN', 'MD', 'VB', 'RBS', 'TO', 'VB', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', 'NN', 'NN', ',', 'VBZ', 'PRP$', 'JJ', 'JJ', 'JJ', 'NN', 'NN', '.', '--s--', 'PRP', 'VBZ', 'DT', 'NN', 'IN', '``', 'NNP', 'NNP', ':', 'WP', 'MD', 'PRP', 'VBZ', '.', "''", '--s--', '(', 'NNP', 'NNPS', ',', 'CD', ')', '.', '--s--', 'NNP', 'NNS', 'RB', 'CC', 'IN', 'JJ', 'NNS', 'VBN', 'NNS', 'IN', 'DT', 'NN', 'TO', 'VB', 'RP', 'DT', 'NNP', 'NN', 'NNS', ':', 'CC', 'VBD', 'RP', 'IN', 'PRP$', 'NN', 'TO', 'VB', 'PRP$', 'NNS', 'JJ', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'DT', 'NNS', ',', 'VBG', 'PRP', 'VBP', 'TO', 'NNS', 'IN', 'DT', 'NNP', 'CC', 'RB', 'IN', 'DT', 'NNP', 'VBD', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NN', 'IN', 'DT', 'NN', 'NN', ',', 'IN', 'PRP', 'VBD', 'IN', 'NNP', 'CD', ',', 'IN', 'DT', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'IN', 'DT', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'NNS', 'CC', 'NNS', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'RB', 'VBD', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'NN', '.', '--s--', 'CC', 'DT', 'NNS', 'VBN', 'IN', 'DT', 'JJ', 'NNS', 'MD', 'RB', 'VB', 'NN', 'NNS', '.', '--s--', 'IN', 'NN', ',', 'NNS', 'IN', 'DT', 'NNP', 'CC', 'IN', 'DT', 'NNP', 'NN', 'VBD', 'IN', 'VBG', 'JJ', 'NNS', 'CC', 'NNS', 'IN', 'DT', 'NN', 'MD', 'VB', 'PRP', 'JJR', 'NN', 'CC', 'NN', 'MD', 'NNP', 'POS', 'NN', 'NN', 'NN', 'IN', 'DT', 'NN', 'POS', 'NN', '.', '--s--', '``', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'DT', 'NN', 'PRP', 'VBP', 'WDT', 'VBZ', 'IN', 'PRP', 'VBP', 'VBG', 'RB', 'RB', 'VBZ', 'TO', 'VB', 'DT', 'NN', 'IN', 'NN', ',', "''", 'VBD', 'CD', 'NN', 'NN', ',', 'VBG', 'IN', 'NN', '.', '--s--', 'DT', 'NNP', 'POS', 'NNS', 'IN', 'NN', 'VBD', 'RB', 'VBD', 'NNP', 'NN', ',', 'WRB', 'PDT', 'DT', 'NNP', 'NNP', 'NNP', 'CC', 'DT', 'NNP', 'NNP', 'VBD', 'NNS', 'VBG', 'DT', 'JJ', 'NNP', 'NN', 'VBG', 'DT', 'JJ', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'NN', 'IN', 'DT', 'NN', 'NN', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'VBN', 'IN', 'DT', 'NNS', ',', 'VBG', 'TO', 'JJ', 'NNS', ',', 'CC', 'VBD', 'PRP', 'VBD', 'RB', 'VBD', 'DT', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'NNP', 'NNS', 'VBD', 'DT', 'NNS', 'VBD', 'RB', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'POS', 'NN', 'NN', '.', '--s--', 'PRP', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'PRP', 'VBD', 'IN', 'NNP', 'CD', ',', 'VBG', 'NN', 'TO', 'DT', 'NNS', 'IN', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'NN', 'RB', '.', '--s--', 'DT', 'NNP', 'VBZ', 'NN', 'TO', 'DT', 'NN', 'NN', 'IN', 'VBG', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'JJ', 'IN', 'DT', 'NN', 'NNP', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'POS', '``', 'DT', 'NN', '.', "''", '--s--', '``', 'NNP', 'IN', 'DT', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'CC', 'DT', 'NN', 'IN', 'DT', 'NNP', 'CC', 'NNS', 'VBP', 'VBN', 'IN', 'JJ', 'NN', '.', '--s--', 'PRP', 'VBP', 'RB', 'PRP', 'MD', 'VB', 'WP', 'VBZ', 'RB', ',', 'WP', 'VBZ', 'JJ', ',', 'WP', 'VBZ', 'JJ', ',', "''", 'PRP', 'VBD', '.', '--s--', 'WRB', 'PRP', 'VBD', 'VBN', 'PRP$', 'NN', 'VBD', 'DT', '``', 'JJ', ',', "''", 'NNP', 'NNP', 'VBD', ':', '``', 'PRP', 'VBZ', 'DT', 'NN', '.', '--s--', 'CC', ',', 'IN', 'DT', 'NN', ',', 'WDT', 'VBZ', 'DT', 'JJ', 'NN', 'TO', 'VB', '.', "''", '--s--', 'IN', 'DT', 'NNP', ',', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'VBG', 'DT', 'NN', 'NN', 'POS', 'NN', '.', '--s--', '``', 'NN', 'POS', 'NN', 'NN', 'NN', 'VBZ', 'RB', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', "''", 'PRP', 'VBD', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'JJ', ',', 'CC', 'DT', 'NN', 'VBZ', 'IN', 'JJ', 'JJ', 'NN', '.', "''", '--s--', 'CC', 'NN', 'NNS', 'VBD', 'IN', 'NNP', 'POS', 'NN', 'VBD', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'DT', 'NN', '.', '--s--', '``', 'CD', 'NN', 'VBZ', 'IN', 'DT', 'VBZ', 'DT', 'JJ', 'NN', ',', 'RB', 'VBN', 'IN', 'PRP$', 'NN', ',', 'CC', 'RB', 'DT', 'JJ', 'NN', ',', "''", 'VBD', 'CD', 'JJ', 'NN', 'NN', ',', 'WP', 'RB', 'VBN', 'IN', 'PRP', 'RB', 'VB', 'VBN', '.', '--s--', '``', 'DT', 'JJ', 'VBZ', 'IN', 'PRP', 'VBP', 'DT', 'JJ', 'NN', ',', 'IN', 'CD', 'NNS', 'RB', '.', '--s--', 'PRP', 'VBP', 'WDT', 'VBZ', 'RBR', 'JJ', '.', "''", '--s--', 'RB', ',', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'PRP$', 'NNS', 'NNP', 'NN', ',', 'VBG', 'NNS', 'IN', 'PRP', 'VBZ', 'IN', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'JJ', 'NN', 'NNS', 'IN', 'CD', 'NN', 'NN', ':', 'WRB', 'NNS', 'EX', 'VBD', ':', 'CC', 'TO', 'VB', 'IN', 'DT', 'NN', 'NN', 'VBD', 'RB', 'IN', 'DT', 'NN', 'IN', 'NNP', 'CC', 'IN', 'RB', 'DT', 'NN', 'IN', 'NNP', '.', '--s--', 'PDT', 'DT', 'NNP', 'CC', 'DT', 'NNP', 'VBD', 'TO', 'VB', 'NN', 'NNS', 'VBG', 'IN', 'DT', 'NN', 'TO', 'VB', 'DT', 'NNS', '.', '--s--', 'IN', 'NNP', ',', 'NN', 'NNS', 'VBD', 'RB', 'IN', 'CD', 'NN', 'IN', 'JJ', 'NNP', 'NN', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'DT', 'NN', 'VBD', 'TO', 'VB', 'VBG', 'RP', 'CC', 'IN', 'JJ', 'NN', 'VBD', 'VBN', 'JJR', '.', '--s--', 'IN', 'DT', 'NNP', 'NN', ',', 'DT', 'NN', 'VBZ', 'VBG', 'VBN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'CD', 'NNS', 'VBD', 'RB', 'IN', 'DT', 'JJ', 'NNP', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'WDT', 'VBD', 'VBN', 'IN', 'DT', 'CD', 'NN', 'TO', 'VB', 'DT', 'NN', 'POS', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'PRP', 'VBP', 'JJ', 'NN', 'IN', 'JJ', 'NNS', ',', 'CC', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'WRB', 'DT', 'NN', 'NN', 'POS', 'NN', 'VBD', '.', '--s--', 'PRP', 'VBD', 'RB', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'WP', 'IN', 'NN', 'VBD', 'VBG', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'CC', 'NNP', 'NNS', '.', '--s--', 'RB', ',', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'NNP', 'CC', 'JJ', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'RB', '.', '--s--', 'NNP', 'VBP', 'PRP', 'VBD', 'TO', 'VB', 'DT', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNS', 'VBP', 'NNP', 'NNP', 'VBD', 'VBN', 'IN', 'NNP', 'NN', 'CC', 'NN', ',', 'RB', 'IN', 'VBG', 'IN', 'NNP', 'NNP', '.', '--s--', 'PRP', 'VBD', 'JJ', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBD', 'IN', 'DT', 'NN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NN', 'IN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'IN', 'NN', ',', 'VBD', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'WRB', 'DT', 'NN', 'NN', 'VBD', ',', 'CC', 'VBD', 'RB', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'RB', 'IN', 'JJ', 'NN', ',', 'NNP', 'NNP', 'NNP', 'VBD', 'RB', 'VBN', 'PRP$', 'NNS', 'TO', 'VB', 'DT', 'NNP', 'NNPS', 'NNP', 'NN', 'IN', 'NNP', 'IN', 'CD', 'NN', 'DT', 'NN', '.', '--s--', 'RB', ',', 'NNP', 'NNP', 'VBD', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'IN', 'NNP', 'CD', ',', 'CD', '.', '--s--', 'PRP', 'VBD', 'TO', 'NNP', 'IN', 'NNP', 'CD', ',', 'WRB', 'DT', 'NN', 'VBD', 'CD', 'NNS', ',', 'CC', 'RB', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'CC', 'VBN', 'TO', 'NNP', 'IN', 'VBG', 'PRP$', 'NN', '.', '--s--', 'VBG', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'JJ', 'NN', 'NNS', 'IN', 'RB', 'VBN', 'NN', 'NN', 'NNS', ',', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'IN', 'NNP', 'POS', 'NN', '.', '--s--', 'RB', 'VBN', 'VBZ', 'DT', 'NN', 'VBN', 'NN', 'NN', 'CC', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NN', 'POS', 'NNS', ',', 'IN', 'DT', 'NN', 'IN', 'NN', '.', '--s--', 'SYM', ':', 'IN', 'IN', 'NNP', 'POS', 'NN', '.', '--s--', 'SYM', ':', 'VBN', 'IN', 'NNP', 'NNP', 'NN', 'NN', '.', '--s--', 'SYM', ':', 'IN', 'JJ', 'NNS', '.', '--s--', 'SYM', ':', 'IN', 'IN', 'NNP', 'POS', 'NN', '.', '--s--', 'NNS', ':', 'CD', 'CD', '.', '--s--', 'NNS', ':', 'RB', 'JJ', '.', '--s--', 'VB', 'RP', 'DT', 'NN', '.', '--s--', 'VBP', 'IN', 'DT', 'NN', ':', 'VB', 'CD', 'JJ', 'NNS', '.', '--s--', 'NNP', 'DT', 'NN', 'TO', 'VB', 'PRP$', 'NN', 'CC', 'VB', 'PDT', 'PRP$', 'NNS', '.', '--s--', 'DT', 'VBZ', 'DT', 'NN', 'IN', 'JJS', 'NN', 'NNS', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'IN', 'JJ', 'WP', 'MD', 'VB', 'NN', '.', '--s--', 'CC', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'IN', 'NNS', 'MD', 'VB', 'RB', 'CC', 'RB', 'IN', 'DT', 'JJ', 'CD', 'NNS', '.', '--s--', 'CC', 'PRP', 'VBP', 'RB', 'JJ', 'IN', 'CD', 'NN', ':', 'VBP', 'RB', 'VB', 'IN', 'DT', 'NN', '.', '--s--', 'NNS', 'WP', 'VBD', 'NN', 'IN', 'DT', 'NN', 'IN', 'CD', 'VBD', 'TO', 'VB', 'PRP', '.', '--s--', 'RB', 'IN', 'NNP', 'POS', 'NN', ',', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'CD', 'NN', 'IN', 'WRB', 'PRP', 'VBD', 'IN', 'NNP', 'CD', 'CD', 'NNS', 'RB', '.', '--s--', 'NNP', 'VBG', 'RB', 'VBD', 'JJ', 'IN', 'JJ', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'CD', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'CC', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NNS', 'VBD', '.', '--s--', 'DT', '``', 'NNP', 'FW', "''", 'IN', 'CD', 'CC', 'CD', 'VBD', 'JJ', ',', 'CC', 'VBD', 'RB', 'VB', 'TO', 'JJ', 'CC', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'DT', 'NNS', 'VBP', 'NNP', 'POS', 'NN', ',', 'CC', 'DT', 'JJ', 'NN', 'WDT', 'MD', 'VB', 'RB', 'DT', 'NN', ',', 'IN', 'DT', 'NN', 'IN', 'NN', 'NN', '.', '--s--', '``', 'EX', 'VBZ', 'VBN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'WDT', 'VBZ', 'DT', 'JJ', 'NN', 'NN', 'IN', 'PRP', 'VBP', 'VBN', 'DT', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNPS', 'NNP', '.', '--s--', 'CC', 'JJS', 'NNS', 'VBP', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'MD', 'VB', 'TO', 'VB', 'NNS', '.', '--s--', '``', 'WRB', 'PRP', 'VBP', 'DT', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'JJ', 'NN', 'NNP', 'CC', 'NNP', ',', '``', 'PRP', 'VB', 'IN', 'DT', 'NN', 'TO', 'VB', '.', "''", '--s--', 'RB', 'IN', 'NNS', 'WP', 'VBP', 'DT', 'NN', 'NN', 'IN', 'VBG', 'NNS', ':', 'CC', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'CC', 'NN', 'NNS', 'VBP', ':', 'DT', 'NN', 'VBZ', ':', 'NNP', 'IN', 'DT', 'NN', 'TO', 'VB', 'RB', ',', 'CC', 'VB', 'NNS', 'RB', 'IN', 'NNS', '.', '--s--', 'DT', 'JJS', 'NN', 'JJ', 'NNS', 'MD', 'VB', 'VBZ', '``', 'RB', 'VBP', 'RB', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'CC', 'NN', 'IN', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', 'NN', 'WDT', 'VBZ', 'IN', '$', 'CD', 'CD', '.', '--s--', 'IN', 'DT', 'CD', 'NN', ',', 'NNP', 'NNP', 'VBZ', ',', 'PRP', 'MD', 'VB', 'JJ', 'TO', 'VB', 'IN', '``', 'DT', 'JJ', 'NN', '.', "''", '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'PRP', 'VBZ', 'RB', 'RB', 'DT', 'JJ', 'NN', 'TO', 'VB', 'RP', 'CC', 'VB', '.', '--s--', '``', 'DT', 'VBZ', 'DT', 'NN', 'RB', 'RB', ',', 'CC', 'WRB', 'NN', 'VBZ', 'TO', 'VB', ',', 'PRP', 'MD', 'VB', 'RBR', 'IN', 'NN', 'VBZ', ',', "''", 'PRP', 'VBD', '.', '--s--', '``', 'RB', 'PRP', 'VBZ', 'RBR', 'JJ', 'TO', 'VB', 'CC', 'VB', 'WRB', 'NNS', 'VBP', '.', "''", '--s--', 'NNP', 'NNP', ',', 'NN', 'IN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NN', 'IN', 'DT', 'NN', 'NN', 'NN', 'NNP', 'NNPS', 'NNP', ',', 'VBZ', ',', '``', 'PRP$', 'JJ', 'NN', 'MD', 'VB', 'TO', 'RB', 'VB', 'IN', 'PRP', '.', '--s--', 'RB', ',', 'PRP', 'VBZ', 'RB', 'JJ', 'TO', 'VB', 'RP', 'CC', 'RB', "''", 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'NN', 'VBZ', '``', 'VBG', 'TO', 'VB', 'DT', 'NN', 'NN', '.', "''", '--s--', 'CC', 'PRP', 'RB', 'VBZ', 'PRP', 'VBZ', '``', 'DT', 'JJ', 'NN', 'TO', 'VB', 'NN', '.', "''", '--s--', 'NNP', 'NNP', ',', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NN', 'NNS', 'CC', 'JJ', 'NNS', ',', 'VBZ', 'PRP$', 'JJ', 'NN', 'NNS', 'VBD', 'RB', 'VB', 'IN', 'DT', 'NN', 'IN', 'CD', ',', 'CC', 'VB', 'DT', 'NN', 'TO', 'VB', 'RB', '.', '--s--', '``', 'PRP', 'VBP', 'VBN', 'JJ', 'NNS', ',', 'RB', 'NNS', ',', "''", 'PRP', 'VBZ', '.', '--s--', '``', 'PRP', 'VBP', 'VBN', 'CC', 'NN', '.', '--s--', 'CC', 'PRP', 'VBP', 'TO', 'VB', 'NN', 'IN', 'NNS', 'CC', 'VB', 'NNS', 'WRB', 'PRP', 'VBP', '.', "''", '--s--', 'CD', 'NN', 'IN', 'NN', 'NNS', 'VBZ', 'WP', 'VBD', 'NNP', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', 'NNS', 'CC', 'NNS', ',', 'PRP', 'VBP', ',', 'NNS', 'MD', 'VB', 'IN', 'NNS', 'TO', 'VB', '.', '--s--', 'NNP', 'POS', 'NN', '``', 'VBD', 'DT', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNPS', 'IN', 'NNP', 'NNP', '.', '--s--', '``', 'NNS', 'VBD', 'VBG', 'VBN', 'RP', 'IN', 'NNS', 'IN', 'DT', 'NN', '.', "''", '--s--', 'PRP$', 'NN', ':', '``', 'PRP', 'MD', 'TO', 'VB', 'RB', 'IN', 'DT', 'NN', 'VBG', 'PRP', '.', "''", '--s--', 'NNP', 'NNP', ',', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NNP', 'NNP', ',', 'WDT', 'VBZ', 'CD', 'IN', 'DT', 'NN', 'POS', 'JJR', 'NN', 'NNS', ',', 'VBD', 'PRP$', 'NN', 'IN', 'NNP', 'POS', 'NN', '.', '--s--', 'NNS', 'JJ', 'IN', 'NNP', 'NNPS', 'NNP', ',', 'NNP', 'NNPS', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'JJ', 'NNS', 'IN', 'PRP', 'MD', 'RB', 'VB', 'PRP', ',', 'PRP', 'VBZ', '.', '--s--', 'CC', 'NNP', 'NNP', 'VBZ', 'TO', 'VB', 'RP', 'JJR', 'NNS', 'NN', '.', '--s--', '``', 'PRP', 'MD', 'VB', 'JJ', 'IN', 'JJ', ',', 'CC', 'PRP', 'MD', 'RB', 'VB', 'VBG', 'IN', 'PRP', 'VBD', 'PRP', 'VBD', 'VBN', 'IN', 'JJ', 'NN', ',', "''", 'PRP', 'VBZ', '.', '--s--', 'PRP', 'VBZ', 'IN', 'NNS', 'VBP', 'RB', 'VBN', 'RB', ',', 'CC', 'DT', 'NN', 'NNS', 'VBP', 'RBR', 'RB', 'IN', 'IN', 'DT', 'NN', 'IN', 'CD', '.', '--s--', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'NN', 'MD', '``', 'VB', 'IN', 'NNS', 'WDT', 'VBP', 'VBN', 'DT', 'NN', 'IN', 'DT', 'IN', 'NN', ',', 'CC', 'NNS', 'WDT', 'VBP', 'VBN', 'VBN', 'RP', 'DT', 'NN', 'RBR', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', '.', "''", '--s--', 'PRP$', 'NN', 'VBZ', 'VBN', 'NN', ',', 'NN', 'CC', 'JJ', 'NNS', '.', '--s--', 'JJ', 'NN', 'NNS', 'VBP', 'RBR', 'JJ', '.', '--s--', 'PRP', 'VBP', 'NNS', 'MD', 'VB', 'NNS', ':', 'CC', 'RB', 'RB', 'RB', 'RB', '.', '--s--', 'JJ', 'IN', 'PRP', 'VBP', 'IN', 'DT', 'NN', 'MD', 'VB', 'JJ', ',', 'JJ', ',', 'CC', 'VBN', 'WRB', 'NN', 'NNS', 'VBP', 'VBG', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', 'NN', ',', 'VBN', 'NNS', 'NNS', 'IN', 'PRP$', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNS', 'MD', 'VB', '.', '--s--', 'PRP', 'VBZ', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'IN', 'CD', 'NN', 'TO', 'CD', 'NN', ',', 'RB', 'JJR', 'IN', 'JJS', 'NNS', 'VBP', '.', '--s--', 'IN', 'DT', 'NN', 'VBZ', 'PRP', 'VBZ', ',', 'NNP', 'NNP', 'VBZ', 'VBN', 'NN', 'NNS', 'CC', 'VBN', 'IN', 'NN', 'CC', 'JJ', 'NN', 'NNS', ',', 'RB', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'PRP', 'VBZ', 'NN', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'DT', 'NN', 'NN', 'WRB', 'NN', ',', 'JJ', 'CC', 'JJ', 'NN', 'NNS', 'VBP', 'RB', 'JJ', '.', '--s--', 'RB', 'PRP', 'VBZ', 'DT', 'NN', 'MD', 'VB', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'NNP', 'VBZ', ',', '``', 'PRP', 'VBP', 'DT', 'NN', 'NN', 'VBZ', 'RB', ',', "''", 'CC', 'NNS', 'VBP', 'RB', 'TO', 'VB', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', ',', 'VBZ', 'NNP', 'NNS', '(', 'IN', 'RB', 'TO', 'CD', 'NNS', 'POS', 'NN', ')', '.', '--s--', 'PRP', 'VBZ', 'DT', 'NNP', 'CD', 'NN', 'NN', ',', 'VBG', 'JJR', 'JJ', 'NN', 'CC', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', ',', 'VBZ', 'DT', 'NN', 'NN', 'TO', 'NNS', '.', '--s--', 'CD', 'NN', 'IN', 'NNS', 'WP', 'VBP', 'TO', 'VB', 'RP', 'CC', 'VB', 'PRP$', 'NNS', 'VBZ', 'TO', 'VB', '``', 'VB', "''", 'NNS', ',', 'RB', 'IN', 'DT', 'JJ', 'NNS', 'PRP', 'VBP', 'CC', 'IN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'PRP$', 'NN', 'DT', 'NN', '(', 'CC', 'RB', 'DT', 'NN', ')', 'TO', 'VB', 'DT', 'NN', '(', 'CC', 'NN', 'NN', ')', 'IN', 'DT', 'JJ', 'NN', '(', 'DT', 'NN', 'NN', ')', 'IN', 'DT', 'NN', 'VBZ', '.', '--s--', 'IN', 'DT', 'NN', 'VBZ', 'JJ', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', ',', 'CC', 'NN', ',', 'VBZ', 'TO', 'VB', 'JJ', 'IN', 'NNS', 'IN', 'NN', '.', '--s--', 'RB', ',', 'VBG', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'MD', 'VB', 'DT', 'JJ', 'NN', 'TO', 'VB', 'IN', 'NN', '.', '--s--', 'DT', 'NNS', 'IN', 'VBZ', 'RB', 'VBD', 'RB', 'VB', 'NNP', '.', '--s--', 'IN', 'NN', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'VBD', 'RP', 'IN', 'CD', 'NN', 'IN', 'NNP', 'POS', 'NN', 'TO', 'RB', 'CD', 'NN', 'IN', 'NNP', 'POS', 'NN', ',', 'RB', 'IN', 'DT', 'NNS', 'VBD', 'JJR', 'IN', '$', 'CD', '.', '--s--', 'CC', 'JJ', 'NNS', 'MD', 'VB', 'WRB', 'NN', 'VBZ', 'NN', '.', '--s--', 'PRP', 'VBZ', 'JJ', 'TO', 'VB', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', '.', '--s--', 'CC', 'NNS', 'MD', 'VB', 'IN', 'NN', ',', 'IN', 'VBG', 'RB', 'RB', ',', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'NN', 'NNS', ',', 'JJ', ',', 'VBZ', 'CD', 'NN', 'TO', 'CD', 'NN', 'DT', 'NN', ':', 'DT', 'NN', 'IN', 'CD', 'NN', 'VBZ', 'VBN', 'NNS', '.', '--s--', 'VBG', ',', 'VBP', ',', 'CD', 'NN', 'IN', 'NN', 'IN', 'NNS', 'VBZ', 'DT', 'RB', 'VB', 'RP', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'TO', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'TO', 'PRP$', 'NNS', 'IN', '$', 'CD', 'DT', 'NN', ',', 'VBG', 'PRP', 'DT', 'JJ', 'NN', 'IN', 'JJR', 'IN', 'CD', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'NN', 'NN', '.', '--s--', 'DT', 'NN', ',', 'WDT', 'VBD', 'VBN', 'IN', 'NNP', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', 'IN', 'DT', 'JJ', 'NN', ',', 'VBD', 'VBN', 'TO', 'VB', 'RP', 'TO', '$', 'CD', 'CD', 'IN', 'NN', ',', 'CC', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', ',', 'TO', 'PRP$', 'CD', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'VBD', 'RB', 'VB', 'RP', ',', 'NNP', 'NNP', 'NNP', ',', 'NN', 'CC', 'JJ', 'NN', 'NN', ',', 'VBD', ',', '``', 'PRP', 'VBP', 'RB', 'JJ', 'IN', 'DT', 'NNS', 'POS', 'NN', '.', "''", '--s--', 'PRP', 'VBD', 'IN', 'IN', 'DT', 'NN', 'NN', 'NN', 'NN', ',', 'WRB', 'DT', 'NN', 'RB', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'TO', 'VB', 'NN', 'IN', 'PRP', 'VBZ', 'RB', 'TO', 'NN', 'NNS', 'IN', 'NN', ',', 'RB', 'NNS', 'VBD', 'TO', 'VB', 'RP', 'PRP$', 'JJ', 'NN', 'IN', 'DT', 'NN', '.', '--s--', '``', 'PRP', 'VBD', 'RP', 'IN', 'PRP$', 'JJ', 'NN', 'RB', 'IN', 'VBN', 'NN', ',', "''", 'NNP', 'NNP', 'VBD', '.', '--s--', '``', 'PRP', 'VBZ', 'RB', 'JJ', '.', "''", '--s--', 'PRP', 'VBD', 'DT', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNS', '.', '--s--', '``', 'PRP', 'VBP', 'IN', 'DT', 'NN', 'NN', ',', 'CC', 'IN', 'DT', 'NN', ',', 'PRP', 'VBZ', 'JJ', 'TO', 'VB', 'PRP$', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'IN', 'PRP', 'VBP', 'DT', 'NN', 'IN', 'DT', 'NN', '.', "''", '--s--', 'DT', 'NN', 'NN', 'MD', 'RB', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNS', 'VBP', 'DT', 'NN', 'TO', 'VB', 'NN', 'IN', 'PRP', ',', 'CC', 'DT', 'NN', 'MD', 'VB', 'DT', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NNS', '.', '--s--', 'PRP', 'MD', 'RB', 'RB', 'VB', 'DT', 'NNS', 'TO', 'JJ', 'NNS', ',', 'CC', 'DT', 'JJ', 'NNS', 'WP', 'VBP', 'DT', 'VBG', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'VBP', 'DT', 'NN', 'TO', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NNS', 'IN', 'NNP', 'NNP', 'VBP', 'NNP', 'NNP', ',', 'DT', 'RB', 'VBN', 'NNP', ',', 'NNP', ',', 'NN', 'NN', 'NN', 'WP$', 'NN', 'VBZ', 'NNP', 'NNP', ',', 'JJ', 'NN', 'IN', 'NN', ':', 'NNP', 'NNP', 'NNP', ',', 'JJ', 'NN', 'TO', 'NNP', 'NNP', ':', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ':', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NNS', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ':', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', ':', 'CC', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'NN', 'NN', 'IN', 'JJ', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', 'CC', 'NNP', 'VBN', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', 'RB', 'VBN', 'CD', 'JJ', 'NN', 'NNS', 'IN', 'PRP$', 'CD', 'NN', 'NN', '.', '--s--', 'DT', 'NNS', 'VBP', 'NNP', 'NNP', ',', 'JJ', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ':', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', ':', 'CC', 'NNP', 'NNP', ',', 'JJ', 'NN', 'IN', 'NN', 'CC', 'RB', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'JJ', 'NN', 'DT', 'NN', 'IN', 'NNP', 'NN', 'CC', 'NNS', 'NN', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'IN', 'DT', 'NN', 'VBG', 'IN', 'DT', 'CD', 'NN', ',', 'NN', 'IN', 'DT', 'NNP', 'POS', 'JJ', 'NNS', 'MD', 'VB', 'IN', 'CD', 'NNS', 'IN', 'DT', 'NN', 'VBZ', 'CC', 'VBZ', 'CD', 'NNS', 'IN', 'NNP', 'POS', 'NN', ',', 'DT', 'NN', 'JJ', 'TO', 'CD', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'MD', 'VB', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'MD', 'VB', 'VBN', 'IN', 'PDT', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'CD', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'CD', 'NN', '.', '--s--', 'DT', 'NNP', 'NNP', 'RB', 'VBD', 'NN', 'NN', 'TO', 'VB', 'JJ', 'VBZ', 'IN', 'NN', 'NN', '.', '--s--', 'JJ', 'IN', 'DT', 'JJ', 'NNS', 'VBD', 'IN', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'CC', 'VBD', 'RB', 'VBN', ',', 'RB', 'IN', 'PRP', 'VBD', 'RB', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'DT', 'JJ', '``', 'NN', 'NNS', "''", 'VBP', 'RB', 'TO', 'VB', 'VBN', '.', '--s--', 'DT', 'JJR', 'NN', 'NN', 'NN', 'MD', 'VB', 'PRP', 'PRP$', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'RB', 'MD', 'VB', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', ',', 'RBR', 'JJ', 'NNS', 'VBN', ':', 'CC', 'RB', 'VBN', ':', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'RBS', 'RB', ',', 'JJ', 'IN', 'DT', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'VBD', 'DT', 'CD', 'NN', ',', 'MD', 'VB', 'VBN', ':', 'RB', 'IN', 'DT', 'NN', 'POS', 'NN', 'VBZ', 'RB', 'DT', 'NNP', 'NN', '.', '--s--', 'DT', 'RBS', 'JJ', 'IN', 'DT', 'NNP', 'NNS', 'VBN', 'VBG', 'DT', 'JJ', 'NN', 'NN', 'TO', 'VB', 'JJ', 'JJ', 'NNS', ',', 'JJ', 'IN', 'VBG', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'NN', 'CC', 'NNS', 'NNS', '.', '--s--', 'CC', 'IN', 'DT', 'NN', ',', 'NN', 'VBZ', 'IN', 'DT', 'NNS', 'WDT', 'VBD', 'VBN', 'IN', 'NN', ',', 'CC', 'NN', 'NNS', 'CC', 'NNS', 'VBD', 'DT', 'NN', 'NNS', 'VBD', 'RB', 'VBN', '.', '--s--', 'NNP', 'NNP', 'CC', 'NNP', 'NNS', 'VBD', 'NN', 'IN', 'DT', 'NNS', 'IN', 'CD', 'NNS', 'VBN', 'IN', 'IN', 'DT', 'NNP', 'POS', 'NNP', 'CC', 'NNP', 'POS', 'CD', 'NN', ',', 'RB', 'RB', 'IN', '``', 'JJ', "''", 'NNS', 'IN', 'NNS', '.', '--s--', 'DT', 'NN', ':', 'IN', 'CD', 'NN', 'TO', 'CD', 'NN', 'NNS', 'CC', 'IN', 'CD', 'NN', 'IN', 'DT', 'NN', 'IN', 'VBG', 'DT', 'NN', 'RB', ':', 'VBN', 'NNS', 'TO', 'VB', 'CC', 'VB', 'NNS', 'IN', 'NNS', 'IN', 'CC', 'JJR', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'IN', 'DT', 'NNP', 'NN', 'VBD', 'VBN', 'CD', 'NNS', ',', 'DT', 'NNP', 'NNP', 'POS', '``', 'UH', "''", 'NN', 'NN', 'RB', 'VBD', 'VBN', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NNS', 'TO', 'VB', 'NN', 'NNS', 'VB', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NN', 'VBN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'DT', 'CD', 'NNS', 'TO', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'VB', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'WP', 'TO', 'VB', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', 'CC', 'DT', 'NNP', 'NN', '.', '--s--', '``', 'EX', 'VBD', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NN', 'PRP', 'VBD', 'PRP', 'TO', 'VB', '.', "''", '--s--', 'IN', 'VBG', 'IN', 'IN', 'CD', 'NNS', ',', 'DT', 'NNP', 'NN', 'VBD', 'TO', 'PRP$', 'JJ', 'NN', 'CC', 'DT', 'JJ', 'NN', 'VBD', 'IN', 'NN', '.', '--s--', 'NNS', 'RB', 'VBD', 'DT', 'JJ', 'NN', '``', 'VBG', 'TO', 'VB', 'IN', 'DT', 'NNP', 'MD', 'VB', 'CD', 'NNS', ',', "''", 'NNP', 'NNP', 'VBD', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'WDT', 'DT', 'NN', 'NN', 'PRP', 'MD', 'VB', 'VBN', 'IN', 'DT', 'NN', '.', '--s--', 'CD', 'NN', 'VBN', 'IN', 'CD', 'NN', 'TO', 'CD', 'NN', 'IN', 'DT', 'NNP', 'NNS', '``', 'VBD', 'RB', 'VBG', 'IN', 'VBG', '.', "''", '--s--', 'CC', 'DT', 'JJ', 'NN', 'NN', 'RB', 'VBD', 'TO', 'VB', 'RP', ',', 'CC', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', 'RB', 'VBD', 'RB', 'VBN', '.', '--s--', 'DT', 'NNP', 'CC', 'DT', 'NNP', 'CD', 'VBP', 'DT', 'CD', 'JJ', 'NNS', 'VBN', 'IN', 'NN', 'NNS', 'TO', 'VB', 'PRP$', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'NNS', 'VBP', 'VBN', 'IN', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'CD', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'IN', 'NNS', 'TO', 'VB', 'DT', 'NNS', 'CC', 'VBD', 'RP', 'DT', 'NN', 'RB', 'IN', 'DT', 'NN', 'IN', 'RB', 'JJ', 'NNS', '.', '--s--', 'IN', 'NNP', 'CD', ',', 'RB', 'IN', 'DT', 'NN', 'NN', ',', 'NNP', 'VBD', 'VBN', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'DT', 'JJ', 'NNS', 'VBP', 'VBN', 'CC', 'VBN', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'VBD', 'VBN', 'RP', 'TO', 'DT', 'NN', 'IN', 'NNS', 'TO', 'VB', 'JJ', 'NN', 'NNS', 'IN', 'VBG', 'NN', 'TO', 'VB', 'NN', 'NNS', 'NN', 'TO', 'NN', 'CC', 'VB', 'NNS', '.', '--s--', 'IN', 'NN', ',', 'NNS', 'VBD', 'VBN', 'IN', 'JJ', 'NN', ',', 'CC', 'NNS', 'VBD', 'VBN', 'TO', 'RB', 'VB', 'DT', 'NN', 'CC', 'NNS', 'NNS', '.', '--s--', 'JJ', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'DT', 'NNS', 'VBP', 'VBN', '.', '--s--', 'IN', 'DT', 'NN', 'DT', 'JJ', 'NN', 'VBD', 'NN', 'NN', ',', 'WDT', 'VBD', 'RB', 'VBN', 'CC', 'RB', 'VBN', 'IN', 'NNP', 'NNP', '.', '--s--', 'EX', 'VBD', 'RB', 'VBZ', 'TO', 'VB', 'DT', 'NN', 'NNS', 'IN', '``', 'JJ', "''", 'NNS', ',', 'JJ', 'IN', 'JJ', 'NNS', 'CC', 'NNS', ',', 'WDT', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'IN', 'NN', ',', 'VBN', 'TO', '``', 'VBZ', 'VBN', 'TO', 'DT', 'JJ', 'NN', '.', "''", '--s--', 'CC', 'EX', 'VBD', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'VBG', 'NN', 'NNS', 'NN', 'IN', 'NNS', ',', 'CC', 'NN', 'NNS', '.', '--s--', 'WRB', 'DT', 'NNP', 'NNP', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'CD', ',', 'PRP', 'RB', 'VB', 'DT', 'NNS', 'POS', 'NN', '.', '--s--', 'NNP', 'IN', 'DT', 'JJ', 'JJ', 'NNS', 'VBD', 'IN', 'CD', ',', 'CC', 'VBG', 'IN', 'DT', 'NN', 'VBD', '``', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'NN', ',', "''", 'DT', 'JJ', 'NN', 'NN', 'VBN', 'IN', 'VBG', 'DT', 'JJ', 'TO', 'VB', 'DT', 'NNS', ',', 'TO', 'VB', 'NNS', 'JJ', 'IN', 'NNS', ',', 'TO', 'VB', 'VBG', 'NNS', 'CC', 'TO', 'VB', 'NN', 'NNS', '.', '--s--', 'RB', 'DT', 'JJ', 'IN', 'DT', 'NNS', 'RB', 'VBD', 'VBN', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNS', 'IN', 'NN', 'POS', 'NN', 'CC', 'VBN', 'DT', 'JJ', 'NN', ':', 'DT', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'NNPS', ':', 'TO', 'VB', 'PRP$', 'NN', 'CC', 'DT', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'IN', 'NNP', 'CD', ',', 'DT', 'NNP', 'NNP', ',', 'VBN', 'RP', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NNP', ',', 'DT', 'NNP', ',', 'DT', 'NNPS', 'CC', 'NNP', 'NNP', ',', 'CC', 'DT', 'NNP', 'NNPS', 'NNP', 'NNP', ',', 'RB', 'VBN', 'JJ', 'NN', 'NNS', '.', '--s--', 'IN', 'JJ', 'JJR', 'NNS', 'IN', 'NNS', 'IN', 'JJ', 'NN', 'NNS', 'CC', 'NNS', 'NNS', ',', 'NN', 'NNS', 'VBD', 'VBN', 'IN', 'NN', ',', 'IN', 'DT', 'RBS', 'JJ', 'VBG', 'NN', 'IN', 'CD', 'CC', 'CD', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', ',', 'DT', 'JJ', 'NNS', 'VBD', 'JJ', 'NNS', 'IN', 'VBG', 'TO', 'VB', '.', '--s--', 'IN', 'DT', ',', 'DT', 'JJ', 'NN', 'VBD', ',', 'DT', 'NNP', 'RB', 'VBD', 'JJR', 'IN', 'CD', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', '``', 'NNP', 'NNS', "''", 'VBD', 'TO', 'VB', 'JJ', 'NNS', ':', '--s--', ':', 'IN', 'NNP', 'NNS', 'VBP', 'CD', 'NNS', 'IN', 'NN', ',', 'NN', 'NN', 'NNS', 'IN', 'CD', 'NNS', '.', '--s--', ':', 'IN', 'NNP', 'NNP', 'NN', 'CD', 'NNS', 'IN', 'NN', ',', 'NN', 'NN', 'NNS', 'IN', 'CD', 'NNS', '.', '--s--', ':', 'IN', 'NNP', 'NNS', 'VBP', 'CD', 'NNS', '(', 'NN', 'TO', 'RB', 'CD', 'NNS', 'IN', 'NNP', ')', ',', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'NN', 'TO', 'DT', 'NN', 'CC', 'JJR', '.', '--s--', 'IN', 'NNP', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'TO', 'VB', 'VB', 'CC', 'VB', 'NNS', '.', '--s--', ':', 'IN', 'NNP', 'NNS', 'VBP', 'CD', 'NNS', ',', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'NN', 'TO', 'DT', 'NN', 'CC', 'JJR', '.', '--s--', ':', 'IN', 'NNP', 'NNP', 'NN', 'CD', 'NNS', ',', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'CC', 'NNP', 'NNS', 'RB', 'VB', '.', '--s--', ':', 'IN', 'NNP', 'VBZ', 'CD', 'NNS', ',', 'NNP', 'NNP', 'NN', 'NN', 'IN', 'CD', 'NNS', '.', '--s--', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'NNS', 'RB', 'VBN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNS', '(', 'NNP', 'CD', ')', ':', '--s--', ':', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNS', '--s--', ':', 'NNP', 'JJ', 'NNS', '--s--', ':', 'VBP', 'NNS', 'JJ', 'IN', 'NN', 'CC', 'NNS', 'NNS', '--s--', 'NNP', 'NNS', '(', 'NNP', 'CD', ')', ':', '--s--', ':', 'NNP', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', 'NNS', '.', '--s--', ':', 'VB', 'NNP', 'NN', 'TO', 'VB', 'JJ', 'IN', 'NNS', 'IN', 'NN', 'NNS', '.', '--s--', ':', 'NNP', 'NN', 'IN', 'JJ', 'NNS', 'TO', 'NNP', 'IN', 'NNP', '.', '--s--', '(', 'NNP', 'IN', 'JJ', 'NNP', 'NN', ')', '--s--', ':', 'VB', 'NNP', 'NN', 'TO', 'VB', 'NNS', 'NN', ',', '(', 'RB', 'VBN', 'IN', 'JJ', 'NNP', 'NN', ')', '.', '--s--', 'JJ', 'NN', ':', '--s--', ':', 'VBG', 'DT', 'NN', 'NN', 'TO', 'VB', 'JJ', 'NN', 'IN', 'DT', 'NNS', 'NNS', 'CC', 'NNS', 'NNS', '.', '--s--', 'NNP', 'DT', 'JJ', 'NN', 'POS', 'NN', 'NN', ',', 'DT', 'JJ', 'NN', 'VBZ', 'VBN', 'NNS', 'TO', 'VB', 'NN', 'NN', ',', 'VBG', 'JJ', 'NNS', '.', '--s--', '``', 'EX', 'MD', 'RB', 'VB', 'DT', 'NN', ',', 'EX', 'MD', 'VB', 'NN', 'IN', 'NN', 'IN', 'DT', 'NNS', 'CC', 'NNS', ',', "''", 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', ',', 'VBD', 'DT', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'VBG', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'JJ', 'JJ', 'NNS', 'CC', 'VBZ', 'TO', 'VB', 'IN', 'DT', 'NN', 'RB', 'IN', 'IN', 'DT', 'NN', ',', 'DT', 'NN', 'VBD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'VBN', 'IN', 'PRP', 'RB', 'VBD', 'VBN', 'TO', 'DT', 'NNP', 'TO', 'VB', 'RP', 'DT', 'NNS', 'TO', 'VB', 'CD', 'NNP', 'NNS', 'IN', 'DT', 'NNP', 'NNP', '.', '--s--', 'NNP', 'POS', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NN', 'NN', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', 'RB', 'IN', 'PRP', 'MD', 'VB', 'JJ', 'IN', 'JJ', 'NN', ',', 'NNP', 'MD', 'VB', 'DT', 'NN', 'NN', 'WDT', 'MD', 'VB', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'POS', 'NNP', 'NNP', 'IN', 'NNPS', 'VBD', 'IN', 'IN', 'CD', 'NN', 'NNS', 'VBD', 'IN', 'JJR', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', '--s--', 'NNP', 'NNP', 'VBD', 'IN', 'NNS', 'IN', 'CD', 'IN', 'DT', 'NN', 'POS', 'NNS', 'VBD', 'IN', 'NN', ',', 'WDT', 'PRP', 'VBD', 'VBD', 'JJ', ',', 'IN', 'JJ', 'NNS', 'VBG', 'IN', 'CD', 'IN', 'DT', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'NNS', 'VBD', 'IN', 'NNP', 'NNP', 'VBG', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'POS', 'NN', 'NNP', 'NNP', 'VBD', 'IN', 'PRP$', 'NNS', 'VBD', 'VBN', 'VBD', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'VBD', 'VBN', 'VBN', 'IN', 'NNP', 'IN', 'DT', 'VBG', 'NNP', 'NNP', 'NNP', 'VBG', 'NN', 'VBZ', 'VBG', 'DT', 'NN', 'IN', 'JJ', 'NNS', ',', 'VBN', 'NN', ',', 'VBD', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', ',', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', ',', 'VBD', 'IN', 'CD', 'NN', 'NNS', 'VBD', 'VBN', 'NNS', 'IN', 'DT', 'NN', 'NN', 'VBG', 'CD', 'CD', 'NNS', '(', 'IN', '$', 'CD', ')', 'CC', 'VBD', 'DT', 'NNS', 'VBD', 'JJ', 'CC', 'NN', 'IN', 'PRP$', 'NNS', 'VBD', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'VBD', 'IN', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', 'VBD', 'VBG', 'RB', 'JJ', 'TO', 'VB', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'IN', 'PRP$', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'NNPS', 'IN', 'NNP', 'VBD', 'DT', 'CD', 'IN', 'NN', 'VBN', 'NNS', 'VBG', 'IN', 'VBG', 'TO', 'VB', '.', '--s--', 'NNP', 'NN', 'NNP', 'NNP', 'VBD', 'NN', 'NN', 'IN', 'NN', 'VBD', 'VBN', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'NN', 'RB', '.', '--s--', 'DT', 'NNP', 'NN', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'CD', 'IN', 'PRP$', 'CD', 'VBG', 'NNS', '.', '--s--', 'NNP', ',', 'NNP', 'CC', 'NNP', 'VBD', 'JJ', 'NNS', 'IN', 'VBG', ',', 'CC', 'DT', 'NNS', 'VBD', 'TO', 'VBG', 'NN', 'IN', 'DT', 'NNS', 'TO', 'VB', 'PRP', 'TO', 'JJ', 'NN', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'VBP', 'NNS', 'IN', 'NN', '.', '--s--', 'JJ', 'NN', 'IN', 'NNP', 'POS', 'NNP', 'NNP', 'VBZ', 'NN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'VBD', 'PRP$', 'JJ', 'NN', 'NN', 'NN', 'RBR', 'DT', 'NN', ',', 'DT', 'NNP', 'VBD', 'DT', 'NN', 'POS', 'NN', 'NN', 'NN', '.', '--s--', 'NNP', 'POS', 'NNP', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'VBN', 'TO', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBZ', 'CD', 'NNS', '(', 'CD', 'NNS', ')', 'IN', 'DT', 'CD', '(', 'IN', '$', 'CD', ')', 'IN', 'DT', 'NN', 'NNS', 'TO', 'DT', 'NN', '.', '--s--', 'IN', 'IN', 'JJ', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NN', ',', 'DT', 'NN', 'NN', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'VBZ', 'DT', 'JJ', '(', 'IN', 'DT', 'NN', ',', 'CD', 'NN', ')', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'POS', 'JJ', 'NN', 'NN', 'VBZ', 'VBN', 'DT', 'NNS', 'IN', 'JJ', 'JJ', 'JJ', 'NN', 'CC', 'VBG', 'NNS', 'CC', 'VBZ', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NN', 'NN', 'CC', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NNP', 'NNP', 'VBD', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', 'VBP', 'RB', 'VBN', 'IN', 'JJ', 'NNP', 'CC', 'VBD', 'DT', 'JJ', 'NN', 'NNS', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'VBG', 'NNS', 'RB', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'JJR', 'IN', 'CD', 'CD', 'NNS', '(', '$', 'CD', 'CD', ')', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'NNS', '.', '--s--', 'DT', 'NN', 'VBN', 'NNS', 'IN', 'VBG', 'DT', 'NNS', 'MD', 'VB', 'DT', 'NN', 'IN', 'NNS', 'CC', 'JJ', 'NNS', 'IN', 'NNS', 'TO', '``', 'VB', 'RB', 'JJ', "''", 'CC', 'VB', 'VBN', 'NNS', 'IN', 'NN', 'IN', 'DT', '``', 'RB', 'JJ', 'NN', 'NN', '.', "''", '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'NNS', 'VBD', 'TO', 'NN', 'JJ', 'NN', ',', 'WDT', 'VBD', 'TO', 'CD', 'NN', 'JJ', 'NN', ',', 'TO', 'CD', 'NN', 'IN', 'CD', '.', '--s--', 'NNS', 'RB', 'VB', 'DT', 'NN', 'NN', 'MD', 'VB', 'IN', 'CD', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'VBG', 'JJ', 'NN', 'IN', 'NN', ',', 'NNP', 'NNP', 'NNS', 'VBD', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'DT', 'NNP', 'POS', 'JJ', 'NNP', 'NNP', ',', 'WRB', 'NNS', 'VBD', 'NNS', ':', '.', '--s--', 'DT', 'CD', ',', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'VBG', 'NNP', '.', '--s--', 'DT', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBD', ':', '``', 'NNP', 'NNP', 'VBZ', 'PDT', 'DT', 'JJ', 'NN', ',', 'PRP', 'RB', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NN', '.', '--s--', 'PRP', 'RB', 'VBD', 'JJ', 'NN', ':', 'JJ', 'NNP', ',', 'CD', 'NNP', 'NNP', 'NNP', 'VBD', 'NNP', 'NNPS', 'NNP', 'RB', ',', 'VBD', 'PRP', 'IN', 'DT', 'JJ', 'NN', 'CC', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'POS', 'JJ', 'NN', 'NNS', 'IN', 'NNS', '.', '--s--', '``', 'PRP', 'VBZ', 'RB', 'JJ', 'TO', 'DT', 'NNP', 'NN', ',', "''", 'NNP', 'NNP', 'VBD', ',', 'IN', 'VBG', 'NNS', 'CC', 'NNS', '.', '--s--', 'NNP', 'DT', 'NNP', 'NN', ':', '``', 'NNP', '.', "''", '--s--', 'DT', 'NNP', 'NN', 'VBD', 'JJ', 'NNS', 'DT', 'CD', 'NN', 'IN', 'DT', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'POS', '``', 'PRP$', 'NNP', 'NNP', "''", 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'JJ', 'CC', 'VBD', 'TO', 'VB', '.', '--s--', 'CC', 'RB', 'IN', 'DT', 'JJ', 'NN', ':', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'VBG', 'NNP', ':', 'NNP', 'VBD', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'PRP', 'VBD', 'RP', ',', 'VBD', 'DT', 'JJ', 'NN', 'VBG', 'DT', 'JJ', 'NN', 'NN', 'TO', 'VB', 'NN', 'IN', 'NNP', '.', '--s--', 'PRP$', 'NN', 'RB', 'VBN', 'RP', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'VBG', 'TO', 'VB', 'NNP', 'POS', '``', 'PRP$', 'NN', "''", 'NN', 'NN', 'IN', 'DT', 'NNP', 'NN', ',', 'NNP', 'JJ', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'NNP', 'JJ', 'NN', 'VBG', 'DT', 'NN', 'IN', 'NN', 'NN', ',', 'JJ', 'NN', 'CC', 'JJ', 'NNS', 'IN', 'NN', 'NN', '.', '--s--', 'IN', 'RB', ',', 'NN', 'VBZ', 'VBN', 'DT', 'NN', 'IN', 'JJ', 'NNP', 'NNP', ',', 'NNP', 'POS', 'NN', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'VBN', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'NN', 'POS', 'NN', 'IN', 'DT', 'VBZ', 'VBN', 'VBG', 'RP', ',', "''", 'PRP', 'VBZ', '.', '--s--', '``', 'PRP', 'VBZ', 'DT', 'RBS', 'JJ', 'NN', 'IN', 'PRP$', 'NN', ':', 'PRP', 'MD', 'RB', 'RB', 'VB', 'TO', 'PRP$', 'NN', '.', "''", '--s--', 'PRP$', 'NN', '(', 'JJ', 'NNS', ':', 'IN', '$', 'CD', 'CD', ')', 'MD', 'VB', 'DT', 'JJ', 'NN', ':', 'VBG', 'DT', 'JJ', 'CD', 'NN', 'IN', 'JJ', 'NNS', ':', 'IN', 'NNP', '(', 'JJ', 'NNS', ':', 'IN', '$', 'CD', 'CD', ')', 'VBZ', '.', '--s--', 'NNP', 'POS', 'NN', 'NNS', 'WP', 'MD', 'VB', 'WRB', 'DT', 'JJ', 'NN', 'RB', 'VBZ', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'JJ', '.', '--s--', 'IN', 'JJ', 'NN', ',', 'DT', 'NN', 'RB', 'VBZ', 'WRB', 'RB', 'NNP', 'VBZ', 'VBN', 'DT', 'NNS', 'IN', 'NNP', 'NN', '.', '--s--', 'NNPS', 'VBD', 'TO', 'VB', 'RB', 'IN', 'VBG', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'NNP', '.', '--s--', 'RB', 'NNP', ',', 'WP$', 'JJ', ',', 'JJ', 'NNS', 'VBP', 'IN', 'DT', 'JJS', 'NN', 'IN', 'NN', 'NNS', ',', 'VBZ', 'RB', 'VBG', 'IN', 'DT', 'JJ', 'NNP', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'NNP', '.', '--s--', '``', 'DT', 'NN', 'IN', 'DT', 'VBZ', 'IN', 'NNP', 'VBD', 'RP', 'DT', 'NNP', 'NN', 'CC', 'NNP', 'VBZ', 'TO', 'VB', 'PRP', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'POS', 'NN', ',', 'WP', 'RB', 'VBN', 'NNP', 'POS', 'NNP', '.', '--s--', '(', 'NNP', 'PRP', 'VBZ', 'TO', 'VB', '.', ')', '--s--', 'IN', 'NNP', 'VBZ', ',', 'NNP', 'NNP', 'VBZ', ',', 'PRP$', 'JJ', 'NN', 'MD', 'VB', 'TO', 'VB', 'TO', 'VB', 'DT', 'NNS', 'RB', '.', '--s--', 'IN', 'JJS', ',', 'PRP', 'VBZ', ',', '``', 'PRP', 'MD', 'VB', 'TO', 'VB', 'PRP', 'RB', 'RB', 'CC', 'VB', 'PRP', 'RB', '.', "''", '--s--', 'NNP', 'NNP', 'VBZ', 'DT', 'NN', 'POS', 'NNS', 'CC', 'VBZ', 'PRP$', 'JJ', 'NN', 'VBD', 'VBG', 'IN', 'NNP', 'IN', 'DT', 'NN', '.', '--s--', '``', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'CC', 'PRP', 'VBD', 'PRP', 'JJR', ',', "''", 'PRP', 'VBZ', '.', '--s--', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBN', 'TO', 'VB', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBG', 'NN', 'IN', 'DT', 'JJ', 'NNS', ',', 'IN', 'WDT', 'NNS', 'VBD', 'IN', 'NNP', '.', '--s--', 'DT', 'NN', 'VBD', 'RP', 'DT', 'JJR', 'NNP', 'NN', 'IN', 'PDT', 'DT', 'NN', ':', 'DT', 'NN', 'VBD', 'IN', 'NN', ':', 'CC', 'NNP', 'VBD', 'IN', 'JJ', 'NN', 'CC', 'NNS', '.', '--s--', 'IN', 'VBG', 'TO', 'VB', 'DT', 'NN', ',', 'NNP', 'VBZ', 'TO', 'VB', 'RP', ',', 'IN', 'JJ', '.', '--s--', 'CC', 'DT', 'NN', 'VBZ', 'JJR', 'IN', 'NNP', 'NN', 'IN', '.', '--s--', '``', 'PRP', 'VBD', 'DT', 'NN', 'PRP', 'VBD', 'VBG', 'IN', 'RB', 'JJ', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'WP', 'VBD', 'NNP', 'IN', 'CD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'VBN', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', 'NN', ',', 'CC', 'JJ', ',', 'RB', 'TO', 'VB', 'PRP', 'VBD', 'VBN', '.', '--s--', 'IN', 'NNP', ',', 'PRP', 'VBD', 'VBG', 'NNS', 'CC', 'NNS', 'VBD', 'RB', 'IN', 'NNP', 'NNP', ',', 'VBG', 'IN', 'NNP', 'NNP', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBN', ':', 'TO', 'RB', 'CD', 'NNS', ',', 'IN', 'CD', 'RB', ',', 'NNP', 'NNP', 'VBZ', '.', '--s--', 'NNP', 'VBZ', 'VBN', 'JJ', 'IN', 'CD', ',', 'VBZ', 'DT', 'NN', ',', 'WP', 'VBZ', 'PRP$', 'NN', 'IN', 'JJ', 'NN', 'NNS', ',', 'DT', 'NNS', 'IN', 'PRP$', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'IN', 'JJ', 'NNS', ',', 'NNP', 'NNP', 'VBD', 'NN', 'IN', 'CD', 'WRB', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', ',', 'VBD', 'DT', 'JJ', 'NN', 'CC', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'JJ', 'JJ', 'NN', 'CC', 'VBD', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'NNS', '.', '--s--', 'DT', 'NN', 'VBN', 'JJ', 'NNS', 'VBD', 'JJR', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NNS', '.', '--s--', '(', 'IN', 'NN', 'NNS', ',', 'NNP', 'VBZ', 'PRP', 'VBZ', 'VBN', 'JJR', 'IN', '$', 'CD', 'CD', 'TO', 'VB', 'DT', 'NN', ',', 'IN', 'VBG', 'NNS', 'IN', 'IN', 'DT', 'CD', 'NNS', '.', ')', '--s--', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'NN', ',', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'PRP$', 'NNS', 'VBD', 'RB', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'PRP$', 'NNS', 'VBP', 'RB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CC', 'RB', 'VB', 'PRP', 'IN', 'NN', ',', 'NN', 'CC', ',', 'RB', ',', 'NN', '.', '--s--', 'NNP', 'VBZ', 'RB', 'JJ', 'CC', 'JJ', ',', 'IN', 'NNS', 'IN', 'JJ', ':', 'CC', 'RB', 'VBZ', 'NNP', ',', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'DT', 'NNS', '.', '--s--', '(', '``', 'TO', 'VB', 'JJ', ',', 'DT', 'NN', 'MD', 'VB', 'DT', 'NN', 'RB', ',', 'RB', 'RB', 'IN', 'PRP', 'VBZ', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'POS', 'NN', '.', ')', '--s--', 'IN', 'JJ', 'NN', ',', 'NNP', 'VBD', 'VBG', 'NNS', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ':', 'CC', 'NNP', 'VBD', '.', '--s--', 'RB', ',', 'VBN', 'IN', 'NNP', 'NNS', 'NNS', 'VBD', 'VBG', 'JJ', 'NN', 'CC', 'JJ', 'NN', ':', 'JJ', 'IN', 'VBG', 'JJ', 'NNP', 'NNS', ':', 'IN', 'NNP', 'POS', 'NNS', ',', 'NNP', 'NNP', 'VBD', 'RP', 'DT', 'NN', 'TO', 'NNP', 'IN', 'NNP', '.', '--s--', 'PRP', 'PRP', 'VBD', 'TO', 'VB', 'DT', 'NN', 'TO', 'DT', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', '.', '--s--', 'CC', 'NNP', 'VBZ', 'RB', 'VBD', 'DT', 'NNS', '(', 'WDT', 'VBD', 'IN', 'NN', ',', 'VBG', 'TO', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', ')', '.', '--s--', 'NNP', 'NNP', 'NNP', ':', '``', 'PRP$', 'NN', 'VBD', 'TO', 'VB', 'PRP', 'TO', 'VB', 'PRP', 'RB', '.', '--s--', 'PRP', 'RB', 'VBN', 'VBG', 'IN', 'NNP', ':', 'PRP', 'VBP', 'RB', 'VB', 'DT', 'NNS', '.', "''", '--s--', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'JJ', 'NN', '.', '--s--', 'IN', 'CC', 'IN', 'IN', 'RB', ',', 'DT', 'NNS', 'VBP', 'VBN', 'IN', 'NN', '.', '--s--', 'CC', 'NNP', ',', 'IN', 'DT', 'NN', 'NN', ',', 'VBZ', ',', '``', 'RB', 'JJ', ',', 'NNP', 'RB', 'VBZ', 'TO', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', 'TO', 'VB', 'RP', 'CC', 'VB', 'NN', 'NNS', '.', "''", '--s--', 'NNP', 'POS', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'POS', 'NNS', ':', 'VBG', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBG', 'IN', 'DT', 'NN', ':', 'VBP', 'JJ', 'IN', 'JJ', 'NNS', '.', '--s--', 'PRP', 'RB', 'VBZ', 'IN', 'NNP', ',', 'DT', 'JJR', 'IN', 'NNS', ',', 'VBZ', 'DT', 'JJ', 'NN', 'TO', 'VB', 'PRP$', '``', 'NN', 'NN', ',', "''", 'IN', 'DT', 'NN', ',', 'RB', 'DT', 'NNS', 'IN', 'PRP', 'VBZ', 'VB', 'PRP$', 'NNP', 'NNP', 'NNS', 'JJ', '.', '--s--', '(', 'NNP', 'VBZ', 'PRP', 'VBD', 'VBG', 'DT', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'NNS', 'RB', 'IN', 'NNP', 'VBD', 'DT', 'NNS', 'POS', 'NN', '.', ')', '--s--', 'WDT', 'PRP$', 'NNS', ',', 'NNP', 'POS', 'JJ', 'NN', 'VBZ', 'VBN', 'IN', 'NNP', '.', '--s--', 'PRP', 'VBZ', 'RB', 'JJ', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'PRP', 'VBZ', 'VBN', 'JJR', 'IN', '$', 'CD', 'IN', 'JJ', 'NNS', 'RB', 'RB', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNP', 'VBZ', ':', '``', 'PRP', 'VBP', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'PRP', 'TO', 'VB', '.', '--s--', 'PRP', 'RB', 'VB', 'VBN', 'PRP', 'IN', 'JJR', 'IN', 'PRP', 'VBP', 'DT', 'NN', 'WRB', 'PRP', 'VBP', 'RP', 'CC', 'VB', 'RP', '.', "''", '--s--', 'IN', 'RB', ',', 'RB', ',', 'PRP', 'VBZ', 'TO', 'VB', 'RP', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNP', 'NNP', ',', 'VBD', 'CD', 'NNS', ',', 'CC', 'CD', 'NN', ',', 'IN', 'NNP', 'POS', 'JJ', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'DT', 'NNPS', 'CC', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'IN', 'DT', 'RB', 'VBN', 'NN', ',', 'DT', 'NN', 'VBD', '.', '--s--', 'IN', 'RB', 'VBD', ',', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'CC', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'VBP', 'NNS', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'JJ', 'NNP', 'NNPS', 'NNPS', '.', '--s--', 'NNP', 'NNPS', 'VBZ', 'VBN', '$', 'CD', 'DT', 'NN', 'IN', 'NNP', 'CC', '$', 'CD', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'VBZ', 'VBN', 'DT', 'NNS', 'TO', 'NNP', 'CD', '.', '--s--', 'DT', 'NNS', 'VBD', 'DT', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'WP$', 'JJ', 'NN', 'VBZ', 'NNP', 'POS', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', '.', '--s--', 'DT', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'POS', 'JJ', 'JJ', 'NN', 'VBZ', 'IN', 'CD', 'NNS', '.', '--s--', 'NNP', 'VBZ', 'VBG', 'TO', 'VB', 'CD', 'NN', 'IN', 'NNP', 'POS', 'NN', 'IN', 'CD', 'NNS', '(', '$', 'CD', ')', 'DT', 'NN', '.', '--s--', 'DT', 'NNS', 'IN', 'DT', 'NN', 'VBD', 'DT', 'NN', 'IN', 'CD', 'CD', 'NNS', '(', '$', 'CD', 'CD', ')', 'IN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', 'DT', 'NNP', 'NNP', 'NN', 'NN', 'VBZ', 'CD', 'NN', 'IN', 'DT', 'NN', ',', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'CD', 'NN', 'CC', 'NNP', 'NNP', ',', 'DT', 'NN', 'NN', ',', 'VBZ', 'CD', 'NN', '.', '--s--', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'DT', 'NN', 'IN', 'CD', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', 'JJ', 'NN', 'TO', 'NNS', 'VBN', 'IN', 'PRP', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNPS', 'CC', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'NNP', ',', 'CC', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'CD', 'NNS', 'IN', 'NNP', 'CD', 'IN', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'PRP', ',', 'NNP', 'NNP', 'NNP', ',', 'NNP', ',', 'NNP', 'NNP', ',', 'CC', 'NNP', 'NNP', 'VBP', 'DT', 'JJ', 'CC', 'VB', 'DT', 'JJ', 'NN', 'IN', 'CD', 'NNS', ',', 'CC', 'CD', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBD', 'PRP', 'VBZ', 'TO', 'VB', 'JJR', 'NNS', '.', '--s--', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NNS', 'VBZ', 'RB', 'JJ', '.', '--s--', 'NNP', ',', 'NNP', 'NNP', ',', 'VBD', 'PRP', 'CC', 'DT', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NNPS', 'NNP', 'NN', 'IN', 'DT', '$', 'CD', 'CD', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'NN', 'NN', 'IN', 'NNP', 'CC', 'NNP', ',', 'DT', 'JJ', 'NN', 'NNS', 'NN', ',', 'VBD', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NN', 'NN', 'CC', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'VBD', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'DT', 'NN', 'VBD', 'NNP', 'CD', '.', '--s--', 'DT', 'NN', 'VBZ', '``', 'RB', 'JJ', "''", 'IN', 'DT', 'NN', 'NN', ',', 'NNP', 'NNP', 'VBD', ',', 'CC', 'PRP', 'VBD', 'TO', 'VB', 'NNS', '.', '--s--', 'IN', 'CD', ',', 'NNP', 'CC', 'NNP', 'POS', 'NNS', 'CC', 'JJ', 'JJ', 'NNS', 'VBD', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP', ',', 'NNP', '.', '--s--', 'IN', 'DT', 'JJS', 'NN', ',', 'NN', 'VBD', 'RP', 'DT', 'JJ', 'NNS', 'POS', 'NN', ',', 'NNP', 'NNP', 'VBD', '.', '--s--', 'NNP', 'NNP', ',', 'NNP', 'NNP', ',', 'NNP', ',', 'VBD', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP$', 'JJ', 'NN', ',', 'IN', 'WDT', 'PRP', 'MD', 'RB', 'VB', 'DT', 'JJR', 'IN', 'DT', 'NN', 'NN', 'POS', 'JJ', 'NN', ',', 'IN', 'NNP', 'CD', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'NNP', '.', '--s--', 'NNP', 'VBD', 'IN', 'PRP$', 'NNS', 'IN', 'NNP', ',', 'DT', 'NNP', ',', 'NNP', ',', 'JJ', 'NN', ',', 'VBP', 'VBG', ',', 'CC', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'VBP', 'JJ', '.', '--s--', 'NNP', 'VBZ', 'VBN', 'PRP$', 'NN', 'IN', 'NNP', 'TO', 'CD', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'VBD', 'IN', 'NNPS', 'CC', 'NNP', 'NNP', 'NNS', 'VBD', 'NNP', 'TO', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNPS', ',', 'DT', 'JJ', 'JJ', 'NN', 'WDT', 'VBZ', 'DT', 'NNP', 'NNP', 'NN', 'NN', 'IN', 'NN', 'TO', 'NNP', 'NNP', 'POS', 'NNP', 'NNPS', '.', '--s--', 'NNP', ',', 'DT', 'NNP', 'NNP', ',', 'NNP', ',', 'NN', 'IN', 'NNS', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'NN', 'NNP', 'NNP', ',', 'VBD', 'RB', 'VB', 'DT', 'NN', 'IN', 'VBG', 'PRP$', 'NN', 'TO', 'VB', 'DT', 'NN', ',', 'CC', 'NNP', 'NNS', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NN', '.', '--s--', 'NNP', 'VBD', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'NNP', 'VBZ', 'RB', 'RB', 'VBN', 'DT', 'JJ', 'NN', ',', 'IN', 'NNP', 'NNP', ',', 'DT', 'NNP', ',', 'JJ', 'NN', 'NN', 'NN', ',', 'VBZ', 'VBN', 'NNP', 'POS', 'NN', 'IN', 'IN', '$', 'CD', 'CD', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'VBN', 'DT', 'JJS', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'NNP', 'NNS', ',', 'WDT', 'VBP', 'VBN', 'IN', 'VBG', 'NN', ',', 'VBG', 'NN', 'CC', 'VBG', 'NN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBZ', 'VBN', 'JJ', 'NN', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'CD', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'IN', 'CD', 'NN', '.', '--s--', 'NNP', 'POS', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'IN', 'NNP', 'NNS', 'VBG', 'NNS', 'IN', 'NN', 'NNS', 'CC', 'VBZ', 'DT', 'JJ', 'NN', 'VBN', 'NNP', 'TO', 'NNP', '.', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NNP', 'CC', 'NNS', 'VBP', 'VBN', 'VBG', 'PRP$', 'NNS', 'IN', 'NNP', 'NNP', 'RB', 'IN', 'WP', 'PRP', 'VBP', 'VBN', '``', 'NN', 'NNS', ',', "''", 'IN', 'DT', 'NN', 'VBZ', 'VBN', 'VBN', 'IN', 'NN', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'VBD', 'RB', 'VB', 'RB', 'IN', 'PRP$', 'NNS', 'IN', 'VBG', 'DT', 'NN', ',', 'CC', 'NNS', 'RB', 'TO', 'DT', 'NN', 'VBD', 'PRP', 'VBD', 'VBN', 'TO', 'VB', 'RB', 'DT', 'NN', 'MD', 'VB', 'DT', 'NN', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'DT', 'NN', ',', 'MD', 'JJ', 'NN', 'NNS', 'VBP', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'VB', ',', 'RB', ',', 'WDT', 'EX', 'VBD', 'DT', 'NN', 'TO', 'JJ', 'NNS', 'DT', 'NN', 'MD', 'VB', 'VBG', 'IN', 'NN', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NNPS', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'VBD', 'PRP', 'VBD', 'TO', 'VB', 'IN', 'NNP', 'NNP', '.', 'NNP', 'IN', 'NNP', ',', 'NNP', ',', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NNP', 'NNP', 'VBZ', 'VBG', 'TO', 'VB', 'CD', 'JJ', 'JJ', 'NN', 'NNS', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'TO', '$', 'CD', 'CD', ',', 'IN', 'CD', 'CC', 'CD', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'DT', 'CD', ',', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'NNP', 'NNPS', 'NNP', ',', 'WDT', 'DT', 'NNP', 'NNP', 'VBZ', 'TO', 'VB', 'NN', 'NNS', '.', '--s--', 'NNP', 'VBD', 'DT', 'NNP', 'NNP', 'MD', 'RB', 'VB', 'DT', 'JJ', 'CD', 'NN', 'NN', 'TO', 'VB', 'PRP$', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'NNP', 'NNP', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'NNP', 'MD', 'VB', 'NNP', 'TO', 'VB', 'DT', 'NNP', 'CD', 'NN', 'NN', 'CC', 'MD', 'VB', 'JJ', 'NNS', '.', '--s--', 'NNP', 'MD', 'VB', 'JJ', 'NNS', 'CC', 'VB', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'NNP', ',', 'NNP', ',', 'NN', 'MD', 'DT', 'NNP', 'NNP', 'NN', 'DT', 'NN', '.', '--s--', 'DT', 'NNP', 'NN', 'IN', 'NNP', ',', 'NNP', ',', 'VBD', 'PRP', 'VBD', 'RB', 'JJ', 'IN', 'WDT', 'JJ', 'NNS', 'MD', 'VB', 'VBG', 'IN', 'DT', 'NNP', 'NNP', 'NN', '.', '--s--', 'NNP', 'NN', 'NNS', 'VBD', 'PRP$', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'RB', 'VB', 'NNS', 'VBD', 'IN', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'DT', 'NNP', 'NN', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNS', 'VBN', 'NN', 'NNS', 'CC', 'VBD', 'DT', 'NN', 'POS', 'NN', 'TO', 'VB', 'PRP$', 'NNS', '.', '--s--', 'DT', 'NNP', 'NNS', 'VBD', 'VBG', 'RP', 'IN', 'DT', 'NN', ',', 'WDT', 'VBZ', 'NN', 'NNS', ',', 'IN', 'DT', 'NN', 'IN', 'NNP', 'IN', 'VBG', 'CD', 'TO', 'CD', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'PRP$', 'NN', 'VBD', 'VBN', 'IN', 'NNS', 'IN', 'NNP', 'TO', 'VB', 'NNS', 'IN', 'NNP', 'IN', 'DT', 'JJ', 'NN', 'DT', 'JJ', 'NN', 'VBD', 'DT', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'NNS', 'NNS', ',', 'NN', 'NNS', ',', 'NN', 'NNS', 'CC', 'JJ', 'JJ', 'NNS', 'VBN', 'TO', 'DT', 'NNP', 'IN', 'NNP', 'MD', 'VB', 'DT', 'RBR', 'JJ', 'NN', 'TO', 'DT', 'NN', 'CC', 'DT', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBP', 'IN', 'PRP', 'VBP', 'RB', 'VBN', 'IN', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'PRP', 'VBP', 'VBZ', 'TO', 'VB', 'IN', 'NN', 'PRP$', 'NN', 'CC', 'NN', 'IN', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'NNS', ',', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'CC', 'JJ', 'NN', 'VBZ', 'DT', 'NN', ',', 'RB', 'VBN', 'NNP', '.', '--s--', 'NNP', 'NN', 'TO', 'DT', 'NN', 'IN', 'NNS', 'VBD', 'TO', 'VB', 'NNS', ',', 'RB', 'TO', 'VB', 'PRP', 'VBZ', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', ',', 'PRP', 'VBD', 'NN', 'NNS', 'VBP', 'RP', 'NN', 'NNS', 'IN', 'PRP$', 'NN', 'NNS', 'IN', 'VBG', 'JJ', 'TO', 'VB', 'RB', 'JJ', 'JJ', 'NNS', '.', '--s--', '``', 'DT', 'NN', 'NN', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'NN', 'RB', ':', 'EX', 'VBP', 'DT', 'NNS', ',', 'RB', 'VBZ', ',', "''", 'VBZ', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'RB', '$', 'CD', 'CD', 'CC', 'WP', 'RB', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', '``', 'RB', 'IN', 'RB', ',', "''", 'PRP', 'VBZ', '.', '--s--', 'NN', 'POS', 'NN', 'VBD', 'VBN', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'WDT', 'VBD', 'DT', 'JJS', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'CD', ',', 'CD', ',', 'NN', '.', '--s--', 'NNP', ',', 'PRP', 'RB', 'VBD', 'VB', 'DT', 'NN', 'POS', 'JJS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'NN', 'NN', 'IN', 'NNS', 'VBD', 'TO', 'VB', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'PRP', 'MD', 'VB', '.', '--s--', 'CC', '``', 'DT', 'JJ', 'NN', 'NNS', "''", 'DT', 'NN', 'NN', 'NNP', 'IN', 'NNS', 'VBD', 'IN', 'NNS', 'IN', 'JJ', 'NNS', 'IN', '``', 'DT', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NNPS', 'NNP', 'NNP', '.', '--s--', 'IN', 'NN', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'POS', '$', 'CD', 'CD', 'IN', 'CD', 'CD', 'NN', 'NNS', 'JJ', 'CD', ':', 'VBN', 'JJR', 'IN', 'CD', 'NNS', 'RB', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', ':', 'VBD', 'CD', 'NN', 'TO', 'RB', 'CD', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'RB', 'RB', 'NNP', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', 'NN', 'NNS', 'WDT', 'VBZ', 'VBN', 'IN', 'NN', '.', '--s--', 'NNP', 'NNS', 'VBD', 'TO', 'VB', '.', '--s--', 'NN', 'NNS', ',', 'WDT', 'VBD', 'IN', 'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'VBP', 'VBN', 'VBG', 'IN', 'NNS', 'IN', 'NN', 'IN', 'NN', 'VBD', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'PRP$', 'JJS', 'NN', 'JJ', 'NN', 'IN', 'NNP', 'NNP', ',', 'WDT', 'VBD', 'PRP$', 'NNP', 'NN', 'NN', 'IN', 'NN', 'NN', '.', '--s--', 'NNP', 'VBD', 'DT', 'NN', 'NN', 'WDT', 'VBD', 'PRP', 'TO', 'VB', 'JJ', 'IN', 'DT', 'NN', 'NNS', 'CC', 'TO', 'VB', 'PRP$', 'JJ', 'NN', 'NN', 'NN', 'RP', 'IN', 'NN', '.', '--s--', 'RB', ',', 'NNS', 'IN', 'NNS', ',', 'VBG', 'NNP', 'NNP', ',', 'NNP', 'NNPS', 'CC', 'NNP', 'NNP', ',', 'WDT', 'VBP', 'VBG', 'RP', 'IN', 'JJS', '$', 'CD', 'CD', 'IN', 'VBN', 'JJ', 'NN', 'NNS', 'TO', 'VB', 'PRP$', 'RB', 'JJ', 'NNS', 'CC', 'NNS', 'RB', ',', 'MD', 'RB', 'VB', 'DT', 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'VBN', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'NNP', 'NNP', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'VBD', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'RB', '.', "''", '--s--', 'IN', 'NNP', 'POS', 'NN', ',', 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', '``', 'RB', 'IN', 'DT', '$', 'CD', 'CD', 'JJ', 'NN', 'MD', 'VB', 'VBN', ',', 'VBG', 'IN', 'DT', 'NN', '.', "''", '--s--', 'IN', 'NNP', ',', 'NNP', 'NNP', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNS', 'IN', 'JJ', 'NNS', 'VBD', '``', 'RB', 'JJ', '.', "''", '--s--', 'DT', 'NN', '``', 'VBZ', 'JJ', 'IN', 'PRP', 'VBP', 'RB', 'VBG', "''", 'IN', 'JJ', 'NNS', ',', 'IN', 'JJ', ',', 'TO', 'VB', 'DT', 'IN', 'PRP$', 'NN', 'NN', 'NNS', 'TO', 'VB', 'RP', 'NNS', 'IN', 'JJ', 'NNS', ',', 'VBZ', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', ',', 'DT', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', '.', '--s--', 'RB', ',', 'DT', 'NNP', 'POS', 'NN', 'WDT', 'VBZ', 'DT', 'JJ', 'NN', 'NNS', 'IN', 'CD', 'JJ', 'JJ', 'NNS', 'VBD', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'NNP', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'DT', 'JJ', 'JJ', 'NNP', 'NN', 'VBD', 'JJR', 'IN', 'CD', 'CD', 'NNS', ',', 'CC', 'IN', '$', 'CD', 'IN', 'DT', '$', 'CD', 'NN', 'NN', ',', 'TO', 'CD', 'CD', ',', 'PRP$', 'JJS', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'POS', 'NN', 'VBD', 'TO', 'CD', 'NN', ',', 'DT', 'JJS', 'IN', 'NNP', 'CD', ',', 'CD', ',', 'VBG', 'TO', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NNS', ',', 'VBN', 'DT', 'NN', 'IN', 'DT', 'NNS', ',', 'VBD', 'IN', 'CD', 'NN', 'NN', 'TO', 'CD', 'NN', ',', 'DT', 'JJS', 'JJ', 'NN', 'IN', 'CD', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NN', 'NN', 'NN', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', '``', 'WRB', 'PRP', 'VBP', 'NN', 'IN', 'CD', 'NN', ',', 'PRP', 'VBP', 'NN', 'TO', 'NN', 'IN', 'DT', 'JJ', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'NN', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'RB', ',', 'DT', 'NNS', 'IN', 'DT', 'NN', 'NN', 'MD', 'VB', 'DT', 'NNP', 'NNP', 'TO', 'VB', 'NN', 'IN', 'DT', 'NNS', 'RB', '.', '--s--', '``', 'DT', 'NNS', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'NN', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNP', '.', '--s--', 'DT', 'JJ', 'NN', 'MD', 'VB', 'DT', 'NN', 'IN', 'NN', 'NNS', 'IN', 'RB', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'NN', 'CC', 'NN', 'NN', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', ',', 'WDT', 'VBZ', '$', 'CD', 'CD', 'IN', 'JJ', 'NNS', ',', 'VBZ', 'PRP', 'VBZ', 'RB', '``', 'VBP', 'EX', 'VBZ', 'DT', 'JJ', 'JJ', 'NN', '(', 'IN', 'DT', 'NN', 'NN', 'NN', ')', '.', '--s--', 'PRP', 'VBD', 'NN', 'NNS', '.', "''", '--s--', 'PRP', 'VBZ', ':', '``', 'DT', 'NN', 'NN', 'VBZ', 'VBG', 'DT', 'NN', 'CC', 'RB', 'DT', 'NNS', 'VBP', 'IN', 'IN', 'DT', 'NN', 'NN', 'VBZ', 'NNS', 'WDT', 'VBZ', 'DT', 'NN', 'MD', 'VB', 'JJ', 'CC', 'WDT', 'VBZ', 'JJ', 'IN', 'NN', '.', '--s--', 'PRP', 'VBP', 'RB', 'VB', 'WDT', 'VBZ', 'DT', 'NN', ',', 'CC', 'PRP', 'VBP', 'IN', 'NNS', 'VBP', 'VBG', 'VBN', '.', '--s--', 'EX', 'VBZ', 'DT', 'NN', 'TO', 'NN', ',', 'CC', 'DT', 'NN', 'VBZ', 'RB', 'IN', 'NNS', 'CC', 'RB', 'IN', 'NN', ':', 'PRP', 'VBZ', 'IN', 'NNPS', '.', "''", '--s--', 'RB', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'RP', 'IN', 'DT', 'JJ', 'NN', ',', 'NNS', 'VBD', 'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'IN', 'NN', ',', 'DT', 'JJ', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', 'NNP', 'NNS', 'VBN', 'IN', 'NNP', 'NNP', 'VBD', 'CD', 'NNS', 'VBN', '.', '--s--', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '.', 'JJ', 'NNS', 'VBD', 'JJ', 'NN', 'IN', 'NNP', 'VBD', 'VBN', '$', 'CD', 'CD', 'IN', 'IN', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNS', 'IN', 'IN', 'NNP', 'CC', 'NNP', 'RB', ',', 'NN', 'NN', 'NNS', 'MD', 'VB', '$', 'CD', 'CD', 'IN', 'NN', 'NN', 'NNS', '.', '--s--', '``', 'DT', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'EX', 'VBZ', 'CC', 'MD', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NN', 'VBG', 'RP', ',', "''", 'VBZ', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'VB', 'WRB', 'NNS', 'MD', 'VB', 'DT', 'NN', 'NN', 'NN', 'VBD', 'WRB', 'EX', 'VBD', '$', 'CD', 'CD', 'IN', 'NNS', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'VBD', 'JJ', 'NN', 'IN', 'NNP', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNP', '.', '--s--', '``', 'WRB', 'DT', 'NN', 'VBZ', ',', 'NN', 'NNS', 'MD', 'VB', 'VBG', 'PRP', 'IN', 'PRP', 'VBP', 'NNS', 'TO', 'VB', 'CC', 'VB', 'PRP', 'MD', '.', '--s--', 'EX', 'VBZ', 'RB', 'IN', 'NN', 'JJ', 'IN', 'NNS', 'WP', 'VBP', 'TO', 'VB', 'JJ', 'NNS', ':', 'PRP', 'VBZ', 'DT', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'JJ', 'IN', '.', "''", '--s--', 'CC', 'JJ', 'RB', 'JJ', 'NNS', 'VBD', 'TO', 'VB', 'VBN', 'DT', 'NN', ',', 'VBG', 'DT', 'NN', 'IN', 'JJ', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'NN', 'IN', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', ',', 'VBZ', 'DT', 'NN', 'IN', 'NN', 'NNS', 'VBZ', 'VBN', 'VBG', 'JJR', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'NNS', '.', '--s--', '``', 'JJ', 'NN', 'PRP', 'RB', 'VBD', 'PRP', 'RP', 'IN', 'DT', 'CD', 'JJ', 'NNS', ',', "''", 'PRP', 'VBZ', '.', '--s--', '``', 'DT', 'NN', ',', 'IN', 'JJS', ',', 'PRP', 'VBZ', 'IN', 'CD', 'IN', 'DT', 'CD', 'CC', 'CD', '.', '--s--', 'CC', 'PRP$', 'NN', 'NNS', 'VBP', 'RB', 'VBD', 'CD', 'NNS', '.', "''", '--s--', 'RB', ',', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'NN', 'VBZ', 'VBG', '``', 'CD', 'NN', 'IN', 'DT', 'NN', 'NN', "''", 'IN', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNS', 'VBP', 'RBR', 'VBN', 'IN', 'DT', '``', 'JJ', "''", 'NN', 'IN', 'DT', '``', 'JJ', "''", 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'VBN', 'JJ', 'IN', "''", 'DT', 'NN', 'IN', 'DT', 'NNP', 'NN', 'NN', 'CC', 'DT', 'NN', 'POS', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', ',', 'VBZ', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', '.', '--s--', '``', 'DT', 'RB', 'VBN', 'DT', 'NN', 'IN', 'DT', 'NNS', 'VBD', 'TO', 'VB', '``', 'WP', 'RB', 'VBZ', 'IN', 'NN', '.', "''", "''", '--s--', 'RB', 'IN', 'NNP', ',', 'RB', ',', 'EX', 'VBD', 'NNS', 'IN', 'DT', 'NN', 'NN', 'VBD', 'VBG', 'IN', 'CD', 'IN', 'PRP$', 'JJS', 'NNS', 'RB', '.', '--s--', 'IN', 'DT', 'RB', 'JJ', 'NN', ',', 'NN', 'NN', 'NNS', 'VBD', 'NN', 'IN', 'VB', 'RP', ',', 'VBN', 'IN', 'DT', 'RB', 'JJ', 'NN', 'IN', 'JJ', 'NN', ':', '--s--', ':', 'IN', 'NNP', ',', 'CD', 'NNS', 'IN', 'PRP', 'MD', 'VB', 'IN', 'NN', 'NNS', 'VBG', 'DT', 'IN', 'PRP$', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'NNS', ',', 'NNP', 'JJ', 'NNP', 'NNPS', 'NNP', 'VBD', 'PRP', 'VBD', 'RP', 'IN', 'VBN', 'NN', '.', '--s--', ':', 'IN', 'NNP', ',', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NN', 'CC', 'JJ', 'NNS', 'NN', 'IN', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', ',', 'RB', 'VBN', 'IN', 'NN', 'IN', 'NNP', 'NN', 'NN', '.', '--s--', ':', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'NNS', 'CC', 'JJ', 'NNP', 'VBZ', 'VBN', 'TO', 'JJR', 'IN', 'CD', 'NN', 'NNS', '.', '--s--', ':', 'IN', 'NNP', ',', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'VBD', 'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', ',', 'VBN', 'NN', 'NNS', '.', '--s--', ':', 'IN', 'NNP', ',', 'RB', 'IN', 'DT', 'NN', 'NN', 'VBD', 'NNP', ',', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', '$', 'CD', 'CD', 'IN', 'NNS', 'CC', 'VBD', 'JJ', 'TO', 'VB', 'NN', 'IN', 'DT', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', '.', '--s--', 'RB', ',', 'NNS', 'VBP', 'VBG', 'RB', 'VBN', 'IN', 'DT', 'NN', 'TO', 'NN', 'VBZ', 'DT', 'NNS', 'IN', 'NNS', '.', '--s--', 'NNP', 'POS', 'VBZ', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NN', 'VBZ', 'DT', 'JJS', 'DT', 'NN', 'IN', 'CD', '.', '--s--', 'IN', 'NN', ',', 'EX', 'VBP', 'CD', 'NNS', 'IN', 'JJ', 'JJ', 'NNS', 'IN', 'EX', 'VBD', 'IN', 'DT', 'NN', 'IN', 'CD', ',', 'VBG', 'TO', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', '``', 'DT', 'NN', 'IN', 'DT', 'NNS', 'VBZ', 'RB', 'JJ', 'NN', 'CC', 'DT', 'VBG', 'IN', 'PRP', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'NN', '.', '--s--', '``', 'WP', 'PRP', 'VBP', 'VBN', 'TO', 'VB', 'VBZ', 'DT', 'NN', 'TO', 'NNS', 'IN', 'VBG', 'JJ', 'NN', 'NN', '.', '--s--', 'NNS', 'VBD', 'VBG', 'IN', 'NN', 'NNS', 'JJR', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'WP', 'PRP', 'VBP', 'VBG', 'RB', 'VBZ', 'DT', 'JJ', 'NNS', 'IN', 'NN', 'NNS', 'TO', 'DT', 'NN', 'WRB', 'JJ', 'NN', 'MD', 'VB', 'JJ', 'TO', 'VB', 'RB', 'IN', 'JJ', '.', "''", '--s--', 'NNP', 'POS', 'NNP', 'NNP', '--s--', 'NNS', 'IN', 'NNP', 'NNS', 'VBD', 'IN', 'DT', 'JJS', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBD', 'DT', 'VBG', 'NN', 'NN', '.', '--s--', 'DT', 'JJ', 'JJ', 'NNP', 'NN', 'VBD', 'VBN', 'CD', 'NN', 'NNP', 'IN', 'CD', 'CD', ',', 'VBN', 'IN', 'CD', 'CD', 'NNP', ',', 'RB', 'CD', 'CD', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'TO', 'CD', 'NN', ',', 'DT', 'JJS', 'IN', 'NNP', 'CD', ',', 'CD', ',', 'VBG', 'TO', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', '``', 'NN', 'TO', 'NN', "''", 'VBD', 'RB', 'IN', 'DT', 'NN', 'CC', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'NNPS', 'VBD', 'JJR', ',', 'VBG', 'RB', 'TO', 'NN', 'IN', 'DT', 'NN', 'NN', 'NN', ':', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', ':', 'VBN', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNS', 'RB', 'VBN', 'IN', 'NN', '.', '--s--', 'JJ', 'NN', 'NN', 'NNS', 'MD', 'VB', 'RB', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', ',', 'WDT', 'MD', 'VB', 'PRP$', 'NNP', 'CD', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'NN', 'TO', 'VB', 'DT', 'JJ', 'NNS', 'CC', 'VB', 'DT', 'NN', 'IN', 'VBG', 'IN', 'DT', 'NN', '.', '--s--', 'NNS', 'IN', 'NNS', ',', 'JJ', 'NNS', 'CC', 'JJ', 'NNS', 'RB', 'VBD', ',', 'CC', 'VBN', 'IN', 'PRP$', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBD', 'IN', 'JJ', 'NN', ',', 'IN', 'JJS', 'IN', 'DT', 'NN', 'VBN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'CD', 'NN', 'NN', 'NNS', ',', 'DT', 'RBS', 'JJ', 'NN', 'NN', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'CD', 'NN', 'NN', 'VBD', 'IN', 'CD', 'CD', ',', 'RB', 'CD', 'NN', 'IN', 'DT', 'NN', ',', 'TO', 'VB', 'IN', 'CD', 'NN', 'TO', 'DT', 'JJ', 'JJ', 'NN', 'NN', '.', '--s--', 'JJ', 'JJ', 'NNS', 'VBD', 'RP', 'IN', 'CD', 'TO', 'CD', 'NN', '.', '--s--', 'CC', 'DT', 'NN', 'NN', 'IN', 'JJ', ',', 'JJ', 'NNS', 'CC', 'JJ', 'NNS', 'VBD', '.', '--s--', 'CC', 'DT', 'NNS', 'IN', 'NN', 'CC', 'NN', 'NNS', 'VBD', 'JJ', 'TO', 'JJ', 'JJ', 'NNS', 'IN', 'NN', 'IN', 'DT', 'NN', 'POS', '$', 'CD', 'CD', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'POS', 'JJ', 'NN', 'MD', 'VB', 'VBN', 'IN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'NNS', ',', 'NNS', 'IN', 'DT', 'NN', 'NN', 'VBD', 'PRP', 'VBP', 'DT', 'NN', 'MD', 'VB', 'VBN', 'RB', 'VBN', '.', '--s--', 'CD', 'IN', 'DT', 'NNS', 'VBD', 'DT', 'JJR', 'NNS', 'IN', '$', 'CD', 'CD', 'DT', 'IN', 'JJ', 'CC', 'JJ', 'NNS', 'VBP', 'RB', 'VBN', '``', 'RB', 'VBN', '.', "''", '--s--', 'JJ', 'NNS', 'VBD', 'RB', 'RB', 'IN', 'CD', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'TO', 'DT', 'NN', '.', '--s--', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'NNS', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NN', 'WDT', 'VBD', 'JJ', 'NNS', 'RB', 'RB', 'TO', 'PRP$', 'CD', 'NNP', 'NN', ',', 'CC', 'NNS', 'VBP', 'VBN', 'VBN', 'IN', 'NN', 'TO', 'VB', 'NNS', 'RBR', 'JJ', '.', '--s--', 'RB', ',', 'DT', 'NN', 'VBZ', 'IN', 'DT', 'JJ', ':', 'IN', 'NNP', 'DT', 'NN', ',', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'IN', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'NN', ',', 'CC', 'PRP', 'VBZ', 'RB', 'JJ', 'DT', 'NN', 'VBD', 'RB', 'IN', 'DT', 'NNP', 'NN', 'NN', '.', '--s--', 'RB', ',', 'DT', 'NN', 'VBZ', 'RB', 'IN', 'JJ', 'IN', 'CD', 'VBG', 'NN', 'VBZ', 'DT', 'NN', 'TO', 'VB', 'RP', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'PRP', 'TO', 'DT', 'NN', 'IN', 'NN', 'VBG', 'IN', 'DT', 'NNP', 'NNP', 'VBZ', 'RB', 'IN', 'DT', 'NN', ':', 'DT', 'NN', 'IN', 'NNP', 'NNPS', 'IN', 'VBG', 'DT', 'NN', 'NNS', 'NN', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNS', ',', 'IN', 'DT', ',', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNS', '(', 'RB', 'DT', 'RBR', 'RB', 'VBN', 'DT', 'NNS', 'IN', 'NNS', 'IN', 'NNS', 'WDT', 'VBP', 'NNS', 'VBP', 'RB', '``', 'JJ', 'NN', "''", ')', '.', '--s--', 'CC', 'NN', 'NNS', 'RB', 'VBP', 'VBN', 'VBN', 'IN', 'DT', 'NN', 'NNS', 'NN', 'IN', 'DT', 'NNP', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NN', ',', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'NN', ',', 'VBG', 'PRP', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'IN', 'DT', 'NN', 'NNS', 'VBD', '.', '--s--', 'EX', 'VBZ', 'DT', 'JJ', 'NN', 'TO', 'VB', 'VBN', ',', 'TO', 'VB', 'JJ', ',', 'IN', 'VBG', 'DT', 'NN', 'RP', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'PRP', 'MD', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'VBD', 'TO', 'VB', 'NNS', 'JJ', 'IN', 'PRP$', 'JJ', 'NNS', 'IN', 'NNS', '.', '--s--', 'IN', 'CD', 'NN', ',', 'DT', 'CD', 'NNS', 'IN', 'DT', 'NN', 'NNS', 'NN', 'MD', 'VB', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'IN', 'NNP', 'NNS', 'VBP', 'RB', 'RB', 'VB', 'PRP', 'RP', 'IN', 'JJ', 'NNS', ',', 'VBZ', 'IN', 'DT', 'JJR', 'NN', 'VBP', 'JJ', '.', '--s--', 'RB', 'RB', 'IN', 'NNP', 'NNP', 'VBZ', 'VBN', 'VBN', 'IN', 'DT', 'NNP', 'NN', '.', '--s--', 'TO', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNS', 'VBD', 'TO', 'DT', 'NN', ',', 'PRP', 'VBP', 'DT', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'JJS', 'NNS', 'IN', 'JJ', 'NNS', 'VBP', 'VBN', 'IN', 'NN', ',', 'IN', 'DT', 'NN', 'DT', 'NN', 'MD', 'VB', 'TO', 'VB', 'PRP', '.', '--s--', 'CC', 'DT', 'NNS', 'VBP', 'RB', 'DT', 'JJ', 'NN', ',', 'VBN', 'JJ', 'NNS', 'IN', 'NNP', 'IN', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'TO', 'VB', 'NN', 'MD', 'VB', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNPS', 'JJ', 'NNS', 'IN', 'NNS', 'WDT', 'VBP', 'NNS', '.', '--s--', 'RBR', ',', 'NNP', 'VBZ', 'VBN', 'TO', 'VB', 'IN', 'DT', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NNP', 'WP', 'VBZ', 'DT', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'POS', 'NN', 'NN', ',', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'JJ', 'NN', 'RB', 'IN', 'PRP', 'MD', 'VB', 'VBN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NNP', 'CC', 'NNP', 'NNP', 'NNPS', 'NNS', '.', '--s--', 'PRP', 'MD', 'VB', 'NNP', 'NNP', 'RB', 'TO', 'CD', 'NNS', 'TO', '``', 'NN', "''", 'DT', 'NN', 'IN', 'CD', 'NN', 'CC', 'JJR', 'IN', 'DT', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNS', 'IN', '$', 'CD', 'CD', 'CC', 'JJR', '.', '--s--', 'IN', 'DT', 'NNP', 'NN', 'VBZ', 'NNS', ',', 'CC', 'DT', 'NN', 'VBZ', 'CD', 'NNS', '.', '--s--', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', '.', '--s--', 'IN', 'DT', 'CD', 'NN', ',', 'VBP', ',', 'DT', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'TO', 'NN', 'NNS', 'RP', 'IN', 'NNS', '.', '--s--', 'RB', 'RBR', 'JJ', ',', 'IN', 'PRP$', 'NN', ',', 'VBD', 'DT', 'NNP', 'POS', 'NN', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'DT', 'NNP', 'VBZ', 'VBG', 'DT', 'JJ', 'NN', 'NN', ':', 'NNS', ',', 'DT', 'NN', 'VBZ', 'RB', 'IN', 'CD', 'NN', '.', '--s--', 'RB', ',', 'NNS', 'VBP', 'IN', 'JJR', 'NN', 'NN', 'IN', 'IN', 'CD', 'TO', 'VB', 'VBG', 'NNS', '.', '--s--', 'PRP', 'VBP', 'RB', 'VBN', '.', '--s--', 'PRP', 'VBP', 'IN', 'JJR', 'NN', 'IN', 'VBG', 'NN', 'RB', 'IN', 'IN', 'NN', 'VBZ', 'CC', 'NN', 'CC', 'NN', 'NNS', '.', '--s--', 'DT', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'VBZ', 'VBN', ':', 'PRP', 'MD', 'VB', 'JJ', 'TO', 'VB', 'IN', '``', 'NN', 'NNS', "''", 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'DT', 'NNS', 'VBP', 'NNS', 'IN', 'JJ', 'NN', ',', 'IN', 'CD', '.', '--s--', 'EX', 'VBZ', 'NN', 'JJ', 'IN', 'DT', 'NN', 'WDT', 'MD', 'RB', 'VB', 'VBN', 'IN', 'DT', 'JJ', 'NNS', 'CC', 'JJ', 'NN', 'IN', 'NNP', '.', '--s--', 'CC', 'IN', 'DT', 'JJ', 'NN', ',', 'WDT', 'MD', 'VB', 'RB', 'RB', 'TO', 'VB', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'VBG', 'JJ', 'JJ', 'NNS', 'IN', 'RB', 'VBG', 'TO', 'PRP$', 'NNS', 'IN', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ',', 'IN', '$', 'CD', 'CD', 'IN', 'NNS', ',', 'VBD', 'PRP', 'VBD', 'RB', '$', 'CD', 'CD', 'TO', 'VB', 'NNS', 'IN', 'NNS', 'CC', 'NNS', 'IN', 'RB', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', ',', 'IN', 'JJ', 'IN', 'CD', 'JJ', 'CD', 'NNS', 'TO', 'JJ', 'NNS', ',', 'VBZ', 'DT', 'NN', 'TO', 'DT', 'NN', 'VBG', 'CD', 'NN', 'IN', 'JJ', 'CC', 'JJ', 'NNS', 'JJ', 'TO', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'IN', 'CD', 'VBZ', 'VBN', 'PRP$', 'NNS', 'TO', 'JJ', 'NNS', 'TO', '$', 'CD', 'CD', 'IN', '$', 'CD', 'CD', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'NNP', 'VBD', 'PRP', 'VBZ', 'RB', 'VBP', 'TO', 'VB', 'NN', 'TO', 'VB', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'DT', 'NN', 'VBD', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', '.', '--s--', 'JJ', 'JJ', 'NN', 'CC', 'NN', 'NN', 'NNS', 'VBD', 'CD', 'NN', 'IN', 'JJ', 'NN', 'POS', 'JJ', 'NN', ',', 'TO', '$', 'CD', 'CD', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNP', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'NN', 'NN', ',', 'DT', 'JJ', 'JJ', 'NN', 'NN', ',', 'VBD', 'CD', 'NN', 'TO', '$', 'CD', 'CD', 'IN', '$', 'CD', 'CD', 'DT', 'NN', 'RB', '.', '--s--', 'NN', 'NN', 'CC', 'JJS', 'NN', 'NN', 'VBD', 'JJ', '.', '--s--', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'DT', 'NN', 'VBZ', 'VBN', 'IN', 'NNP', 'CD', '.', '--s--', 'IN', 'DT', 'VBG', 'TO', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NN', 'VBN', ':', 'IN', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', 'CC', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', ':', 'EX', 'VBZ', 'CC', 'CD', 'JJ', 'NN', ':', 'DT', 'NNS', 'RB', 'JJ', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'CC', 'PRP$', 'JJ', 'JJ', 'NN', 'VBP', 'IN', 'VBZ', 'VBN', 'IN', 'DT', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', ',', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'TO', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'JJ', 'NN', 'NN', 'WDT', 'VBD', 'IN', 'JJ', 'NNS', 'VBD', ',', 'IN', 'NN', ',', 'RB', 'JJR', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', ':', 'IN', 'CD', 'NN', ',', 'DT', 'JJ', 'NNP', 'NNP', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', '.', '--s--', 'IN', 'DT', 'JJ', ',', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NNP', 'CC', 'NNP', ':', 'DT', 'NN', 'CC', 'RB', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NNP', 'NNP', 'IN', 'NNP', 'CC', 'PRP$', 'JJ', 'NN', ',', 'DT', 'NNP', 'NNP', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'RB', ':', 'NNP', '(', 'NNP', 'NNP', 'POS', 'NN', ')', 'IN', 'JJ', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'VBD', 'DT', 'NN', 'IN', 'NN', 'NN', ',', 'VBG', 'IN', 'DT', 'NNS', ',', 'CC', 'DT', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'CC', 'PRP$', 'JJS', 'NNS', '.', '--s--', 'IN', 'NN', 'IN', 'NN', ',', 'DT', 'NN', 'VBD', 'JJ', 'IN', 'NNS', '.', '--s--', 'WRB', 'VBN', 'TO', 'VB', 'NN', 'IN', 'JJ', 'NN', 'NNS', ',', 'RB', ',', 'DT', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', ',', 'IN', 'NNS', 'RB', 'IN', 'JJ', 'NNS', ',', 'RB', 'NN', '.', '--s--', 'IN', 'NN', 'TO', 'NNP', 'POS', 'JJ', 'JJ', 'NN', ',', 'NNP', 'NNP', 'POS', 'JJ', 'NN', 'VBZ', 'NNP', 'NNP', ',', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'IN', 'NNP', 'CC', 'JJ', 'NNS', ',', 'RB', 'VBN', 'IN', 'DT', 'NN', 'NN', 'IN', 'NNP', ',', 'NNP', ',', 'IN', 'WRB', 'PRP', 'VBZ', 'VBG', 'NN', 'NNS', 'WDT', 'MD', 'VB', 'PRP', 'TO', 'NNP', '.', '--s--', 'NNP', 'NNP', 'POS', 'NN', 'VBZ', ',', 'IN', 'JJS', ',', 'JJ', '.', '--s--', 'PRP', 'VBZ', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', ',', 'CC', 'PRP$', 'NN', 'VBZ', 'RB', 'VBN', 'VBN', 'JJR', 'IN', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'DT', 'NNP', 'NNP', 'CC', 'JJ', 'NNS', 'VBD', 'PRP$', 'NNS', 'IN', 'NN', 'IN', 'NNP', 'NN', '.', '--s--', 'IN', 'JJ', 'NNS', 'VBP', 'IN', 'NNP', 'NNP', 'VBD', '$', 'CD', 'CD', 'IN', 'JJ', 'NNP', 'IN', 'NNP', 'NNS', 'CC', 'DT', 'JJ', '$', 'CD', 'IN', 'NNS', ',', 'RB', 'VBD', ':', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'NN', 'NNS', 'TO', 'VB', 'DT', '$', 'CD', 'CD', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'CC', 'RB', 'VBN', 'RP', 'DT', 'NN', ':', 'CC', ',', 'WP', 'NNP', 'POS', 'NN', 'VBD', 'IN', 'DT', '$', 'CD', 'CD', 'NNP', 'NNP', 'NNS', '.', '--s--', 'CD', 'JJ', 'NNS', 'VBD', 'RB', 'RB', 'VBN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'NNS', 'IN', 'NNP', 'VBD', 'TO', 'VB', 'PRP', 'TO', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'JJ', 'JJ', 'NN', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', 'VBG', 'NNP', 'NNP', 'VBD', 'IN', 'NNP', 'NN', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', ',', 'DT', 'NN', 'VBD', 'RB', 'JJ', 'NN', 'CC', '``', 'NNS', "''", 'IN', 'NN', 'TO', 'NNP', ',', 'RB', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'VBD', 'VBN', 'WRB', 'PRP$', 'JJ', 'NN', 'NN', ',', 'DT', 'NNP', ',', 'RB', 'VBN', 'DT', 'NN', 'NNS', 'IN', 'JJ', 'NNS', ',', 'VBG', 'JJ', 'NN', 'NNS', ',', 'NNS', 'CC', 'NNP', 'NN', 'NNS', '.', '--s--', 'IN', 'JJ', 'NNS', ',', 'PRP', 'VBD', 'RB', 'VBN', 'IN', 'NNP', 'NNP', 'RB', 'VBN', 'CC', 'VBD', 'DT', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'DT', '``', 'NNS', "''", 'NNS', 'VBD', 'VBN', 'RP', 'RB', 'RB', 'WRB', 'NNS', 'VBN', 'IN', 'NNS', 'NNS', 'CC', 'NNS', '.', '--s--', 'DT', 'NN', 'NN', 'VBD', 'TO', 'VB', 'IN', 'NNP', 'NNS', 'VBD', 'VBN', 'JJ', 'NNS', 'IN', 'VBG', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'CC', 'NNP', 'CD', 'NN', 'NN', ',', 'VBN', 'IN', 'DT', 'JJ', 'NNP', 'NNPS', 'NNP', 'CC', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', ',', 'RB', '.', '--s--', 'RB', ',', 'CC', 'NNP', 'NNPS', 'CC', 'NNP', 'MD', 'VB', 'VBN', 'TO', 'VB', 'PRP$', 'JJ', 'JJ', 'NNS', 'IN', 'VBG', 'NNS', 'IN', 'NNS', 'VBN', '(', 'CC', 'RB', ')', 'TO', 'JJ', 'JJ', 'NNS', 'IN', 'NNS', 'VBD', '.', '--s--', 'RB', 'PRP', 'VBZ', 'IN', 'NNP', 'NNP', 'CC', 'PRP$', 'JJ', 'NNS', 'MD', 'VB', 'NNS', 'VBD', 'NNP', 'NNP', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', ':', 'NNP', ',', 'IN', 'JJ', 'CC', 'RB', ',', 'MD', 'RB', 'VB', 'DT', 'NN', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'POS', 'NN', '.', '--s--', 'CC', 'IN', 'JJ', 'JJ', 'NNS', 'RB', 'MD', 'RB', 'VB', 'RB', 'VB', 'NN', ',', 'NNP', 'NNP', 'MD', 'RB', 'VB', 'VBN', '.', '--s--', 'NNP', 'VBD', 'PRP$', 'JJ', 'NNS', ':', 'PRP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NNP', 'CC', 'NNP', 'VBD', 'VBN', 'RP', 'PRP$', 'JJ', 'NNS', 'IN', 'VBG', 'IN', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'JJ', 'NNS', 'CC', 'VBG', 'DT', 'NN', 'IN', 'NN', 'IN', 'NNP', '.', '--s--', 'RB', ',', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'DT', 'NN', ',', 'PRP', 'VBZ', 'DT', 'JJR', 'JJ', 'NNS', 'TO', 'VB', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', ',', 'PRP', 'VBD', 'TO', 'VB', 'DT', 'NNS', ':', 'JJ', 'IN', 'WP', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'POS', 'NN', ':', 'IN', 'VBG', 'IN', 'VBG', 'DT', 'JJ', 'NNP', 'NNP', 'NN', '.', '--s--', 'IN', 'NN', ',', 'CC', 'IN', 'NNS', 'IN', 'VBG', 'DT', 'JJ', 'NN', 'NNS', ',', 'PRP', 'VBD', 'TO', 'VB', 'DT', 'JJ', 'NNP', 'NNP', 'NNP', 'NN', 'CC', 'VBN', 'JJ', 'NN', 'IN', 'NNP', 'NNP', '.', '--s--', 'RB', 'JJ', 'NNS', 'IN', 'DT', 'NNS', 'RB', 'RB', 'IN', 'JJ', 'NN', ',', 'CC', 'RB', 'IN', '``', 'NN', 'IN', 'NN', "''", 'IN', 'NNS', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'NNP', 'IN', 'DT', 'NNS', 'VBP', 'IN', 'NNP', ',', 'VBG', 'NNS', 'IN', 'NN', 'VBP', 'VBG', 'VBN', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'POS', 'JJ', 'NNP', 'NN', 'VBZ', 'VBN', ',', 'IN', 'NN', '.', '--s--', 'NNP', 'RB', 'MD', 'VB', 'JJ', 'NNS', 'IN', 'NN', 'IN', 'NN', 'IN', 'DT', 'CD', 'NNS', 'IN', 'JJ', 'JJ', 'NN', 'NN', '.', '--s--', 'NNP', 'POS', 'JJ', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'NN', 'RB', 'RB', 'RB', '.', '--s--', 'IN', 'CD', ',', 'DT', 'JJ', 'NN', 'VBG', 'NNP', 'JJ', 'NNS', 'IN', 'NNP', 'NNS', 'IN', 'NNP', 'CD', '.', '--s--', 'NNP', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NNP', 'NNS', ',', 'CC', 'DT', 'NN', 'VBZ', 'VBG', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'WP', 'VBZ', 'VBN', 'IN', 'DT', 'NNP', 'IN', 'DT', 'CD', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNPS', 'NN', '.', '--s--', 'DT', 'JJ', 'NNS', 'VBP', 'VBN', 'IN', 'NN', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'CC', 'DT', 'NN', 'VBZ', 'JJ', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'NN', '.', '--s--', 'DT', 'NNS', 'VBP', 'RB', 'JJ', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'IN', 'NNP', 'MD', 'VB', 'VBN', 'IN', 'CC', 'RB', 'IN', 'JJ', 'NNS', 'TO', 'VB', 'NN', '.', '--s--', 'DT', 'NNP', 'MD', 'VB', 'DT', 'NNS', 'RB', 'VBZ', 'VBN', ':', 'DT', 'JJ', 'NN', ',', 'RB', 'JJ', 'TO', 'VB', 'RP', 'NNP', 'POS', 'NN', ',', 'VBZ', 'VBN', 'IN', 'NNP', 'NNP', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBZ', 'DT', 'RB', 'JJ', '.', '--s--', 'IN', 'NNP', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', 'NN', ',', 'PRP', 'MD', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'PRP', 'MD', 'VB', 'JJ', 'NNS', 'RB', 'JJ', 'NN', 'IN', 'JJ', 'DT', 'NNS', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'RB', 'RB', 'VBZ', 'VBN', 'DT', 'NN', 'CC', 'VBD', 'PRP$', 'JJ', 'NNS', 'TO', 'JJ', 'NN', 'IN', 'NNP', ',', 'VBG', 'TO', 'VB', 'TO', 'DT', 'NN', 'NNS', 'CC', 'DT', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'IN', 'IN', 'NNP', 'NNP', '.', '--s--', 'PRP', 'VBZ', 'RB', 'RB', 'VBG', 'RB', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'IN', 'PRP', 'VBZ', 'RB', 'CD', 'IN', 'DT', 'RBS', 'JJ', 'NN', 'NNS', 'WP', 'RB', 'JJ', 'NN', ',', 'PRP', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'PRP', 'RB', 'VBN', ',', 'CC', 'RB', 'RB', 'RB', 'VBN', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', ',', 'VBZ', 'VBN', 'IN', 'NNP', 'IN', 'CD', '.', '--s--', 'NNP', 'NNP', 'NNP', 'TO', 'VB', 'RP', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NN', 'VBG', 'NNP', 'POS', 'NN', 'IN', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NN', 'IN', 'DT', 'NN', 'NN', 'TO', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'JJ', 'NNP', 'CC', 'JJ', 'NNS', 'RB', 'VBN', 'RP', 'NNS', ',', 'IN', 'PRP', 'VBD', 'PRP$', 'NNS', 'JJ', 'TO', 'VB', 'VBG', 'DT', 'JJ', 'NNS', 'RBR', 'JJ', '.', '--s--', 'NNP', 'POS', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NN', 'NN', 'CC', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', '.', '--s--', 'DT', 'NNP', 'NNP', 'NNS', 'VBD', 'CD', ',', 'TO', 'CD', '.', '--s--', 'DT', 'NN', 'NN', 'NN', 'VBD', 'TO', 'DT', 'NN', ',', 'IN', 'NNP', 'NNS', 'VBD', 'CC', 'DT', 'NN', 'VBD', '.', '--s--', 'JJ', 'NNS', 'VBD', 'JJ', 'NNP', ',', 'CC', 'IN', 'JJ', 'NN', 'VBD', 'VBG', 'RP', '.', '--s--', 'DT', 'NN', 'VBD', 'VBG', 'RB', 'JJR', 'IN', 'NNP', '.', '--s--', 'NNP', 'IN', 'DT', 'JJ', 'NNP', 'NN', 'NN', 'VBP', 'JJ', '.', '--s--', 'JJ', 'NNS', 'VBD', 'TO', 'VB', 'DT', '$', 'CD', 'CD', 'NN', ',', 'CC', 'NNS', 'VBD', 'PRP', 'VBD', 'RB', 'IN', 'DT', 'NN', 'TO', 'VB', 'NNS', '.', '--s--', 'DT', 'NN', 'VBD', 'VBN', 'RB', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'JJ', 'NN', ',', 'PRP', 'VBD', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'VBD', 'CD', 'NNS', 'IN', 'NNS', ',', 'CC', 'NNS', 'VBD', 'VBN', 'IN', 'IN', 'DT', 'CD', 'NN', 'NN', 'VBD', 'DT', 'JJ', 'VBG', 'IN', 'NN', '.', '--s--', 'RB', ',', 'JJ', 'NNS', 'VBD', 'CD', 'NN', 'JJ', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'NNP', 'POS', 'NN', 'NN', 'NN', ',', 'CC', 'NNS', 'RB', 'VBP', 'TO', 'VB', 'DT', 'NN', 'DT', 'NN', '.', '--s--', 'NNP', 'MD', 'RB', 'VB', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'WRB', 'JJ', 'NNS', 'VBP', 'RB', 'DT', 'NN', '.', '--s--', 'NNP', 'NNPS', 'VBD', 'IN', 'RB', 'TO', 'CD', 'JJ', 'JJ', 'NN', 'NNS', 'MD', 'VB', 'IN', 'DT', 'NNS', 'IN', 'PRP', 'VBZ', 'TO', 'VB', 'JJ', 'NN', '.', '--s--', 'NNP', 'NN', 'CC', 'NN', 'NNS', 'VBD', 'CD', 'NN', 'IN', 'JJ', 'NNP', ',', 'DT', 'JJ', 'NNS', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'VBN', 'RP', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNS', '.', '--s--', 'NNP', 'CC', 'NNP', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'RP', 'NNP', 'POS', 'NNS', 'TO', 'VB', 'DT', 'NNP', 'NN', 'NN', '.', '--s--', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'JJS', 'IN', 'PRP$', 'NN', 'CC', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'VB', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', '.', '--s--', 'DT', 'NNP', 'NN', 'VBZ', 'VBG', 'TO', 'VB', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'NNP', 'PRP', 'MD', 'VB', 'PRP$', 'JJ', 'NN', 'RP', 'IN', 'NNP', 'IN', 'IN', 'NNS', 'IN', 'VBG', 'NN', 'IN', 'NNP', '.', '--s--', 'NNP', 'CC', 'NNP', 'MD', 'VB', 'DT', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBD', ',', 'DT', 'NN', 'VBD', '.', '--s--', 'NNP', 'NNP', 'CC', 'NNP', 'POS', 'NNP', 'VBP', 'VBG', 'DT', 'NN', 'TO', 'VB', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'JJ', 'NNS', 'MD', 'VB', 'DT', 'JJR', 'NN', 'TO', 'VBG', 'NNS', 'WDT', 'VBP', 'RB', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'POS', 'NN', 'NN', 'MD', 'VB', 'VBN', 'RB', '.', '--s--', 'CD', 'JJ', 'NNS', 'VBD', 'VBN', 'RP', 'IN', 'NN', 'NNS', ',', 'CC', 'JJ', 'NNS', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', '.', '--s--', 'NNP', ':', '--s--', 'NNS', ':', 'NN', 'CD', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'CD', ',', 'RB', 'CD', ':', 'NN', 'CD', ',', 'RB', 'CD', ':', 'NNS', 'CD', ',', 'RB', 'CD', '.', '--s--', 'NNS', ':', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', 'CD', ',', 'RB', '--s--', 'NNS', ':', 'NNP', 'NNP', 'NNS', 'NN', 'CD', ',', 'RB', 'CD', ':', 'NN', 'NN', 'CD', ',', 'RB', 'CD', '.', '--s--', 'NNP', ':', 'CD', 'NNS', ',', 'RB', 'CD', ':', 'CD', 'NNS', ',', 'RB', 'CD', '.', '--s--', 'DT', 'JJ', 'NNS', 'NN', 'IN', 'NNP', 'NNP', 'VBD', 'IN', 'NNS', 'MD', 'RB', 'VB', 'JJ', 'NNS', 'JJ', 'IN', 'JJ', 'NNS', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'NNS', 'VBD', 'VBN', 'JJ', 'NN', 'IN', 'DT', 'NN', 'POS', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'IN', 'DT', 'CD', 'NN', 'VBN', 'IN', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', ',', 'VBG', 'DT', 'NN', 'NNS', 'VBD', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NN', ',', 'VBN', 'IN', 'CD', '.', '--s--', 'NNS', 'VBG', 'IN', 'NN', 'NNS', 'VBD', 'PRP', 'VBP', 'VBN', 'IN', 'NN', 'IN', 'DT', '``', 'NN', 'NN', "''", 'IN', 'NNS', 'MD', 'VB', 'JJ', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'VBG', 'TO', 'DT', 'NN', ',', 'NNP', 'NNS', 'VBD', 'JJ', 'NN', 'IN', 'VBG', 'NNP', 'IN', 'DT', 'NN', 'NN', 'WDT', 'MD', 'VB', 'DT', 'NN', 'CC', 'VB', 'RB', 'JJ', 'IN', 'PRP$', 'JJ', 'NN', '.', '--s--', 'DT', 'NNS', 'RB', 'VBN', 'IN', 'JJ', 'TO', 'DT', 'NN', ',', 'DT', 'NN', 'VBD', 'JJ', 'NNS', 'VBG', 'NNS', 'IN', 'NNP', '.', '--s--', 'DT', 'NN', 'VBD', 'NNP', 'POS', 'NN', 'VBD', 'TO', 'DT', 'JJ', 'IN', '$', 'CD', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'POS', 'JJ', 'NNS', '.', '--s--', 'CC', 'WRB', 'DT', 'NN', 'VBD', 'NNP', 'POS', 'JJ', 'NNS', 'RB', 'IN', 'CD', ',', 'DT', 'NN', 'VBD', 'TO', 'DT', 'JJ', 'IN', '$', 'CD', 'DT', 'NN', ',', 'VBG', 'TO', 'DT', 'NN', '.', '--s--', 'DT', 'NNS', 'VBD', 'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'NNS', '.', '--s--', 'IN', 'CD', ',', 'DT', 'NNP', 'NNP', 'NN', 'NN', 'VBD', 'DT', 'NN', 'RB', 'IN', 'NN', 'NNS', 'VBD', 'RB', 'VB', 'DT', 'NN', 'POS', 'NNS', 'IN', 'JJ', 'NNS', 'TO', 'NNP', 'POS', 'NNS', '.', '--s--', 'RB', 'JJ', 'NN', ',', 'DT', 'NNS', 'NN', 'VBD', 'IN', 'JJS', 'IN', 'DT', 'NN', 'MD', 'VB', 'VBN', '.', '--s--', 'RB', ',', 'PRP', 'VBD', 'DT', 'NNS', 'DT', 'NN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'NN', 'IN', 'VBG', 'TO', 'NNP', 'POS', 'NN', 'NN', ',', 'VBN', 'IN', 'NNP', '.', '--s--', 'DT', 'NN', 'VBD', 'IN', 'DT', 'NN', 'NNS', 'VBD', 'RB', 'VB', 'NNP', 'POS', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'NN', 'POS', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'CC', 'VBD', 'RP', 'IN', 'DT', 'NN', 'VBD', 'IN', 'DT', 'NN', 'MD', 'VB', 'VBN', 'IN', 'RB', 'JJ', 'TO', 'DT', 'NNP', 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBD', 'JJ', 'TO', 'VB', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'IN', 'DT', 'RB', 'JJ', 'NNS', 'CC', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'POS', 'NNS', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'CC', 'NNP', ',', 'WDT', 'VBD', 'RB', 'VBN', 'IN', 'DT', 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'VBG', 'IN', 'DT', 'JJ', 'NNS', 'VBP', 'TO', 'VB', 'VBN', 'TO', 'DT', 'NN', 'IN', 'JJ', 'NN', 'CC', 'NN', 'IN', 'DT', 'NNS', 'IN', 'JJ', 'NNS', '.', "''", '--s--', 'NNP', 'POS', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'PRP', 'VBD', ':', '``', 'DT', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NN', ':', 'IN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'POS', 'NN', 'VBP', 'RB', 'VBD', ',', 'DT', 'NN', 'MD', 'VB', 'NN', 'IN', 'NN', 'IN', 'NNS', 'NN', '.', "''", '--s--', 'NNP', 'NNP', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'JJ', 'NNS', 'TO', 'JJ', '--s--', 'DT', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBG', 'VBG', 'DT', 'NN', 'VBG', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'VBZ', 'RB', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NNP', '.', '--s--', 'DT', 'NN', 'IN', 'CD', 'VBN', 'IN', 'DT', 'NNP', 'POS', 'NNS', 'IN', 'DT', 'NNP', 'VBZ', 'JJ', 'NN', 'NN', '.', '--s--', 'CC', 'DT', 'NN', 'VBD', 'DT', 'NNP', 'RB', '$', 'CD', 'IN', 'NNS', ',', 'VBG', 'IN', 'IN', 'DT', 'JJ', 'NNS', '.', '--s--', 'JJ', 'NN', ',', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'IN', 'NNP', 'NNP', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'NNS', 'NNS', 'TO', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'IN', 'NNP', 'CC', 'NNP', ',', 'RB', 'IN', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', ',', 'VBD', 'DT', 'JJ', 'NN', 'NN', ',', 'CC', 'PRP$', 'JJ', 'NN', 'VBD', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'CC', 'NNP', 'VBZ', 'IN', '$', 'CD', 'CD', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'TO', 'DT', 'NN', 'TO', 'VB', 'PRP', 'IN', 'NNS', 'RB', 'VBN', '.', '--s--', 'NNP', 'CC', 'NNP', 'MD', 'VB', 'IN', '$', 'CD', 'IN', 'DT', 'NNS', 'IN', 'VBG', 'DT', 'NN', '.', '--s--', 'DT', 'JJ', 'NNS', 'NN', 'VBN', 'IN', 'DT', 'JJ', 'NNS', 'CC', 'DT', 'NN', 'TO', 'VB', 'DT', 'NNS', 'VBD', 'RB', 'VB', 'DT', 'NN', 'IN', 'VBG', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBP', 'IN', 'JJ', 'NNS', 'MD', 'VB', 'VBN', 'IN', 'NNS', 'POS', 'NNS', '.', '--s--', 'CC', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NNP', ',', 'VBZ', 'PRP$', 'NN', 'MD', 'VB', 'VBG', 'IN', 'DT', 'NN', 'CC', 'VBG', 'TO', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', ',', 'IN', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', 'IN', 'NNP', 'NNP', ',', 'VBZ', 'DT', 'NN', 'VBZ', 'JJ', 'CC', 'DT', 'NN', 'NN', 'VBZ', 'JJ', 'IN', 'DT', 'NNP', 'VBD', 'PRP$', 'JJ', 'NNS', ',', 'VBG', 'PRP$', 'NN', 'IN', 'DT', 'NNP', 'JJ', 'NN', 'IN', 'NN', 'NNS', '.', '--s--', '``', 'DT', 'NNP', 'VBD', 'RB', 'DT', 'JJ', 'NN', ',', "''", 'NNP', 'NNP', 'VBZ', '.', '--s--', 'NNP', 'NNP', ':', '--s--', 'NNP', 'CC', 'NNP', 'IN', 'NNP', 'CC', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', 'IN', 'NNP', ',', 'NNP', ',', 'VBP', 'VBN', 'TO', 'VB', 'VB', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'JJ', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'NNS', 'VBP', 'RB', 'VBN', 'TO', 'VB', 'NN', 'RB', 'TO', 'DT', 'JJ', 'CC', 'VB', 'JJ', 'NNS', '.', '--s--', 'CC', 'PRP', 'MD', 'VB', 'RP', 'IN', 'NN', ':', ',', 'JJ', ':', 'CC', 'JJ', 'NNS', 'CC', 'NN', 'NNS', 'IN', 'NNS', 'IN', 'JJ', 'NN', ',', 'VBD', 'NNP', 'NNP', 'NNP', 'IN', 'CD', 'NNS', 'CC', 'NNP', '.', '--s--', 'IN', 'NN', ',', 'NNP', 'CC', 'NNP', 'POS', 'NNP', ',', 'NNP', ',', 'NN', 'MD', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNS', 'VBP', 'RB', 'IN', 'JJ', 'NNS', ',', 'RB', 'JJ', 'NNS', ',', 'IN', 'PRP$', 'NNS', '.', '--s--', 'DT', 'NN', ',', 'VBN', 'IN', 'IN', 'CD', 'NNS', 'IN', 'NNS', ',', 'VBD', 'RP', 'IN', 'CD', 'NNP', 'NNP', 'POS', 'NN', 'TO', 'VB', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NN', 'IN', 'VBN', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'CC', 'NNP', ',', 'VBD', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', '.', '--s--', 'IN', 'NNP', 'NNP', 'NNP', 'DT', 'NN', 'IN', 'NN', 'NNS', ',', 'NNP', 'NNP', 'CC', 'NNP', 'VBD', 'DT', 'CD', 'CD', 'NN', 'IN', 'NNP', 'IN', 'IN', 'PRP', 'MD', 'VB', 'VBN', 'TO', 'VB', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'IN', 'NNP', ',', 'VBD', '$', 'CD', 'CD', 'IN', 'JJ', 'NNS', 'IN', 'NNP', ',', 'VBG', 'IN', 'DT', 'NN', ',', 'WDT', 'MD', 'VB', 'JJ', 'NNS', 'CC', 'NN', 'NN', ',', 'VBD', 'DT', 'NN', 'TO', 'NNS', '.', '--s--', 'NNP', 'RB', 'VBN', 'IN', 'VBG', 'DT', 'NN', 'NNS', 'MD', 'VB', 'DT', 'JJR', 'NN', 'NN', 'IN', 'VBG', 'PRP', 'RB', '.', '--s--', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'DT', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'IN', 'NN', 'IN', 'DT', 'NN', 'NN', 'WRB', 'DT', 'NN', 'VBD', 'DT', 'NN', 'CC', 'DT', 'NN', 'IN', 'NN', 'VBD', 'VBN', 'IN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', ':', '--s--', 'NNP', 'NNP', 'NNP', 'MD', 'VB', 'CD', 'NNP', 'CC', 'NNP', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NNP', ',', 'NNP', ',', 'NN', 'IN', 'NNP', '.', '--s--', 'PRP', 'MD', 'VB', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'JJ', 'NN', 'POS', 'NNS', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'VBD', 'IN', 'NNP', 'NNPS', 'NNP', 'JJ', 'NN', 'RB', 'DT', 'NN', 'IN', 'RB', 'CD', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'TO', 'VB', 'VBN', 'NNP', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'RB', 'NNP', 'NNPS', 'POS', 'JJ', 'NN', ',', 'VBD', 'DT', 'NNP', 'NNP', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNP', ',', 'NNP', 'NNP', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'CD', ',', 'VBD', 'CD', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNP', 'CC', 'VBD', 'VBN', 'JJ', 'NN', 'IN', 'CD', '.', '--s--', 'PRP', 'VBD', 'DT', 'NN', 'IN', 'CD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'MD', 'VB', 'PRP$', 'NN', 'IN', 'DT', 'JJ', 'NN', 'POS', 'NNS', 'IN', 'NNP', ',', 'NNP', ',', 'CC', 'NNP', 'NNP', ',', 'IN', 'NNS', 'IN', 'NN', 'CC', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'MD', 'VB', 'NN', 'NNS', 'IN', 'JJ', 'IN', 'PRP$', 'NNS', 'IN', 'NNP', 'CD', 'TO', 'NNP', 'CD', ',', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'NN', '.', '--s--', 'NNP', 'MD', 'VB', 'DT', '$', 'CD', 'NN', 'IN', 'PRP$', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'NNP', 'CC', 'NN', 'NN', ':', '$', 'CD', 'IN', 'DT', 'JJ', 'NNP', 'CC', 'JJ', 'NN', 'NN', ':', '$', 'CD', 'IN', 'DT', 'NNP', 'NNP', 'JJ', 'NN', 'NN', ':', '$', 'CD', 'IN', 'DT', 'NNP', 'CD', 'NNS', ',', 'CC', '$', 'CD', 'IN', 'DT', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'NN', 'NN', 'CC', 'NNP', 'NN', '.', '--s--', 'DT', 'NNS', ',', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'NN', 'TO', 'CD', 'NN', 'TO', 'CD', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'NNS', 'TO', 'NNS', 'WP', 'VBP', 'IN', 'PRP$', 'NN', 'NNS', 'CC', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', 'VBD', 'PRP', 'VBP', 'VBN', 'DT', 'CD', 'NNS', '(', '$', 'CD', 'CD', ')', 'JJ', 'NN', 'TO', 'VB', 'NN', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'POS', 'JJ', 'JJ', 'NN', ',', 'NNP', 'MD', 'VB', 'CD', 'NN', 'IN', 'DT', 'NN', 'CC', 'DT', 'NNP', 'NN', ',', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'CD', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'DT', 'CD', 'NNS', 'VBD', 'VBN', 'IN', 'NNP', '.', '--s--', 'DT', 'JJ', 'NN', 'VBZ', 'TO', 'VB', 'VBN', 'NNP', 'NNP', 'NNP', 'NNP', '.', 'NNP', '.', '--s--', 'PRP', 'MD', 'VB', 'PRP$', 'NN', 'IN', 'NNP', '.', '--s--', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NN', 'MD', 'VB', 'TO', 'CD', 'NN', 'NNP', 'CD', ',', 'CD', ',', 'CC', 'TO', 'CD', 'NN', 'DT', 'JJ', 'NNP', 'CD', '.', '--s--', 'NNP', 'MD', 'VB', 'NN', 'NN', 'NNS', '.', '--s--', 'DT', 'NN', 'NNS', 'VBP', 'NNS', 'IN', 'DT', 'NN', 'IN', 'IN', 'CD', 'CD', 'NNS', 'IN', 'PRP$', 'JJ', 'JJ', 'NN', ',', 'NNP', 'VBD', '.', '--s--', 'NNS', 'VBP', 'VBN', 'TO', 'VB', 'TO', 'CD', 'CD', 'NNS', 'IN', 'CD', 'NNS', '.', '--s--', 'DT', 'NN', 'MD', 'VB', 'NN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', '.', '--s--', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'VBD', 'NNP', 'IN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'IN', 'IN', 'NNS', 'IN', 'NNS', 'IN', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'DT', 'NN', 'POS', 'JJ', 'NNS', 'IN', 'NNP', '.', '--s--', '``', 'RB', ',', 'PRP', 'VBD', 'PRP$', 'NN', ',', "''", 'VBD', 'NNP', 'NNP', '.', '--s--', 'IN', 'NNP', 'NNP', 'POS', 'NN', ',', 'DT', 'NN', 'POS', 'NN', 'NNS', 'CC', 'NN', 'VBP', 'VBN', 'RB', '.', '--s--', 'CC', 'IN', 'CD', 'CD', 'JJ', 'VBN', 'NN', ',', 'NNP', 'NNP', 'RB', 'VBZ', 'JJ', 'IN', 'NNP', 'NNP', 'NNP', 'POS', 'NN', 'NN', ',', 'IN', 'CD', 'CD', 'NN', ',', 'CC', 'NNP', 'NNP', 'NNP', 'POS', 'NNP', ',', 'IN', 'CD', 'CD', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'NN', 'CC', 'NN', 'IN', 'NN', ',', 'VBD', 'NNP', 'NNP', 'MD', 'VB', 'VBN', 'VBG', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NN', 'POS', 'JJ', 'NN', ',', 'CC', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'VBP', 'VBN', 'TO', 'DT', 'JJ', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP$', 'NN', 'MD', 'VB', 'PRP$', 'JJ', 'NN', ',', 'WDT', 'VBZ', 'DT', 'NN', 'IN', 'NN', 'CC', 'NN', 'NNS', 'IN', 'JJ', ',', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NN', 'IN', 'VBG', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', ',', 'VBD', 'PRP', 'VBD', 'JJ', 'NN', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'PRP', 'VBD', 'NN', '.', '--s--', 'DT', 'NNS', 'VBD', 'TO', 'VB', 'IN', 'DT', 'NN', ',', 'RB', 'DT', 'NN', '.', '--s--', 'PRP', 'VBD', 'PRP', 'MD', 'RB', 'VB', 'DT', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'NN', 'NN', 'IN', 'NNS', 'CC', 'NNS', 'TO', 'NN', 'NN', 'VBD', 'CD', 'NN', 'TO', 'CD', 'CD', 'NNS', '(', '$', 'CD', 'CD', ')', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', 'IN', 'CD', 'CD', 'NNS', 'DT', 'NN', 'RBR', '.', '--s--', 'DT', 'JJ', 'NN', 'NN', 'VBD', 'RP', ',', 'RB', ',', 'DT', 'JJ', 'NNS', 'IN', 'PRP$', 'NN', 'IN', 'NN', 'VBP', 'RB', 'JJ', 'IN', 'IN', 'JJ', 'NN', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'PRP', 'VBD', ',', 'IN', 'NN', ',', 'DT', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'CD', 'CD', 'NNS', ',', 'RB', 'IN', 'CD', 'NN', 'IN', 'CD', 'CD', 'NNS', 'DT', 'NN', 'RBR', '.', '--s--', 'RB', ',', 'PRP', 'VBD', 'PRP', 'VBZ', 'NNS', 'IN', 'DT', 'IN', 'CD', 'TO', 'VB', 'IN', 'DT', 'NN', 'IN', 'CD', 'CD', 'NNS', ',', 'VBG', 'JJ', 'NNS', 'IN', 'CD', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'IN', 'CD', ',', 'NNP', 'VBD', 'NN', 'NN', 'IN', 'CD', 'CD', 'NNS', 'IN', 'NN', 'IN', 'CD', 'CD', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'RB', 'RB', 'VBN', 'NNS', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', ',', 'CC', 'VBZ', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNPS', 'NNP', 'VBZ', 'TO', 'VB', 'NNS', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'IN', 'CD', 'NNS', 'DT', 'NN', ',', 'IN', 'DT', 'JJ', 'NN', ',', 'VBN', 'IN', 'DT', 'NN', 'JJ', 'NN', ',', 'VBD', 'NNP', 'NNP', 'NNP', ',', 'NN', 'CC', 'JJ', 'NN', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'NN', 'IN', 'IN', '$', 'CD', ',', 'NNP', 'VBZ', 'TO', 'VB', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'IN', 'CD', 'NNS', 'DT', 'NN', ',', 'NNP', 'NNP', 'VBD', '.', '--s--', 'IN', 'JJ', 'NN', 'JJ', 'NN', ',', 'NNP', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'VBG', 'NNS', 'CC', 'DT', '$', 'CD', 'NN', 'IN', 'JJ', 'NNS', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NN', 'IN', 'DT', 'JJS', 'JJ', 'NN', 'VBD', 'IN', '$', 'CD', 'CD', ',', 'RB', 'CD', 'NN', 'IN', '$', 'CD', 'CD', 'JJ', 'NN', ',', 'PRP', 'VBD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'CC', 'NN', 'NNS', ',', 'WDT', 'VBZ', 'VBN', 'TO', 'VB', 'JJ', 'IN', 'DT', 'NN', 'IN', 'JJ', ',', 'VBN', 'NN', '.', '--s--', 'DT', 'JJ', '$', 'CD', 'CD', 'NN', 'NN', 'NN', ',', 'WDT', 'VBD', 'RP', 'TO', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'CD', ',', 'RB', 'VBZ', 'VBG', 'RB', 'CC', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'RBR', 'RB', 'VBG', 'NNP', 'POS', 'NNS', ',', 'NNP', 'NNP', 'VBD', '.', '--s--', 'IN', 'DT', 'NN', ',', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'IN', '$', 'CD', 'TO', '$', 'CD', 'DT', 'NN', ',', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'JJ', 'NN', ',', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NN', 'IN', 'CD', 'VBZ', 'VBN', 'TO', 'VB', 'IN', '$', 'CD', 'CD', ',', 'RB', 'IN', 'CD', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'CD', '.', '--s--', 'IN', 'DT', 'CD', 'NNS', 'VBD', 'NNP', 'CD', ',', 'NNP', 'VBZ', 'TO', 'VB', 'JJ', 'NN', 'IN', '$', 'CD', 'NNS', ',', 'CC', 'IN', '$', 'CD', 'DT', 'NN', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', '.', '--s--', 'JJ', 'NN', ',', 'DT', 'NN', 'VBD', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', ',', 'VBG', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'VBG', 'NNS', 'CC', 'DT', '$', 'CD', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NN', 'IN', 'DT', 'CD', 'NNS', 'VBZ', 'VBN', 'TO', 'VB', 'IN', '$', 'CD', 'CD', ',', 'RB', 'IN', 'CD', 'NN', 'IN', '$', 'CD', 'CD', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'NNP', 'POS', 'JJ', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'IN', 'CD', 'NNS', 'IN', 'NN', 'VBZ', 'DT', 'NN', ',', 'RB', 'IN', 'CD', 'NNS', 'IN', 'CD', '.', '--s--', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'VBN', 'DT', 'NN', 'IN', 'NNP', 'POS', 'NN', 'NN', 'TO', 'VB', 'PRP', ',', 'PRP', 'VBD', ',', 'WDT', 'MD', 'VB', 'VBN', 'TO', 'VB', 'NNS', ',', 'CC', 'JJ', 'NN', 'NNS', ',', 'IN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'CC', 'JJ', 'NN', '.', '--s--', 'NNP', 'MD', 'VB', 'TO', 'VB', 'VBG', 'PRP$', 'NN', 'NN', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', 'TO', '$', 'CD', 'CD', ',', 'IN', 'DT', 'JJ', 'CD', 'TO', 'CD', 'NNS', ',', 'NNP', 'NNP', 'VBD', '.', '--s--', 'IN', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBD', 'VBN', 'NN', 'CC', 'JJ', 'NN', 'NN', 'JJ', 'NN', ',', 'NNP', 'VBZ', 'VBN', 'PRP$', 'NNS', 'VBP', 'TO', 'RB', 'CD', 'NNS', 'IN', 'IN', 'CD', 'CC', 'VBZ', 'TO', 'VB', 'PRP$', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'CC', 'JJ', 'NNS', '.', '--s--', '``', 'RB', 'TO', 'DT', 'NN', 'RB', ',', 'NNP', 'VBD', 'DT', 'NN', '.', '--s--', 'RB', 'PRP', 'VBP', 'PRP', 'VBP', 'DT', 'NN', 'IN', 'NN', 'NNS', 'WP', 'VBP', 'RP', 'VBG', 'TO', 'NNS', 'CC', 'NNS', ',', "''", 'NNP', 'NNP', 'VBD', '.', '--s--', 'RB', ',', 'PRP', 'VBD', ',', 'DT', 'JJ', 'NN', 'VBZ', 'TO', 'VB', 'PRP$', 'JJ', 'NN', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'CC', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NN', 'NN', 'NN', ',', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'NN', 'NNS', 'CC', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'JJ', 'NNS', 'IN', 'NNS', 'IN', 'PRP$', 'NNP', 'NNP', 'JJ', 'NN', 'IN', 'NNP', ',', 'NNP', ',', 'MD', 'VB', 'IN', 'JJ', 'NNS', 'NN', 'TO', 'PDT', 'DT', 'NN', 'CC', 'PRP$', 'NNP', 'NNP', 'NNPS', 'NNP', 'NNP', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'RB', 'VB', '.', '--s--', 'DT', 'JJ', 'NN', 'CC', 'NNS', 'NN', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'JJ', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NN', ',', 'CC', 'IN', 'DT', 'NN', 'MD', 'VB', 'VBN', 'RB', 'WRB', 'DT', 'NN', 'VBZ', '.', '--s--', 'RBR', 'IN', 'CD', 'NNS', 'IN', 'VBG', 'DT', 'NN', 'VBP', 'VBN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'CD', 'NNS', '.', '--s--', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'WDT', 'VBZ', 'JJR', 'IN', 'CD', 'NNS', 'IN', 'NNS', 'RB', ',', 'MD', 'VB', 'TO', 'VB', '.', '--s--', 'NNP', 'VBD', 'DT', 'NN', 'MD', 'RB', 'VB', 'NNS', 'NNS', 'IN', 'JJ', 'NNS', 'NNS', 'IN', 'PRP$', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', ',', 'CC', 'MD', 'VB', 'DT', 'NN', 'IN', 'NN', 'VBN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNPS', ',', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'VBZ', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNS', 'MD', 'VB', 'DT', 'NN', 'POS', 'NN', 'VBG', 'RP', 'TO', 'PRP', '.', '--s--', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'VBG', 'IN', 'RB', 'CD', 'NNS', 'IN', 'DT', 'NNP', 'CD', '``', 'NNP', 'NNP', "''", 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', ',', 'VBZ', 'DT', 'NN', 'VBZ', 'VBN', 'IN', 'CD', 'IN', 'DT', 'NNS', '.', '--s--', 'NNS', 'WP', 'VBP', 'RP', 'DT', 'NN', 'IN', 'NN', 'CC', 'VB', 'DT', 'NN', 'MD', 'VB', 'DT', 'JJ', ':', 'CC', 'JJ', 'NN', ':', 'NN', 'VB', ',', '``', 'PRP', 'VBP', 'DT', 'VBG', 'NN', ',', "''", 'IN', 'PRP', 'VBZ', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'DT', 'VBG', 'NN', 'VBZ', 'RB', 'JJ', ':', 'DT', 'JJ', 'NN', 'TO', 'NNP', 'NNP', 'VBZ', 'IN', '$', 'CD', ',', 'CC', 'WDT', 'VBZ', 'IN', 'VBG', 'IN', 'NNP', 'NNP', 'POS', 'NN', ':', 'CC', 'NNP', 'NNP', 'VBZ', 'PRP', 'VBZ', 'DT', 'JJ', '.', '--s--', 'JJ', 'NNS', 'VBP', 'VBN', 'JJ', 'NNS', ',', 'VBG', 'JJ', 'JJ', 'NNS', 'WDT', 'VBP', 'DT', 'NN', ',', 'CC', 'DT', 'NN', 'NNS', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'RBR', 'JJ', ',', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'PRP', 'VBZ', ',', 'IN', 'NN', ',', 'WRB', 'VBG', 'NN', 'NN', 'NNS', '.', '--s--', 'CC', 'IN', 'DT', 'WP', 'VBP', 'DT', 'NN', 'DT', 'JJ', 'NN', 'RB', ',', 'RB', 'TO', 'VB', ':', 'CD', 'JJ', 'NNS', 'VBP', 'JJ', 'NN', 'IN', 'IN', 'JJ', 'IN', 'CD', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'NNP', ',', 'VBD', 'PRP', 'MD', 'VB', 'DT', 'NNP', 'NN', 'NN', 'TO', 'VB', 'PRP', 'TO', 'VB', 'NNP', 'NNP', 'CC', 'NNP', 'TO', 'VB', 'PRP', 'VB', 'PRP$', 'NN', 'NN', '.', '--s--', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', 'NNP', 'NNPS', ',', 'VBZ', 'VBN', 'VBG', 'TO', 'VB', 'PRP$', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'JJ', 'NNS', ',', 'RB', 'IN', 'NN', '.', '--s--', 'DT', 'JJ', 'NN', 'CC', 'NN', 'NN', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'NNS', 'IN', 'DT', 'NN', 'TO', 'VB', 'PRP$', 'NN', '.', '--s--', 'IN', 'NN', 'IN', 'NN', 'NNS', 'CC', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'JJ', 'NN', ',', 'NNP', 'VBD', 'IN', '$', 'CD', 'CD', 'IN', 'NNS', 'WDT', 'VBD', 'JJ', 'NNP', 'CD', '.', '--s--', 'PRP', 'VBD', 'IN', 'NN', 'IN', 'NNS', 'IN', 'NNP', 'CD', 'IN', 'DT', 'JJ', 'NNP', 'NNP', 'NNP', 'CD', 'TO', 'VB', 'PRP', 'JJ', 'NN', 'TO', 'VB', 'IN', 'DT', 'NN', 'TO', 'VB', 'PRP$', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBD', 'NNP', 'NNP', ',', 'WDT', 'VBZ', 'NNS', 'CC', 'NN', 'IN', 'NNP', ',', 'MD', 'RB', 'VB', 'IN', 'NNP', 'POS', 'NN', 'NN', 'IN', 'PRP', 'VBZ', 'RB', 'DT', 'NN', '.', '--s--', 'PRP', 'VBD', 'PRP', 'VBD', 'NNP', 'IN', 'NN', 'IN', 'IN', 'NNP', 'POS', 'NNS', 'IN', 'NNP', 'CC', 'NNP', ',', 'WRB', 'NNS', 'MD', 'VB', 'VBN', 'IN', 'DT', 'NNP', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'VBD', 'PRP', 'MD', 'VB', 'PRP$', 'JJ', 'NNS', 'TO', 'VB', 'PDT', 'PRP$', 'NNS', 'TO', 'JJR', 'JJ', 'NNS', ',', 'IN', 'NNP', ',', 'VBG', 'IN', 'DT', 'JJ', 'NN', 'TO', 'CD', 'NNS', 'IN', 'CD', 'CD', 'JJ', 'NNS', '(', '$', 'CD', 'CD', ')', '.', '--s--', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'CC', 'JJ', 'NN', ',', 'VBD', 'DT', 'NN', 'POS', 'JJ', 'NNS', 'VBP', 'PRP', 'TO', 'VB', 'DT', 'JJ', 'JJ', 'JJ', 'NN', 'TO', 'VB', 'RP', 'NNS', 'VBG', 'PDT', 'PRP$', '$', 'CD', 'CD', 'IN', 'NNP', 'NNP', 'NN', '.', '--s--', '``', 'PRP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNS', ',', "''", 'PRP', 'VBD', '.', '--s--', 'DT', 'NN', 'VBD', 'IN', '$', 'CD', 'CD', 'MD', 'VB', 'VBN', 'TO', 'PRP$', 'JJ', 'NNP', 'CC', 'JJ', 'NN', 'NNS', 'IN', 'PRP$', 'JJ', 'NN', ',', 'VBG', 'NNP', 'CD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'TO', 'NNS', 'MD', 'VB', 'TO', 'RB', '$', 'CD', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'POS', 'JJ', 'NN', 'IN', 'DT', 'CD', 'NNS', 'VBD', 'NNP', 'CD', 'VBD', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'MD', 'VB', 'PRP$', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', ',', 'IN', 'WDT', '$', 'CD', 'CD', 'VBZ', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'NNP', 'NN', 'NNS', '.', '--s--', 'DT', 'NN', 'VBZ', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'VBG', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'CC', 'VBZ', 'TO', 'VB', 'IN', 'IN', 'JJ', 'JJ', 'NNS', 'VBN', 'IN', 'NNP', 'NNP', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'POS', 'NN', 'IN', 'NNP', 'NN', 'VBZ', 'VBN', '``', 'JJ', "''", 'CC', 'DT', 'JJS', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'VBN', 'CD', 'NNS', 'RB', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'NNS', ',', 'NN', 'NNS', 'CC', 'NN', 'NNS', '.', '--s--', 'PRP', 'VBD', 'DT', 'NN', 'MD', 'RB', 'RB', 'VB', 'IN', 'NNP', 'NN', 'NN', ',', 'CC', 'MD', 'VB', 'NN', 'NN', 'CC', 'JJ', 'NNS', 'WDT', 'VBP', 'DT', 'NN', 'POS', 'NNS', '.', '--s--', 'DT', 'NN', 'IN', 'NN', 'NNS', 'NNP', 'VBZ', 'RB', 'VB', 'DT', 'NN', 'IN', 'JJ', 'CC', 'RB', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'RBR', 'JJ', 'NN', ',', 'NNP', 'NNP', 'POS', 'JJ', 'NN', 'NNS', 'VBP', '.', '--s--', 'NNP', 'IN', 'RB', 'RB', 'VBP', 'RBR', 'JJ', 'TO', 'VB', 'JJ', 'NNS', ',', 'RB', 'IN', 'NNS', 'CC', 'JJ', 'NNS', '.', '--s--', 'CC', 'PRP', 'MD', 'VB', 'VBG', 'JJ', 'CC', 'RB', 'JJR', 'NNS', ',', 'RB', 'IN', 'PRP', 'VBZ', 'JJR', 'TO', 'VB', 'PRP', '.', '--s--', 'DT', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'RB', 'VBZ', 'VBN', 'VBN', 'IN', 'NN', 'NN', 'CC', 'NN', 'NNS', 'IN', 'NNS', 'WDT', 'VBD', 'PRP', 'RP', 'IN', 'NN', '.', '--s--', 'NNP', 'DT', 'CD', 'NN', 'NN', 'VBZ', 'TO', 'VB', 'VBG', ',', 'NN', 'NNS', 'NN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'VBN', ',', 'RB', 'JJ', ',', "''", 'VBZ', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'NNP', 'JJ', 'NN', 'NNP', 'NNP', '.', '--s--', 'PRP', 'RB', 'VBD', 'VBG', 'PRP$', 'NN', 'NN', 'TO', 'CD', 'NN', 'IN', 'PRP$', 'NN', '.', '--s--', '``', 'PRP', 'VBP', 'IN', 'WRB', 'NNS', 'MD', 'VB', 'VBN', ',', "''", 'PRP', 'VBZ', ',', '``', 'CC', 'PRP', 'VBP', 'RB', 'VBG', 'VBN', "''", 'IN', 'JJ', 'NNS', '.', '--s--', 'NNS', 'VBP', 'VBG', 'RB', 'JJ', 'IN', 'JJ', 'NNS', ',', 'NN', 'NNS', 'VBP', '.', '--s--', '``', 'PRP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'WDT', 'VBD', 'RB', 'IN', 'DT', 'NN', 'IN', 'NNP', 'IN', 'WP', 'VBD', 'TO', 'VB', 'VBG', 'TO', 'DT', 'NN', 'NN', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'DT', 'NN', 'CC', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'NNP', '.', '--s--', 'NNP', 'VBD', 'RP', 'NNP', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NNS', ',', 'RB', 'VBN', 'TO', 'VB', 'NNS', '.', '--s--', 'PRP', 'VBD', 'DT', 'JJS', 'IN', 'DT', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', 'NN', ',', 'WRB', 'NNS', 'VBD', 'VBG', 'JJ', 'NN', 'IN', 'NNP', 'VBD', 'DT', 'NN', 'NN', '.', '--s--', 'CC', 'NNS', 'VBP', 'TO', 'VB', 'VBG', 'DT', 'RB', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'DT', 'NN', 'VBG', 'TO', 'VB', 'NNP', 'VBD', 'NNP', 'IN', 'PRP', 'MD', 'RB', 'VB', 'DT', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'PRP', 'VBZ', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'NNPS', 'IN', '$', 'CD', 'DT', 'NN', '.', '--s--', 'JJ', 'NNS', 'NN', 'MD', 'VB', 'VBG', 'TO', 'VB', 'IN', 'DT', 'NNP', 'NNS', ',', 'CC', 'JJ', 'NN', 'NNS', 'VBP', 'JJ', 'TO', 'VB', 'IN', 'NN', '.', '--s--', 'NNP', 'NNS', 'VBG', 'DT', 'NN', 'NN', 'CC', 'NNP', 'POS', 'NN', 'VBD', 'DT', 'JJ', 'NN', 'VBG', 'TO', 'VB', 'RP', 'JJ', 'NNS', 'WDT', 'MD', 'VB', 'RBR', 'JJ', 'TO', 'DT', 'NNS', '.', '--s--', 'IN', 'NNP', ',', 'DT', 'NN', 'VBN', 'IN', 'RBS', 'JJ', 'VBZ', 'NNP', 'NNPS', 'POS', 'NN', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', '$', 'CD', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'JJ', 'NN', 'NN', 'NNP', 'NNP', '.', '--s--', 'NN', 'IN', 'NNP', 'NNS', 'VBD', 'VBN', 'RB', 'IN', 'CD', 'NN', 'NNP', 'NNP', 'CC', 'VBD', 'RB', 'VB', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'RB', 'VBN', 'IN', 'CD', 'CD', '.', '--s--', 'NNP', 'NNP', 'NN', ',', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'CC', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NNS', 'NN', ',', 'VBD', 'NNP', 'NNS', 'IN', 'NNS', 'IN', 'JJ', 'IN', 'CD', '.', '--s--', 'RB', ',', 'NNP', 'NNP', 'NNPS', 'CC', 'NNP', 'NNP', 'VBD', 'CD', 'NN', 'CC', 'CD', 'NN', ',', 'RB', ',', 'IN', 'NNP', 'CC', 'MD', 'VB', 'RBR', '.', '--s--', 'IN', 'DT', 'NN', ',', 'RB', ',', 'CD', 'NNS', 'IN', 'JJ', 'NNS', 'VBD', 'DT', 'JJ', 'NNS', 'CC', 'NNP', 'NNP', 'NNS', 'RB', 'VBP', 'JJ', 'TO', 'VB', 'NNS', 'IN', 'NNS', 'TO', 'VB', 'NN', 'NNS', 'VBN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NN', 'NN', ',', 'VBD', 'NN', 'IN', 'PRP', 'VBD', 'TO', 'VB', 'NNP', 'NNP', 'NNP', 'IN', 'DT', 'NN', 'NN', 'IN', '$', 'CD', 'DT', 'NN', ',', 'VBN', 'IN', 'DT', 'JJ', '$', 'CD', 'NN', 'NNP', 'VBD', 'CD', 'NNS', 'RB', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNS', 'VBD', 'CD', 'CD', 'TO', 'VB', 'IN', 'CD', 'CD', '.', '--s--', 'IN', 'DT', 'JJR', 'NN', ',', 'DT', 'JJS', 'NN', 'IN', 'DT', 'JJ', 'NN', 'MD', 'VB', 'IN', 'PRP', 'VBD', 'VBN', 'RB', 'JJ', 'NNP', 'NN', ':', 'IN', 'DT', 'NN', 'NN', ':', 'CC', 'VBZ', 'DT', '$', 'CD', 'CD', 'JJ', '``', 'NN', "''", 'NN', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', 'NNPS', 'CC', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'RB', ',', 'TO', 'VB', 'DT', 'JJ', 'NNP', 'NNP', 'NN', 'CC', 'VB', 'JJ', 'NN', ',', 'NNP', 'VBD', 'PRP', 'VBZ', '``', 'RB', 'JJ', "''", 'WDT', 'PRP', 'MD', 'VB', 'JJ', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', 'NN', '.', '--s--', 'NNP', 'POS', 'NN', 'VBZ', 'RB', 'RB', 'DT', 'JJ', 'NN', ',', 'IN', 'DT', 'NNS', 'MD', 'VB', 'TO', 'VB', 'VBN', 'IN', 'NNP', 'POS', 'NN', 'NN', 'MD', 'VB', 'VBN', 'RP', '.', '--s--', 'NNP', 'NNP', ',', 'RB', ',', 'VBD', 'PRP', 'VBZ', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'IN', 'NNP', '.', '--s--', 'IN', 'DT', 'NN', 'NN', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', 'VBD', 'NN', 'IN', 'NNP', 'VBZ', 'VBN', 'VBN', 'IN', 'CD', 'JJ', 'NNS', 'IN', 'PRP', 'VBP', '``', 'RB', 'JJ', "''", 'PRP', 'MD', 'VB', '$', 'CD', 'CD', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'POS', 'NN', 'NN', 'IN', 'IN', 'CD', 'NN', 'IN', 'NNP', 'NNP', ',', '``', 'RB', 'JJ', 'NNS', '.', "''", '--s--', 'NNP', 'VBZ', 'VBG', '$', 'CD', 'DT', 'NN', 'IN', 'CD', 'CD', 'NNP', 'NNS', ',', 'RB', 'VBG', 'NNP', 'POS', 'NN', 'TO', 'VB', 'RP', 'PRP$', 'NN', 'NNS', ',', 'VB', 'NNS', 'DT', '$', 'CD', 'JJ', 'NN', 'CC', 'VB', 'PRP$', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'JJ', 'NN', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNS', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'NNS', 'WDT', 'VBD', 'RB', 'VB', 'RB', ',', 'VBG', 'CD', 'CD', ',', 'CC', 'CD', 'NN', ',', 'TO', 'VB', 'IN', 'CD', 'CD', '.', '--s--', 'NNS', 'CC', 'NN', 'NNS', 'VBD', 'NNP', 'NNS', 'VBD', 'RB', 'VBN', 'RB', 'IN', 'NNP', 'VBZ', 'VBN', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'JJ', 'TO', 'VB', 'VBN', 'IN', 'NN', 'IN', 'NNS', 'CC', 'NN', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'VBP', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'JJ', 'NNS', 'IN', 'NNS', '``', 'VBD', 'VBG', 'IN', 'NNS', 'VBD', 'VBG', 'TO', 'VB', 'JJ', ',', "''", 'VBZ', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', '``', 'NN', 'VBZ', 'NNS', 'VBP', 'JJR', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', ',', "''", 'NNP', 'NNP', 'VBZ', '.', '--s--', '``', 'RB', 'IN', 'EX', 'VBP', 'NNS', 'VBG', 'TO', 'VB', 'NN', 'NNS', 'IN', 'DT', 'NN', 'NN', ',', 'EX', 'VBP', 'NNS', 'VBG', 'TO', 'VB', 'NN', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', "''", '--s--', 'NNP', 'NNS', 'VBP', 'RBS', 'IN', 'DT', 'NN', 'NNS', 'TO', 'VB', 'JJ', 'NNS', '.', '--s--', '``', 'IN', 'DT', 'NN', ',', 'JJ', 'NNS', 'VBD', 'RB', 'VBN', 'IN', 'VBG', 'NNS', 'IN', 'IN', 'NN', 'IN', 'NN', 'NNS', ',', 'WDT', 'VBD', 'RB', 'VBN', 'TO', 'VB', "''", 'DT', 'NNS', ',', 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNS', 'CC', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', '.', '--s--', 'RB', ',', '``', 'JJ', 'NNS', 'MD', 'VB', 'JJ', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'VBZ', 'VBN', 'VBN', '.', "''", '--s--', 'JJ', 'NN', 'NNS', 'VBD', 'NNP', ',', 'IN', 'NNS', 'VBD', 'PRP$', 'NN', 'IN', 'JJ', 'NNS', 'CC', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'NNS', ',', 'DT', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'VBZ', 'VBN', 'TO', 'DT', 'JJ', '$', 'CD', 'NN', 'IN', 'NNP', 'IN', 'NNP', ',', 'VBD', 'RB', 'CD', 'TO', 'VB', 'IN', 'CD', 'CD', '.', '--s--', 'CC', 'JJ', 'NNS', 'RB', 'VBN', 'TO', 'VB', 'JJ', 'IN', 'DT', 'NN', 'CC', 'NN', 'VBD', 'CD', 'NN', 'CC', 'JJR', '.', '--s--', 'PRP', 'VBP', 'NNP', ',', 'RB', 'CD', 'NN', ':', 'NNP', ',', 'RB', 'CD', 'NN', ':', 'NNP', 'NNP', ',', 'RB', 'CD', 'NN', ':', 'NNP', 'NNPS', ',', 'RB', 'CD', 'NN', ':', 'NNP', ',', 'RB', 'CD', 'NN', ':', 'NNP', 'NNP', 'NNPS', ',', 'RB', 'CD', 'NN', ',', 'CC', 'NNP', ',', 'RB', 'CD', 'NN', '.', '--s--', 'DT', 'NN', ':', 'CC', 'NN', 'NNS', ':', 'VBP', 'RB', 'RBR', 'JJ', 'IN', 'NNS', 'WDT', 'VBP', 'VBN', 'IN', 'JJS', 'CD', 'NN', ',', 'NN', 'NN', 'CC', 'NN', 'NN', 'NN', 'IN', 'RB', '.', '--s--', 'VBN', 'DT', 'NN', 'IN', 'PDT', 'DT', 'NN', 'NN', 'NN', 'CC', 'DT', 'NN', 'NN', ',', 'NNS', 'VBP', 'IN', 'DT', 'NNS', 'MD', 'VB', 'VBN', 'RB', 'RB', '.', '--s--', 'NNP', 'VBP', 'NNP', ',', 'WP$', 'NN', 'TO', 'VB', 'VBN', 'IN', '$', 'CD', 'DT', 'NN', 'IN', 'NNP', 'VBD', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNS', 'VBD', 'RB', 'CD', 'NN', 'IN', 'NNP', 'TO', 'VB', 'IN', 'CD', 'CD', '.', '--s--', 'NNP', ',', 'WDT', 'RB', 'VBN', 'CC', 'RB', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'NN', 'NN', 'WDT', 'VBD', 'VBN', 'TO', 'VB', 'VB', 'DT', 'NN', ',', 'VBD', 'CD', 'NN', 'TO', 'VB', 'IN', 'CD', 'CD', '.', '--s--', 'NNP', 'VBZ', 'VBD', 'PRP', 'VBZ', 'TO', 'VB', 'DT', 'JJ', 'NN', 'NN', 'CC', 'VBZ', 'RB', 'VBN', 'WRB', 'PRP', 'MD', 'VB', 'RB', '.', '--s--', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', ',', 'WDT', 'VBD', 'JJ', 'NN', 'TO', 'VB', 'DT', 'JJR', 'NN', 'IN', 'DT', 'NN', 'NN', 'WDT', 'VBZ', 'NNP', 'NNP', 'NNP', 'CC', 'DT', 'NNP', 'NN', 'IN', 'NNP', ',', 'VBD', 'CD', 'NN', 'IN', 'NNP', 'TO', 'VB', 'IN', 'CD', '.', '--s--', 'DT', 'NN', 'NN', 'VBZ', 'VBG', '$', 'CD', 'DT', 'NN', 'IN', 'CD', 'CD', 'NNP', 'NNP', 'NNS', ',', 'RB', 'IN', 'PRP$', 'NN', 'IN', 'NNP', 'IN', '$', 'CD', 'DT', 'NN', 'IN', 'CD', 'CD', 'NNS', '.', '--s--', 'CC', 'NN', 'NNS', 'VBP', 'DT', 'NN', 'MD', 'VB', 'VBN', 'DT', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNPS', ',', 'IN', 'NN', ',', 'VBD', 'IN', 'DT', 'JJ', 'NNS', ',', 'VBG', 'CD', 'NN', 'TO', 'VB', 'IN', 'CD', ',', 'RB', 'CD', 'CD', 'IN', 'NNP', '.', '--s--', 'NNP', 'RB', 'VBZ', 'VBG', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NN', 'CC', 'DT', 'IN', 'PRP$', 'NN', 'CC', 'NN', 'NNS', '.', '--s--', 'NNS', 'JJ', 'IN', 'NNP', 'VBD', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNS', 'VBD', 'JJ', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'MD', 'VB', 'JJ', 'IN', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'PRP', 'VBD', 'IN', 'DT', 'JJ', 'NNS', 'MD', 'VB', 'IN', 'NN', 'NNS', 'CC', 'MD', 'VB', 'VBN', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'TO', 'VB', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NNS', 'VBD', 'PRP', 'VBD', 'RB', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'POS', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'NNP', 'POS', 'JJ', 'NN', ',', 'VBD', 'NNS', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'CC', 'DT', 'IN', 'PRP$', 'NNS', 'VBP', '``', 'IN', 'NN', "''", 'IN', 'WP', 'VBZ', 'VBN', 'VBN', 'RB', 'IN', '``', 'DT', 'JJ', 'CC', 'JJ', 'NN', '.', "''", '--s--', '``', 'PRP', 'MD', 'RB', 'VB', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'MD', 'VB', 'VBN', 'IN', 'CD', 'NN', 'POS', 'NN', ',', "''", 'NNP', 'NNP', 'VBD', '.', '--s--', 'CC', 'DT', 'NN', 'NN', 'IN', 'DT', 'NN', ',', 'VBN', 'IN', 'PRP', 'VBZ', 'IN', 'NN', 'NN', ',', 'VBZ', 'JJ', 'TO', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', ',', 'DT', 'IN', 'DT', 'JJ', 'NN', 'CC', 'IN', 'JJ', 'NNS', '.', '--s--', 'IN', 'JJ', 'NNS', ',', '``', 'DT', 'NN', 'NN', 'VBZ', 'VBN', 'RP', 'IN', 'DT', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', 'NN', 'NN', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'VBN', 'VBG', 'CD', 'NN', 'IN', 'PRP$', 'NN', 'IN', 'NN', '.', '--s--', '``', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'RP', 'TO', 'DT', 'NN', '.', "''", '--s--', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', 'VBD', 'TO', 'DT', 'NN', '.', '--s--', 'IN', 'PDT', 'DT', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'CC', 'JJ', 'NN', ',', 'JJ', 'VBP', 'JJR', 'CC', 'JJR', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'CC', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'TO', 'VB', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NN', ',', 'RB', ',', 'DT', 'NN', 'CC', 'JJ', 'NN', 'MD', 'VB', 'JJR', 'IN', 'VB', 'DT', 'JJS', 'NN', 'NN', 'CC', 'NN', 'NNS', '.', '--s--', 'PRP', 'MD', 'RB', 'VB', 'DT', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'RB', ',', 'PRP', 'MD', 'VB', 'WRB', 'JJ', 'NNS', 'VBN', 'TO', 'NNS', 'IN', 'JJ', 'NNS', 'CC', 'RB', 'VB', 'NN', 'IN', 'DT', 'NNS', '.', '--s--', 'WRB', 'DT', 'JJ', 'NN', 'VBZ', 'JJ', ',', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'VBN', '.', '--s--', 'IN', 'JJ', 'NNS', ',', 'NN', 'NN', 'NNS', ',', 'JJ', 'IN', 'VBG', 'NNS', 'CC', 'NN', 'NN', ',', 'VBP', 'RB', 'RBR', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', '.', '--s--', 'DT', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'NNP', 'NNP', 'CC', 'DT', 'NNS', 'IN', 'NNP', 'NNP', '.', '--s--', 'CC', 'DT', 'NN', 'VBZ', 'RB', 'JJ', ':', 'WRB', 'DT', 'NN', 'POS', 'NN', 'NNS', 'IN', 'DT', 'NN', ',', 'DT', 'NN', 'NNS', 'MD', 'VB', 'RB', 'RBR', 'IN', 'DT', 'VBG', 'NN', '.', '--s--', 'WRB', 'VBP', 'JJ', 'NN', 'NNS', 'VBN', 'TO', 'JJ', 'JJ', 'NNS', '.', '--s--', 'IN', 'NN', 'NNS', 'CC', 'NNS', 'VBP', 'JJ', 'JJ', 'JJ', 'NNS', 'IN', 'JJS', 'NNS', 'CC', 'NNS', 'VBP', 'DT', 'NNS', 'RB', 'WRB', 'JJ', 'IN', 'JJ', 'JJ', 'NN', 'CC', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'VBN', 'RP', 'RB', 'IN', 'DT', 'JJ', 'NNS', ',', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBD', 'DT', 'JJ', 'NNS', 'NN', 'CC', 'NN', '.', '--s--', 'NNP', 'NN', 'VBD', 'IN', 'CD', 'NN', 'IN', 'PRP$', 'CD', 'NN', 'CC', 'JJ', 'NNS', 'VBD', 'RB', 'CD', 'NN', '.', '--s--', 'RB', ',', 'DT', 'NN', 'NN', 'VBD', 'VBN', 'RB', 'RBR', '.', '--s--', 'IN', 'NN', ',', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'NN', 'NN', 'IN', 'CD', 'VBD', 'RP', 'CD', 'NN', 'IN', 'DT', 'CD', 'NN', 'NN', '.', '--s--', 'RB', 'JJ', 'NNP', 'POS', 'NN', 'VBD', 'IN', 'DT', 'NNS', ',', 'JJ', 'NN', 'VBD', '.', '--s--', 'NNP', 'VBZ', ',', 'IN', 'NN', ',', 'VBD', 'DT', 'JJ', 'CD', 'NN', 'IN', 'CD', 'CC', 'DT', 'NN', 'NN', 'IN', 'CD', '.', '--s--', 'WRB', ',', 'JJ', 'NNS', 'CC', 'JJ', 'NNS', 'VBP', 'RB', 'VBG', ',', 'VBP', 'NN', 'POS', 'NNS', '.', '--s--', 'VB', 'DT', 'JJ', 'IN', 'JJ', 'NNP', 'CC', 'NNP', ',', 'CD', 'IN', 'DT', 'RBS', 'JJ', ',', 'JJ', 'CC', 'RB', 'VBN', 'NN', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'NN', ',', 'DT', 'NNS', 'VBP', 'VBD', 'NNP', 'CC', 'NNP', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'RB', 'VBZ', 'TO', 'VB', 'DT', 'NN', 'TO', 'VB', '.', '--s--', 'NNP', 'NNP', 'POS', 'NN', 'VBD', 'NN', 'CC', 'RB', 'VB', 'IN', 'DT', 'NNS', 'IN', 'PRP$', 'CD', 'NN', '.', '--s--', 'IN', 'DT', 'NNS', ',', 'NN', 'VBD', 'RP', 'IN', 'JJ', 'IN', 'CD', 'NN', 'IN', 'DT', 'CD', 'NN', 'CC', 'JJ', 'NNS', 'VBD', 'RP', 'CD', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBD', 'RB', 'RBR', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NN', 'VBD', 'CD', 'NN', '.', '--s--', 'DT', 'NN', 'NN', 'VBD', 'JJR', 'IN', 'CD', 'NN', 'IN', 'RB', 'DT', 'NN', 'NN', ',', 'CC', 'JJR', 'IN', 'CD', 'NN', 'IN', 'NN', 'NN', 'VBD', 'JJ', '.', '--s--', 'TO', 'DT', 'NNS', ',', 'DT', 'JJ', 'NN', 'NNS', 'IN', 'NNP', 'POS', '``', 'JJ', 'NN', "''", 'VBD', 'JJ', 'IN', 'DT', 'RB', 'JJ', 'NN', '.', '--s--', 'IN', 'JJ', ',', 'DT', 'NN', 'NN', 'VBD', 'VBN', '.', '--s--', 'RB', ',', 'DT', 'NN', 'POS', 'NN', 'VBD', 'DT', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', 'POS', 'NN', 'VBZ', 'VBN', 'JJ', 'IN', 'DT', 'JJ', 'CD', 'NNS', '.', '--s--', 'CC', 'JJ', 'NN', ',', 'IN', 'VBG', 'IN', 'DT', 'NNS', ',', 'VBZ', 'VBN', 'TO', 'PRP$', 'CD', 'NN', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', ',', 'JJ', 'NNP', 'POS', 'NN', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'RBR', 'IN', 'CD', 'NNS', 'VBD', 'VBN', 'IN', 'NN', 'NNP', 'JJ', 'NN', ',', 'RB', 'IN', 'NN', ',', 'NN', 'NN', ',', 'CC', 'DT', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'NNP', 'IN', 'DT', 'JJ', 'CD', 'NN', 'NNS', 'IN', 'DT', 'NN', 'JJ', 'NN', '.', '--s--', 'CC', 'NNS', 'VBP', 'DT', 'CD', 'NN', 'TO', 'CD', 'NN', 'NN', 'NN', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'JJ', 'NNS', ':', 'RB', 'RB', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', 'WRB', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'VBN', 'TO', 'DT', 'NN', 'POS', 'JJ', 'NNS', ',', 'IN', 'NNP', 'VBZ', 'RB', 'VBN', 'IN', 'JJ', 'DT', 'JJ', 'NN', 'IN', 'NNP', '.', '--s--', 'IN', 'DT', 'NNS', ',', 'JJ', 'NNP', 'POS', 'NN', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'JJ', 'CD', 'NNS', '.', '--s--', 'PRP$', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'IN', 'CD', ',', 'DT', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'NNP', 'VBZ', 'RB', 'CD', 'NN', 'JJR', 'IN', 'IN', 'CD', '.', '--s--', 'VBG', 'NN', 'IN', 'JJ', 'NNS', 'RB', 'VBD', 'CD', 'NN', 'IN', 'CD', 'CC', 'CD', '(', 'DT', 'RBS', 'JJ', 'NN', 'JJ', ')', '.', '--s--', 'DT', 'NNS', 'IN', 'NN', ',', 'RB', ',', 'VBP', 'JJR', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NNS', ',', 'CC', 'DT', 'VBZ', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'JJ', 'NNS', ',', 'RB', ',', 'DT', 'NN', 'POS', 'NN', 'NN', 'RB', 'CC', 'VBD', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'NN', 'NNS', 'VBP', 'RB', 'JJR', 'IN', 'CD', 'NN', 'IN', 'PRP$', 'CD', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'MD', 'VB', 'WRB', 'JJ', 'NNP', 'POS', 'NN', 'NN', 'MD', 'VB', ',', 'JJ', 'NN', 'VBZ', 'NNS', 'VBG', 'JJ', 'CD', 'NNS', 'VBP', 'IN', 'NN', 'CC', 'JJ', ':', 'JJ', 'IN', 'DT', 'JJ', 'NN', 'NN', ',', 'DT', 'JJ', 'NNS', 'VBD', 'DT', 'JJ', 'NN', ',', 'CC', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'WRB', 'NNP', 'POS', 'JJ', 'NN', 'VBZ', 'TO', 'VB', 'RBR', ':', 'PDT', 'DT', 'NN', 'MD', 'VB', 'IN', 'JJ', 'IN', 'JJ', 'NN', ':', 'NN', 'CC', 'NN', 'NN', 'MD', 'VB', ',', 'CC', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NN', 'MD', 'VB', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'WP', 'NN', 'NN', 'MD', 'PRP', 'VBP', 'IN', 'DT', 'JJ', 'NN', 'WP$', 'JJ', 'NN', 'VBZ', 'RB', 'IN', 'JJ', 'TO', 'VB', 'IN', 'NNP', 'POS', 'CC', 'NNP', 'POS', '.', '--s--', 'RB', ',', 'NN', 'IN', 'JJ', 'JJ', 'NN', 'VBZ', 'VBG', 'IN', 'PRP$', 'NNS', 'NN', ',', 'RB', 'IN', 'JJ', 'RB', 'JJ', 'JJ', 'NNS', 'IN', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNS', 'IN', 'DT', 'CD', 'NN', 'TO', 'CD', 'NN', 'NN', 'NN', 'MD', 'RB', 'VB', 'TO', 'DT', 'JJ', 'TO', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNS', '.', '--s--', 'IN', 'DT', 'NNS', ',', 'RB', 'DT', 'VBG', 'RP', 'IN', 'JJ', 'NN', ':', '``', 'VBG', 'JJ', "''", ':', 'IN', 'DT', 'JJ', 'JJ', 'NNS', 'MD', 'VB', 'JJ', 'NNS', 'IN', 'NNS', 'CC', 'JJ', 'NN', 'NNS', 'VBG', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'NNS', '.', '--s--', 'DT', 'VBG', 'IN', 'NN', 'NNS', 'IN', 'JJ', 'JJ', 'NNS', 'MD', 'VB', 'IN', 'NNS', 'CC', 'VB', 'RB', ':', 'DT', 'VBG', 'IN', 'JJ', 'JJ', 'NNS', 'MD', 'VB', 'DT', 'JJ', '(', 'JJ', ')', 'NN', 'CC', 'VB', 'IN', 'DT', 'NNS', 'RB', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'VBG', 'NN', 'IN', 'DT', 'NN', 'JJ', 'NN', 'VBN', 'IN', 'NNP', 'NNP', ',', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBZ', 'PRP$', 'JJ', 'JJ', 'NNS', 'TO', 'VB', 'IN', 'CD', 'NNS', 'CC', 'CD', 'NNS', 'DT', 'NN', ',', 'VBN', 'IN', 'CD', 'NNS', 'DT', 'NN', 'DT', 'NN', 'RB', '.', '--s--', 'NNS', 'VBD', 'VBN', 'VBG', 'IN', 'DT', 'NN', 'POS', 'NNS', 'MD', 'VB', 'IN', 'CD', 'NNS', 'CC', 'CD', 'NNS', 'DT', 'NN', '.', '--s--', 'DT', 'JJ', 'JJ', 'NNS', 'VBD', 'TO', '$', 'CD', 'CD', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBN', 'NNS', 'NN', 'IN', 'PRP$', 'CD', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'DT', 'NN', 'VBD', 'PRP$', 'JJ', 'NNS', 'NN', 'VBD', 'TO', 'VB', 'NNS', '.', '--s--', 'NNP', ',', 'DT', 'NN', 'IN', 'JJ', 'NN', 'CC', 'JJ', 'NN', 'NN', ',', 'VBD', 'PRP', 'VBZ', 'PRP$', 'JJ', 'NNS', 'TO', 'VB', 'RBR', 'RB', 'IN', 'NN', 'IN', 'JJ', 'NNS', ',', 'WDT', 'VBP', 'IN', 'CD', 'NNS', 'CC', 'CD', 'NNS', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PDT', 'DT', 'NNP', 'NNP', 'NNP', 'CC', 'DT', 'NNP', 'NNP', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'JJ', 'NNS', 'IN', 'NNP', 'NN', 'NNP', 'NNP', 'TO', 'CD', 'NNP', 'NNS', 'WP', 'VBZ', 'IN', 'JJ', 'NNS', 'IN', 'PRP', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'RB', 'DT', 'NN', 'IN', 'DT', '$', 'CD', 'CD', 'JJ', 'JJ', 'NN', 'VBG', 'PRP', 'IN', 'VBG', 'RP', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'PRP', 'VBD', ',', 'NNP', 'NNPS', 'CC', 'NNP', 'NNP', ',', 'CC', 'VBG', 'PRP', 'IN', 'NNS', '.', '--s--', 'NNP', 'NNP', ':', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NNS', 'NN', ':', 'VBD', 'NNP', 'NNP', 'RB', 'VBZ', 'VBN', 'VBG', 'TO', 'VB', 'NN', 'IN', 'DT', 'NNS', ':', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ',', 'NNP', 'NNP', 'IN', 'NNP', ',', 'NNP', 'NNP', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'IN', 'NNP', ':', 'CC', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNP', 'VBN', 'DT', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'WDT', 'VBD', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'NNP', ':', '``', 'CD', 'NN', ':', 'VBD', 'TO', 'VB', 'IN', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'DT', 'NN', 'VBD', 'JJ', 'JJ', 'NNS', 'TO', 'VB', 'RP', 'PRP$', 'NN', '.', '--s--', 'PRP', 'VBP', 'TO', 'VB', 'IN', 'DT', 'RBS', 'JJ', 'NN', 'PRP', 'MD', ':', 'PRP', 'RB', 'VBP', 'RB', '.', "''", '--s--', 'IN', 'DT', 'RB', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'NNP', 'CD', ',', 'DT', 'CD', 'NNS', 'VBD', 'JJ', 'NNS', 'TO', 'VB', 'RP', 'IN', 'NNP', '.', '--s--', 'VBG', 'TO', 'NNS', 'VBN', 'IN', 'CD', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NN', ',', 'DT', 'NNS', 'VBD', 'NNP', 'VBD', 'VBG', 'RB', 'IN', 'NNS', 'POS', 'RB', 'JJ', 'NN', 'CC', 'VBD', '``', 'DT', 'JJ', 'NN', 'NN', '.', "''", '--s--', 'NNP', 'NNP', 'VBD', 'VBN', 'IN', 'DT', 'NNS', 'VBD', 'VBG', 'RB', 'VBN', '.', '--s--', 'DT', 'NNS', 'VBP', 'IN', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'POS', 'NNS', '``', 'RB', 'JJ', ',', "''", 'CC', 'IN', 'NNP', 'NNP', 'VBD', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'VBD', '``', 'JJ', 'CC', 'JJ', '.', "''", '--s--', 'IN', 'DT', 'JJ', 'CD', 'NNS', ',', 'DT', 'NNP', 'NNP', ',', 'WDT', 'IN', 'DT', 'NN', 'VBD', 'DT', 'NN', 'JJ', 'IN', 'VBG', 'NNS', ',', 'VBD', 'TO', 'VB', ':', 'RB', 'IN', 'JJ', 'NNS', 'VBD', 'IN', 'NNP', 'CD', 'IN', 'NNP', 'NNP', 'VBD', 'VBN', 'NNP', 'TO', 'VB', 'JJ', '.', '--s--', 'NNP', 'POS', 'NN', 'NN', ',', 'NNP', 'NNP', 'NNP', ',', 'VBD', 'NN', 'NNS', 'DT', 'NNP', 'CD', ',', 'CC', 'NNS', 'VBD', 'DT', 'NN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'RB', 'VBN', 'NNP', 'NNP', 'NNP', ',', 'NN', 'TO', 'DT', 'NNP', 'NNP', ',', 'VBN', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'JJ', 'NNS', 'IN', 'NNP', 'CD', '.', '--s--', 'NNP', 'NNP', 'VBZ', 'VBN', 'PRP$', 'JJ', 'NN', ',', 'VBG', 'IN', 'PRP$', 'NN', 'VBD', 'VBN', 'RB', '.', '--s--', 'DT', 'NN', 'TO', 'NNS', 'IN', 'NNP', 'POS', 'NN', 'VBZ', 'VBN', 'VBN', 'IN', 'RB', 'RB', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNS', 'IN', 'DT', 'NN', 'VBP', 'VBN', 'NN', 'RB', 'IN', 'DT', 'JJ', 'CD', 'NNS', ',', 'RB', 'IN', 'DT', 'NN', 'IN', 'VBG', 'IN', 'JJ', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'CD', 'NN', 'NN', ',', 'NNP', 'NNP', ',', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', ',', 'VBD', '$', 'CD', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'VBD', 'IN', 'NNP', 'NNP', 'VBD', 'VBN', 'DT', 'NN', 'IN', 'PRP', 'RB', 'CD', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNS', '.', '--s--', 'NNP', 'NNP', ',', 'IN', 'NNS', 'IN', 'VBG', 'RP', 'JJ', 'NN', 'NN', ',', 'VBD', '$', 'CD', 'RB', 'JJ', 'NN', ',', 'RB', 'IN', 'DT', 'NN', 'RB', 'VBN', 'NNP', 'NNP', 'IN', 'VBG', 'NNP', '.', '--s--', 'IN', 'NN', ',', 'NNP', 'NNP', 'JJ', 'NN', 'VBN', 'IN', 'PRP', 'VBZ', 'VBN', 'VBN', '$', 'CD', 'TO', 'NNP', 'NNP', 'IN', 'NN', 'IN', 'NNS', 'PRP', 'CC', 'PRP$', 'NN', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'TO', 'NNP', 'NNP', 'POS', 'NN', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NNPS', ',', 'IN', 'CD', 'IN', 'CD', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'VBN', 'TO', 'VB', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NN', 'CC', 'IN', 'DT', 'NN', '``', 'VBD', 'IN', 'DT', 'NNS', '.', "''", '--s--', 'NNP', 'NNP', ',', 'PRP$', 'NN', 'NNS', 'CC', 'NNS', 'RB', 'VBN', '$', 'CD', 'TO', 'NNP', 'NNP', 'POS', 'JJ', 'NNS', 'IN', 'DT', 'NNS', ',', 'VBG', 'TO', 'VB', 'NNS', '.', '--s--', 'CC', 'NNP', 'NNP', 'VBZ', 'NNP', 'NNP', 'VBD', 'RP', 'PRP$', 'NN', 'RB', 'IN', 'CD', ',', 'IN', 'DT', 'NN', 'VBD', 'TO', 'VB', 'DT', 'NN', 'NN', 'POS', 'NN', 'IN', 'RB', 'IN', 'NNP', 'NNP', 'VBD', '.', '--s--', '``', 'PRP', 'VBD', 'RB', 'JJ', 'IN', 'DT', ',', 'VBD', 'PRP$', 'NN', 'CC', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', 'PRP', 'VBD', 'DT', 'NN', ',', "''", 'NNP', 'NNP', 'VBZ', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNS', 'VBN', 'IN', 'NNP', 'NNP', 'VBD', '$', 'CD', 'IN', 'JJ', 'NNS', 'TO', 'CD', 'JJ', 'NN', 'NN', 'NNS', 'IN', 'CD', 'CC', 'CD', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'WP', 'VBD', 'IN', 'VBG', 'DT', 'NN', 'VBD', '``', 'DT', 'JJ', 'JJ', 'NN', 'TO', 'VB', 'RB', '.', "''", '--s--', 'IN', 'NN', ',', 'NNP', 'NNP', 'VBD', '$', 'CD', 'IN', 'NN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'CC', 'DT', 'NNP', 'JJ', 'NN', 'VBD', '$', 'CD', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'CD', 'JJ', 'NN', 'WDT', 'VBD', 'DT', 'NN', 'POS', 'NN', 'NN', 'IN', 'NN', '.', '--s--', 'RB', 'IN', 'NNP', ',', 'NNP', 'NNS', 'VBD', '$', 'CD', 'IN', 'JJ', 'NNS', 'IN', 'NNP', 'NNP', 'TO', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'NNP', 'NNP', '.', '--s--', 'DT', 'VBD', 'IN', 'NN', 'TO', '$', 'CD', 'IN', 'JJ', 'NN', 'NNS', 'VBN', 'IN', 'NNP', 'NNP', 'TO', 'DT', 'NNP', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNP', 'NNP', 'MD', 'VB', 'DT', 'JJ', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', ',', 'RB', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', ',', 'NN', 'NN', 'IN', 'DT', 'NNS', 'NN', ',', 'MD', 'RB', 'VB', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'MD', 'VB', 'IN', 'DT', 'NN', ',', 'CC', 'IN', 'PRP$', 'NN', 'VBD', 'VBN', '``', 'RB', 'JJ', '.', "''", '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'VBN', '``', 'RB', 'VBN', "''", 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'PRP', 'MD', 'VB', 'TO', 'VB', 'JJ', ',', 'CC', 'VBD', 'PRP', 'VBD', '``', 'RB', 'JJ', "''", 'IN', 'PRP', 'TO', 'VB', 'JJ', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'RB', 'VBN', 'DT', 'JJ', 'NN', 'IN', 'PRP$', 'NN', 'IN', 'DT', 'NN', 'CC', '``', 'PRP', 'VBP', 'RB', 'VBN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', "''", '--s--', 'NNP', 'NNP', 'VBD', ',', '``', 'WRB', 'DT', 'VBZ', 'VBN', 'CC', 'VBN', ',', 'PRP', 'VBP', 'TO', 'VB', 'RB', 'VBD', '.', "''", '--s--', 'NNP', 'NNP', ',', 'WP', 'VBD', 'RB', 'VBD', 'PRP$', 'NN', 'TO', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'VBD', 'PRP$', 'NN', 'IN', 'CD', 'IN', 'VBG', 'NNS', 'TO', 'VB', '.', '--s--', '``', 'WRB', 'VBD', 'RB', 'DT', 'NNP', 'NNP', 'NN', 'RBR', '.', "''", 'PRP', 'VBD', '.', '--s--', '``', 'DT', 'VBZ', 'WP', 'NNP', 'NNP', 'MD', 'VB', 'VB', 'VBN', '.', '--s--', 'NNP', 'NNPS', 'NNP', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'TO', 'VB', 'CD', 'NN', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'NNP', '.', '--s--', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJR', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNP', 'VBD', 'PRP', 'VBZ', 'TO', 'VB', 'NN', 'IN', 'DT', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'CD', '.', '--s--', 'PRP', 'VBD', 'DT', 'CD', 'NN', 'NNS', 'VBP', 'IN', 'NN', 'TO', 'DT', 'JJ', 'NN', 'NN', 'NNS', 'RB', 'VBG', 'VBN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NNP', ',', 'JJ', 'NN', 'NN', '.', '--s--', 'WRB', 'NNP', 'VBD', 'PRP$', 'NNS', 'TO', 'JJ', 'NNS', 'IN', 'CD', ',', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'VBD', 'IN', 'DT', 'JJ', 'TO', 'VB', 'RP', '.', '--s--', 'NNP', ',', 'IN', 'IN', 'CD', 'NN', 'IN', 'DT', 'NNS', 'POS', 'NNS', 'VBG', 'VBN', 'IN', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNPS', 'NNP', ',', 'NNP', 'NNP', ',', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'NN', ':', '``', 'VBN', 'IN', 'NNP', 'IN', 'NNP', 'NNP', 'NNPS', '.', "''", '--s--', 'DT', 'NN', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'VBG', 'NN', 'CC', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'IN', 'NNP', 'CD', ',', 'DT', 'NNP', 'NNP', 'NN', 'NN', 'VBZ', 'VBG', 'IN', 'PRP$', 'JJ', 'NN', 'IN', 'NNP', '.', '--s--', 'IN', 'DT', 'NNS', 'VBP', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'VBN', 'IN', 'NNP', 'POS', 'JJ', 'NN', ',', 'PRP', 'VBP', 'VBG', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'NN', 'NNS', ',', 'CC', 'PRP$', 'JJ', 'NNS', ',', 'VBP', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'CC', 'JJ', 'NNS', '.', '--s--', '``', 'NN', 'VBZ', 'TO', 'VB', 'PDT', 'PRP$', 'NNS', 'IN', 'CD', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'CC', 'JJ', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '.', '--s--', 'RB', ',', 'NNP', 'CC', 'JJ', 'JJ', 'NNP', 'NNP', 'NN', 'NNS', 'VBD', 'VBG', 'RP', 'NNS', 'IN', 'NNP', 'NNP', ',', 'RB', 'IN', 'NNP', ',', 'RB', 'IN', 'DT', 'NN', '.', '--s--', 'PRP$', 'NNS', 'VBD', 'RB', 'VBN', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'NNS', 'IN', 'JJ', 'NNP', '.', '--s--', 'IN', 'VBG', 'NN', 'NNS', ',', 'DT', 'NN', 'NNS', 'VBP', 'RB', 'VB', 'TO', 'VB', 'IN', 'NNP', ',', 'NNS', 'CC', 'JJ', 'NNS', 'VBP', '.', '--s--', 'PRP', 'MD', 'RB', 'VB', 'JJ', 'TO', 'VB', 'RB', 'DT', 'NN', 'NN', 'VBN', 'RP', 'IN', 'JJ', 'NNP', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'JJ', 'NN', 'CC', 'DT', 'NN', 'TO', 'NNP', 'NNP', 'POS', 'NN', ',', 'NN', ',', 'NNS', 'CC', 'NN', 'NNS', ',', 'JJ', 'IN', 'NN', 'NNS', ',', 'VBP', 'VBN', 'NNP', 'POS', 'NNP', 'NN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', '``', 'NNP', 'NNP', 'VBZ', 'DT', 'RBS', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'NNPS', 'NNP', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NN', 'NNS', 'VBP', 'DT', 'NN', 'NNS', 'POS', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'CD', ',', 'NNS', 'IN', 'RB', 'VBN', 'NNS', 'CC', 'NNS', 'VBD', 'CD', 'NN', 'IN', 'CD', ',', 'TO', '$', 'CD', 'CD', '(', '$', 'CD', 'CD', ')', '.', '--s--', 'CC', 'JJ', ',', 'RB', 'IN', 'NNP', ',', 'VBD', 'CD', 'NN', ',', 'TO', '$', 'CD', 'CD', '.', '--s--', 'IN', 'CD', 'POS', 'JJ', 'CD', 'NNS', ',', 'JJ', 'NNS', 'VBD', 'CD', 'NN', ',', 'TO', '$', 'CD', 'CD', ',', 'IN', 'NNP', 'VBD', 'CD', 'NN', ',', 'TO', '$', 'CD', 'CD', '.', '--s--', 'NNPS', 'VBP', 'EX', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNP', ',', 'WRB', 'DT', 'JJ', 'CD', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'NN', 'NN', '.', '--s--', '``', 'IN', 'DT', 'JJ', 'JJ', 'NNS', ',', 'IN', 'PRP', 'CC', 'RB', ',', 'NNP', 'VBZ', 'VBG', 'TO', 'VB', 'DT', 'JJ', 'NN', ',', "''", 'VBZ', 'NNP', 'NNP', ',', 'NN', 'NN', 'IN', 'NNP', 'NNPS', 'NNP', ',', 'CD', 'IN', 'DT', 'JJ', 'JJ', 'NNP', 'NNP', 'NN', 'NNS', 'TO', 'VB', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'IN', 'NNS', 'CC', 'NNS', 'VBP', 'JJ', 'NNS', ',', 'PRP', 'VBP', 'VBG', 'RB', 'IN', 'NNP', 'NNP', '.', '--s--', 'JJ', 'JJ', 'NNS', 'VBP', 'VBN', 'VBG', 'JJ', 'NNS', 'IN', 'NNP', ',', 'VBG', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'NN', 'VBN', 'IN', 'NNP', 'NNP', '.', '--s--', 'NNP', ',', 'DT', 'NNPS', 'CC', 'NNP', 'RB', 'VBP', 'VBG', 'VBN', '.', '--s--', 'IN', 'DT', 'NNP', 'NNP', 'VBD', 'TO', 'VB', 'PRP$', 'JJ', 'NN', 'NNS', 'IN', 'CD', ',', 'JJ', 'NNP', 'NNP', 'NNS', 'VBP', 'VBG', 'TO', 'VB', 'NNP', ',', 'NNP', 'CC', 'NNP', 'IN', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'IN', 'NNP', 'VBD', 'RB', 'IN', 'NNP', 'NNP', 'POS', 'NN', 'NN', 'VBD', 'VBG', 'IN', 'DT', 'CD', 'NNS', 'NN', 'CC', 'NN', 'NNS', 'IN', 'CD', 'JJ', 'NNP', 'NNS', ',', 'NNP', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'NNPS', 'NNP', '.', '--s--', 'NNP', 'NNP', 'NNS', 'VBP', 'JJ', 'NN', 'NNS', 'CC', 'JJ', 'JJ', 'NNS', 'VBG', 'IN', 'DT', 'CD', 'NNS', 'VBD', 'DT', 'JJ', 'NN', ',', 'VBG', 'PRP', 'TO', 'VB', 'NN', 'NNS', 'CC', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'JJ', 'NN', 'CC', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', 'WDT', 'MD', 'VB', 'TO', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'PRP', 'RB', 'IN', 'DT', '``', 'JJ', "''", 'NN', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'DT', 'NNP', ',', 'NNP', 'NNP', 'POS', 'JJS', 'NN', 'IN', 'NNS', 'CC', 'NNS', '.', '--s--', 'DT', 'JJS', 'NN', 'DT', 'NN', 'POS', 'NNS', 'VBP', 'TO', 'DT', 'NNP', 'VBD', 'DT', 'NN', 'VBZ', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'NN', 'VBZ', 'VBN', 'IN', 'NNP', 'NNPS', 'NNP', '.', '--s--', 'NNP', 'IN', 'CD', ',', 'DT', 'JJ', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'CC', 'NN', 'NN', '.', '--s--', 'NNP', 'NNP', ',', 'VBG', 'NN', 'IN', 'NNP', ',', 'VBZ', 'CD', 'CD', 'NNP', 'NNP', 'VBP', 'VBN', 'VBN', ',', 'VBG', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'CD', 'JJ', 'NNS', 'IN', 'DT', 'NNP', '.', '--s--', 'NNP', 'NNS', 'VBP', 'TO', 'VB', 'JJ', 'IN', 'DT', 'NNP', 'NN', ',', 'WDT', 'VBZ', 'IN', 'IN', 'CD', 'NN', 'IN', 'NNP', 'JJ', 'NN', 'NNS', ',', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNP', 'MD', 'VB', 'CD', 'DT', 'NN', 'NNS', 'NN', 'IN', 'NNP', '.', '--s--', 'JJ', 'NNP', 'NNP', 'NNS', 'VBP', 'PRP$', 'NNS', 'TO', 'VB', 'RB', 'RB', 'DT', 'NN', 'IN', 'CD', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'PRP', 'VBP', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', 'VBN', 'IN', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', 'NNP', 'NNS', 'VBP', 'JJ', 'NN', 'VBD', 'TO', 'VB', 'IN', 'JJ', 'NNS', '.', '--s--', 'JJ', 'IN', 'DT', 'NN', 'NNS', 'POS', 'NNS', 'VBD', 'RB', 'IN', 'NNP', 'CD', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'CC', 'NNP', 'POS', 'NN', 'NNS', 'VBD', 'RB', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'NNP', 'VBZ', 'VBN', 'VBN', 'RB', 'CD', 'NNS', 'DT', 'NN', 'IN', 'JJ', 'NNS', ',', 'RB', 'IN', 'CD', 'NNS', 'DT', 'NN', ',', 'IN', 'DT', 'NN', 'POS', 'JJ', 'NN', 'VBZ', 'VBN', 'PRP$', 'NN', 'NN', '.', '--s--', 'NNPS', 'VBP', 'VBG', 'JJ', 'NN', 'NNS', '.', '--s--', 'NNP', 'NN', 'IN', 'NNP', 'NNS', 'RB', 'CC', 'VBD', 'RP', 'VBG', 'NNP', 'CD', '.', '--s--', 'RB', ',', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NNP', 'NN', 'MD', 'VB', 'PRP$', 'NN', 'IN', 'NNP', ',', 'WDT', 'VBZ', 'VBN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'CC', 'JJ', 'NNS', '.', '--s--', 'CC', ',', 'NN', 'NNS', 'CC', 'JJ', 'NNS', 'VBP', 'NNP', 'MD', 'VB', 'VBN', 'IN', 'VBG', 'NNS', 'IN', 'JJ', 'JJ', 'NNP', '.', '--s--', 'PRP', 'VBP', 'NNP', 'POS', 'NN', 'NN', 'VBZ', 'VBG', 'CC', 'DT', 'NN', 'VBZ', 'RB', 'JJ', 'IN', 'JJ', 'NN', 'IN', 'PRP', 'TO', 'VB', 'NN', 'IN', 'NNP', '.', '--s--', '``', 'DT', 'JJ', 'NNS', 'VBP', 'TO', 'VB', 'IN', 'PRP', 'VBP', 'NN', 'CC', 'IN', 'DT', 'VBP', 'NNS', ',', "''", 'VBZ', 'NNP', 'NNP', 'IN', 'NNP', '.', '--s--', 'DT', 'NNP', 'NN', ',', 'VBG', 'DT', 'NNP', 'NNP', 'TO', 'VB', 'NNS', 'JJR', 'NN', 'TO', 'VB', 'NNS', ',', 'VBD', 'NNS', 'VBP', 'RB', 'DT', 'NN', 'TO', 'NN', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NNS', '.', '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'IN', 'DT', 'CD', 'NNP', 'NNP', 'NN', ',', 'NNP', 'FW', 'NNP', ',', 'VBG', 'DT', 'JJ', 'NN', 'TO', 'NN', ',', 'VBD', 'JJ', '.', '--s--', 'PRP', 'RB', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'JJ', 'IN', 'CD', 'TO', 'VB', 'IN', 'NNS', 'VBP', 'DT', 'NN', 'TO', 'NN', 'WDT', 'MD', 'RB', 'VB', 'RB', 'VBN', 'IN', 'PRP$', 'NNS', '.', '--s--', 'DT', 'NN', 'POS', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'CD', 'IN', 'CD', 'NN', 'NNS', 'DT', 'NNP', 'NNP', 'MD', 'VB', 'VBN', 'CC', 'MD', 'VB', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBG', 'DT', 'NNP', 'NN', 'WDT', 'VBZ', 'IN', 'DT', 'NNS', 'IN', 'DT', 'JJ', 'VB', 'VBN', 'IN', 'PRP', 'MD', 'VB', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NNS', 'TO', 'VB', 'DT', 'JJ', 'NN', 'VBD', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NNP', 'WRB', 'DT', 'JJ', 'NN', 'VBD', 'NNP', 'POS', 'NN', 'NNS', '.', '--s--', 'IN', 'DT', 'NN', ',', 'WDT', 'VBZ', 'DT', 'NNS', 'IN', 'RB', 'CD', 'IN', 'DT', 'CD', 'NNS', ',', 'DT', 'NN', 'NN', 'IN', 'NN', 'VBZ', 'JJ', 'IN', 'DT', 'NN', 'VBZ', 'DT', '``', 'JJ', "''", 'NN', 'IN', 'VBG', 'PRP', '.', '--s--', 'DT', 'VBZ', 'DT', 'RB', 'JJR', 'NN', 'IN', 'DT', 'NN', 'TO', 'VB', 'IN', 'DT', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'CD', ',', 'WDT', 'VBZ', 'DT', 'NN', 'TO', 'VB', 'DT', '``', 'JJ', "''", 'NN', 'IN', 'VBG', 'NN', '.', '--s--', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NNP', 'NN', ',', 'DT', 'NNP', 'NN', 'VBD', 'IN', 'VBG', 'IN', 'DT', 'NNS', 'VB', 'VBN', 'VBZ', 'DT', 'JJ', 'NN', ',', 'CC', 'IN', 'EX', 'VBZ', 'DT', 'NN', 'TO', 'VB', 'DT', 'NN', 'WDT', 'VBZ', 'VBN', 'TO', 'VB', 'TO', 'NN', 'IN', 'DT', 'NN', 'POS', 'NN', 'RB', '.', '--s--', 'DT', 'NN', ',', 'FW', 'FW', 'NNP', ',', 'MD', 'VB', 'VBN', 'NNP', 'CD', '.', '--s--', 'NNP', 'NNP', 'IN', 'NNP', ',', 'VBD', 'RB', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'RB', ',', 'VBD', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'CD', 'NN', 'TO', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'POS', 'NN', 'CD', 'NN', 'NN', 'VBD', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', ',', 'DT', 'NN', 'RBR', '.', '--s--', 'NNP', 'VBD', 'CD', 'NN', 'TO', '$', 'CD', 'CD', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNS', ',', 'WP', 'VBD', 'VBG', 'NNS', 'TO', 'VB', 'IN', '$', 'CD', 'TO', '$', 'CD', 'DT', 'NN', ',', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'JJ', 'NNS', '.', '--s--', '``', 'PRP', 'VBZ', 'JJ', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NNP', 'CC', 'NNP', '.', '--s--', 'RB', 'IN', 'DT', 'NNS', 'NN', 'VBD', 'VBN', 'IN', 'JJ', 'NN', 'NNS', ',', 'WDT', 'VBD', 'DT', 'JJ', 'NN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'CD', 'NNS', 'DT', 'NN', ',', 'VBN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'CD', 'NNS', 'DT', 'NN', ',', 'DT', 'JJ', 'NN', '.', '--s--', 'JJR', 'NNS', 'IN', 'NN', 'NNS', 'CC', 'JJ', 'NN', 'NNS', 'CC', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'RB', 'VBD', 'TO', 'JJR', 'NNS', ',', 'DT', 'NN', 'VBD', '.', '--s--', '``', 'IN', 'NN', ',', 'NNS', 'VBD', 'RBR', 'RB', 'JJ', 'TO', 'JJ', 'NN', 'NNS', 'IN', 'NN', 'NN', ',', "''", 'DT', 'NN', 'VBD', '.', '--s--', 'VBG', 'DT', 'JJR', 'NN', 'NN', ',', 'WDT', 'VBD', 'CD', 'NN', 'NNS', 'TO', 'CD', 'NN', ',', 'CC', 'DT', 'JJ', 'NN', 'NN', 'NN', ',', 'DT', 'NN', 'MD', 'VB', 'VBN', 'NNS', 'POS', 'NNS', ',', 'VBD', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'NNP', ',', 'NNP', 'CC', 'NNP', '.', '--s--', 'VBG', 'IN', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'DT', 'NN', 'NN', 'POS', 'JJS', ',', 'NNP', 'NNP', 'VBD', ',', '``', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'DT', 'NN', 'IN', 'NN', 'IN', 'PRP', 'RB', '.', "''", '--s--', 'RB', ',', 'DT', 'NN', 'VBD', 'NNS', 'VBD', 'RP', 'RB', 'TO', 'CD', 'JJ', 'NNS', 'IN', 'CD', ',', 'VBG', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', '.', '--s--', 'DT', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'NN', 'VBD', '.', '--s--', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NN', 'NNP', ',', 'NNP', 'VBD', 'IN', '$', 'CD', 'DT', 'NN', ',', 'RB', '$', 'CD', ',', 'IN', 'DT', 'RB', 'JJR', 'NN', '.', '--s--', 'IN', 'CD', 'NNS', ',', 'JJ', 'NNS', 'VBP', 'VBN', 'DT', 'CD', 'JJ', 'NN', 'NNS', 'IN', 'VBG', 'IN', 'CD', 'IN', 'DT', 'RBS', 'JJ', 'CC', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', '--s--', 'CC', 'IN', 'CD', 'NNS', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', 'VBP', 'VBN', 'IN', 'NNP', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'TO', 'VB', 'IN', '.', '--s--', 'CC', 'IN', 'JJ', 'NNS', 'VBG', 'RP', 'NNP', 'NN', 'NNS', ',', 'DT', 'NNS', 'VBP', 'VBG', 'PRP$', 'NN', 'RBR', 'IN', 'RB', '.', '--s--', 'PRP', 'VBP', 'DT', 'JJ', 'NNS', 'MD', 'VBP', 'DT', 'NNP', 'NN', 'CC', 'VB', 'NNP', 'TO', 'VB', 'IN', 'VBG', 'JJ', 'NNS', 'WDT', 'VBP', 'DT', 'NNS', 'IN', 'VBG', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'CC', 'VBG', 'NN', 'IN', 'DT', 'VBZ', 'PRP', 'VBD', 'IN', 'DT', 'NN', '.', '--s--', 'RB', 'NN', 'NNS', ',', 'RB', ',', 'VBP', 'RB', 'IN', 'NN', ':', 'RB', 'IN', 'NNP', 'CC', 'IN', 'NNS', 'IN', 'DT', 'NNS', ':', 'VBZ', 'RB', 'JJ', 'DT', 'NN', 'RB', '.', '--s--', 'CC', 'RB', 'DT', 'NNS', 'RB', 'VBP', 'VBG', 'IN', 'JJ', 'NNS', 'TO', 'VB', 'DT', '``', 'JJ', "''", 'NNS', ',', 'VBN', 'RB', 'IN', 'DT', 'NNP', 'NNP', 'CC', 'NNP', 'NNPS', '.', '--s--', 'DT', 'VBD', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', ',', 'VBN', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'NN', 'NNP', 'NNP', 'NNP', ',', 'VBD', 'VBN', 'TO', 'VB', 'NN', 'NNP', 'NNP', 'NNP', 'POS', 'JJ', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', 'NNPS', 'NNP', '.', '--s--', 'NNP', 'POS', 'NN', 'MD', 'VB', 'DT', 'NN', ',', 'WDT', 'NNP', 'VBD', 'JJ', 'NN', 'WRB', 'DT', 'JJ', 'NN', 'VBD', 'NN', 'VBG', 'NN', '.', '--s--', 'RB', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', ',', 'NNP', 'VBZ', 'TO', 'VB', 'DT', '.', '--s--', '``', 'PRP$', 'NNS', 'VBP', 'JJ', ',', "''", 'DT', 'NNP', 'NN', 'VBZ', '.', '--s--', 'NNP', 'MD', 'RB', 'VB', 'DT', 'NN', 'TO', 'VB', 'DT', 'NN', ',', 'NN', 'NN', 'IN', 'DT', 'NN', 'IN', 'VBG', 'DT', 'NNS', '.', '--s--', 'CC', 'DT', 'NNP', 'NN', 'MD', 'VB', 'NNP', 'NNP', 'CC', 'NNP', 'POS', 'NN', ',', 'NNP', 'NNP', 'NNP', ',', 'TO', 'VB', 'IN', 'NNS', 'IN', 'VBG', 'DT', 'JJ', 'NNS', '.', '--s--', 'CC', 'DT', 'NNS', 'POS', 'NN', 'MD', 'RB', 'VB', 'DT', 'RB', 'JJ', 'NN', 'IN', 'PRP', 'CC', 'NNP', 'NNS', '.', '--s--', 'DT', 'NNS', 'VBP', 'TO', 'VB', 'RP', 'JJ', 'NN', 'IN', 'RB', 'DT', 'NN', 'IN', 'WRB', 'PRP', 'MD', 'VB', 'IN', 'VBG', 'JJ', 'NNS', '.', '--s--', 'JJ', 'NNS', 'VBN', 'TO', 'DT', 'NNS', 'VBP', 'DT', 'NNS', 'TO', 'VB', '.', '--s--', 'DT', 'NNS', 'RB', 'VBP', '``', 'NN', 'IN', 'PRP$', 'NN', "''", 'IN', 'WP', 'VBZ', 'VB', 'IN', 'DT', 'NN', ',', 'VBZ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'DT', 'RBS', 'JJ', 'NN', 'IN', 'VBG', 'DT', 'NNS', '.', '--s--', 'NNPS', 'VBP', '``', 'FW', "''", 'TO', 'VB', 'NNS', 'IN', 'JJ', 'NNS', 'CC', 'VB', 'PRP', 'RB', 'RB', 'RB', 'TO', 'VB', 'IN', 'JJ', 'NN', 'NNS', ',', 'PRP', 'VBZ', '.', '--s--', 'CC', 'WDT', 'VBZ', 'WRB', 'DT', 'NNS', ',', 'IN', 'DT', 'JJS', 'NN', ',', 'MD', 'VB', 'IN', 'NN', ',', 'PRP', 'VBZ', '.', '--s--', 'NNP', 'NNS', 'IN', 'IN', 'DT', 'NNS', ':', 'VBG', 'NNS', 'IN', 'NNP', 'NNPS', 'NNP', ',', 'NNP', 'NNP', 'NNP', ',', 'NNP', 'NNPS', 'NNP', 'CC', 'NNP', 'NNP', ':', 'VBD', 'TO', 'VB', 'VBN', '.', '--s--', 'CC', 'NNP', 'NNP', ',', 'WP', 'VBZ', 'DT', 'NNS', ',', 'VBZ', ':', '``', 'DT', 'JJ', 'NN', 'NN', ',', 'TO', 'DT', 'NN', ',', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'VBG', "''", 'DT', 'NNS', '.', '--s--', 'JJ', 'NNS', 'VBP', 'NN', 'NNS', 'PDT', 'DT', 'RBR', 'JJ', 'IN', 'DT', 'NNS', 'MD', 'VB', '.', '--s--', '``', 'PRP', 'VBP', 'RB', 'VB', 'TO', 'VB', 'DT', 'NN', 'TO', 'VB', ',', "''", 'VBZ', 'CD', 'NN', 'NN', '.', '--s--', '``', 'CC', 'EX', 'VBZ', 'DT', 'NN', 'IN', 'NNP', 'VBZ', 'VBN', 'TO', 'VB', 'DT', 'NNS', '.', '--s--', 'DT', 'VBZ', 'DT', 'NN', ',', 'VBZ', 'RB', 'PRP', '.', "''", '--s--', 'RB', 'PRP', 'VBZ', '.', '--s--', 'NNP', 'VBZ', 'VBD', 'RB', 'PRP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'VBG', 'DT', 'NNS', 'CC', 'DT', 'NNS', ',', 'WDT', 'VBP', 'NNS', 'TO', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'NNS', ',', 'TO', 'VB', 'RP', 'PRP$', 'JJ', 'NN', '.', '--s--', 'CC', 'JJ', 'NNS', 'VBP', 'VBN', 'DT', 'NNS', ':', 'CC', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'IN', 'JJ', ':', 'RB', 'RBR', 'JJ', 'IN', 'DT', 'NNS', 'MD', 'VB', 'VBN', 'TO', 'VB', '.', '--s--', 'DT', 'JJS', 'NNS', ':', 'NNP', 'NNP', 'POS', 'NN', 'TO', 'VB', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', '$', 'CD', 'CD', ',', 'CC', 'TO', 'VB', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', '$', 'CD', 'CD', '.', '--s--', '``', 'PRP', 'VBP', 'WP', 'NNP', 'NNP', 'MD', 'VB', 'IN', 'DT', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'RB', 'VBN', 'RB', 'IN', 'NNP', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', 'NN', ',', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'POS', 'NN', 'IN', 'NNP', 'NN', '.', '--s--', 'IN', 'PRP$', 'NN', ',', 'NNP', 'NNP', ',', 'WP', 'VBD', 'IN', 'NNP', 'JJ', 'NN', 'CC', 'JJ', ',', 'VBD', 'IN', 'DT', '``', 'JJ', 'NNS', "''", 'IN', 'JJ', 'NNS', 'VBP', '``', 'VBD', 'RB', 'TO', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'JJ', 'NNS', '.', '--s--', "''", '(', 'PRP', 'VBD', 'RB', 'VB', 'IN', 'NNP', ',', 'IN', 'DT', 'NN', ',', 'VBD', 'VBG', 'TO', 'VB', 'NNP', 'POS', 'NNP', 'IN', 'NN', '--s--', 'DT', 'NNP', 'NN', 'NNS', 'IN', 'NNP', 'NNP', 'POS', 'NNS', 'VBD', '``', 'RB', 'JJ', "''", 'IN', 'NNP', 'VBZ', 'IN', 'DT', 'NNP', 'NNS', 'MD', 'VB', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'RB', 'RB', 'RB', 'TO', 'VB', '``', 'JJ', 'NNS', 'RB', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'JJ', 'TO', 'NNP', ':', 'CC', 'NN', 'RB', 'IN', 'DT', 'CD', 'NNS', '.', "''", '--s--', 'DT', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'NN', 'VBD', 'VBG', 'IN', 'DT', 'NN', 'WRB', 'DT', 'JJ', 'NNP', 'NNP', 'VBD', 'RB', 'IN', 'NNS', 'TO', 'VB', 'NNP', '.', '--s--', 'IN', 'NNP', 'RB', 'VBD', 'DT', 'JJ', 'NN', 'TO', 'VB', 'NNS', 'IN', 'NNP', ',', 'DT', 'NN', 'MD', 'VB', 'NNP', 'VBZ', 'RB', 'JJ', 'JJ', 'NN', ',', 'NNP', 'NNP', 'NNP', ',', 'VBP', 'DT', 'NNP', 'NNS', 'VBN', 'IN', 'NNP', ':', 'DT', 'JJ', 'NN', 'WDT', 'VBZ', 'RB', 'VBN', 'IN', 'JJ', 'NNS', '.', '--s--', 'NNP', 'POS', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'IN', 'VBG', 'DT', 'NNS', 'POS', 'NN', 'TO', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'PRP', 'VBD', 'RP', 'RB', 'WRB', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'VBD', 'VBN', '.', '--s--', 'IN', 'NNP', 'POS', 'NN', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'VBD', 'VBN', ',', 'NNP', 'NNP', 'VBZ', 'RB', 'VBN', 'JJ', 'IN', 'NN', '.', '--s--', 'IN', 'DT', 'NNP', 'NN', ',', 'NNP', 'MD', 'VB', 'IN', 'JJ', 'NN', ':', 'RB', 'VBG', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'IN', 'NNP', '.', '--s--', '``', 'PRP', 'VBZ', 'RB', 'RB', 'VBN', 'VBN', ',', "''", 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'WP', 'IN', 'DT', 'JJ', 'NNP', 'NN', 'VBD', 'NNS', 'VBG', 'TO', 'VB', 'DT', 'NNS', 'VBN', '.', '--s--', 'PRP', 'RB', 'VBZ', 'NNP', 'NNPS', 'IN', 'NNP', ',', 'VBG', 'TO', 'NNS', 'NNS', '.', '--s--', 'JJ', 'NNS', 'VBP', 'RB', 'RB', 'VB', 'DT', 'NN', 'IN', 'VBG', 'DT', 'NN', ',', 'JJ', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NNS', 'MD', 'RB', 'VB', '.', '--s--', 'RB', ',', 'NNP', 'MD', 'VB', 'JJ', 'TO', 'VB', ',', 'VBP', ',', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'NN', 'JJ', 'IN', 'NNP', '.', '--s--', 'IN', 'DT', 'NN', 'VBD', 'NNS', ',', 'DT', 'NN', 'POS', 'NN', 'NNS', 'MD', 'VB', 'VBN', 'RP', 'IN', 'DT', 'JJ', 'NN', 'IN', 'WDT', 'DT', 'NN', 'VBZ', 'RB', 'VB', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'JJ', 'NNS', 'MD', 'RB', 'VB', 'DT', 'NNS', 'IN', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NNS', 'IN', 'NN', '.', '--s--', 'IN', 'JJ', 'NNS', ',', 'RB', 'WRB', 'DT', 'NN', 'NNS', 'RB', 'IN', 'DT', 'JJ', 'NN', ':', 'NNP', ',', 'IN', 'NN', ',', 'VBD', 'DT', 'NN', 'IN', 'VBG', 'PRP$', 'JJ', 'NN', '``', 'UH', "''", ':', 'PRP', 'VBZ', 'RB', 'VBN', 'TO', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'WRB', 'DT', 'NNS', 'VBP', 'VBN', 'TO', 'JJ', 'NNS', '.', '--s--', 'RB', ',', 'NNP', 'MD', 'VB', 'TO', 'VB', 'RP', 'DT', 'NNS', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'DT', 'NNS', 'VBP', 'IN', 'DT', 'NNS', 'IN', 'VBG', 'DT', 'NN', 'PRP', 'VBP', 'VBP', 'JJ', ':', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'JJS', '.', '--s--', 'CD', 'NNS', 'IN', 'PRP', 'VBD', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'NNP', 'NNPS', 'NNP', 'VBD', 'VBN', 'IN', 'NN', 'IN', 'VBG', 'JJS', 'IN', 'DT', 'NNS', '.', '--s--', 'JJ', 'NN', ',', 'VBG', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'DT', 'CD', 'NN', 'IN', 'DT', 'NNS', 'VBD', 'VBN', 'CC', 'VBD', 'RB', 'VBN', 'IN', 'DT', 'NNS', ',', 'VBD', 'DT', 'NNS', 'VBD', 'RB', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NNS', 'RB', 'VBD', 'VBN', 'VBN', '.', '--s--', 'CC', 'RB', 'CD', 'NNS', 'IN', 'DT', 'NNS', 'VBD', 'TO', 'VB', ',', 'JJ', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', ',', 'VBN', 'IN', 'NN', 'IN', 'NNP', '.', '--s--', 'DT', 'NNP', 'NN', 'VBD', '.', '--s--', 'DT', 'NNS', 'CC', 'NNS', 'VBP', 'VBN', 'RB', 'IN', '.', '--s--', 'NNP', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'NNS', 'MD', 'VB', 'DT', 'JJ', 'NN', 'VBZ', 'JJR', 'NN', 'IN', 'NNP', ',', 'CC', 'WDT', 'VBZ', 'JJ', '.', '--s--', 'IN', 'NNP', ',', 'DT', 'NN', 'NNS', ':', 'PRP', 'VBZ', 'IN', 'NN', ',', 'RB', 'NN', 'NN', ',', 'CC', 'NNP', 'VBZ', 'NNS', 'IN', 'NN', 'VBN', 'PRP$', 'NN', 'VBG', 'IN', 'NNS', 'CC', 'NNS', 'VBG', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', '(', 'NNP', ')', ',', 'WP', 'VBZ', 'DT', 'NN', 'WDT', 'VBZ', 'DT', 'NNP', ',', 'VBZ', 'NNP', 'NNP', 'VBZ', '``', 'DT', 'NN', 'VBZ', 'VBN', 'RB', 'VBN', 'IN', 'DT', 'JJ', 'NN', '.', "''", '--s--', 'CC', 'PRP', 'VBD', 'NNP', 'NNP', 'VBZ', 'DT', 'VBZ', 'DT', 'NNS', 'CC', 'NNS', 'TO', 'VB', 'PRP', 'RP', 'IN', 'PRP$', 'JJ', '.', '--s--', 'CC', 'IN', 'DT', 'NNP', ',', 'RB', ',', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBD', 'PRP', 'VBZ', 'DT', 'CD', 'NNS', 'TO', 'VB', 'RP', 'PRP$', 'JJ', 'NN', '.', '--s--', 'NNP', 'NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'PRP', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NN', 'IN', 'NNP', ',', 'VBG', '$', 'CD', 'CC', 'JJ', 'JJ', 'NNS', 'VBN', 'TO', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBN', 'DT', 'NN', ',', 'JJ', 'JJ', 'NN', 'NN', 'NNP', 'NNP', 'NNP', 'NNP', ',', 'JJ', 'NN', 'NN', 'NNP', 'NNP', 'NNP', 'CC', 'CD', 'NNS', 'WP', 'VBD', 'RB', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'VBD', 'DT', 'NNS', 'IN', 'VBG', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'IN', 'NNS', ',', 'TO', 'VB', '$', 'CD', 'IN', 'NNS', ',', 'NNS', 'CC', 'NNS', '.', '--s--', 'NNP', 'NNP', 'VBD', 'RB', 'VBN', 'TO', 'CD', 'NNS', 'IN', 'NN', 'CC', 'VBD', '$', 'CD', 'IN', 'PRP$', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'IN', 'DT', 'NN', ',', 'NNP', 'VBD', 'TO', 'VB', 'DT', 'NN', '$', 'CD', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NN', ',', 'NNP', 'NNP', 'CC', 'NNP', 'NNP', '.', '--s--', 'DT', 'CD', 'JJ', 'NNS', 'VBD', 'RB', 'NNS', 'TO', 'DT', 'NN', '.', '--s--', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'CC', 'DT', 'JJ', 'CD', 'JJ', 'NNS', 'VBD', 'NNP', 'CD', 'IN', 'JJ', 'NN', 'IN', 'DT', 'NNP', 'IN', 'NNP', '.', '--s--', 'PRP', 'VBD', 'VBN', 'JJ', 'NNP', 'IN', 'NNS', 'IN', 'NN', ',', 'NN', 'CC', 'NN', 'VBN', 'TO', 'DT', 'NN', 'TO', 'VB', '$', 'CD', 'CD', 'IN', 'NNP', 'NNP', 'NN', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NN', 'NN', 'NN', '.', '--s--', 'DT', 'NN', 'CC', 'PRP$', 'NNS', 'VBP', 'DT', 'NNS', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNP', 'VBZ', 'VBN', 'DT', 'NN', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'CD', 'NNS', '.', '--s--', 'DT', 'NNP', 'NNP', 'RB', 'VBN', 'DT', 'NN', 'NNP', 'CD', ',', 'CD', ',', 'CC', 'VBZ', 'VBN', 'VBG', 'DT', 'NN', 'RB', 'IN', '.', '--s--', 'DT', 'NN', 'VBD', 'PRP', 'MD', 'VB', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '--s--', 'NNP', 'NNP', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'JJ', 'JJ', 'NN', ',', 'RB', 'IN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNS', '.', '--s--', 'DT', 'JJ', 'NN', 'CC', 'NN', 'NN', ',', 'VBD', 'PRP', 'VBD', 'JJ', 'IN', '$', 'CD', 'CD', ',', 'CC', 'CD', 'NNS', ',', 'RB', 'CD', 'NN', 'IN', '$', 'CD', 'CD', ',', 'CC', 'CD', 'NNS', 'DT', 'NN', '.', '--s--', 'DT', 'JJR', 'NN', 'VBD', 'DT', 'NN', 'IN', '$', 'CD', 'CD', ',', 'JJ', 'TO', 'CD', 'NNS', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'IN', 'DT', 'NN', 'IN', 'VBG', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', '.', '--s--', 'NNP', 'RB', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'IN', 'IN', 'VBG', 'JJ', 'NN', '.', '--s--', 'NNS', 'VBD', 'TO', '$', 'CD', 'CD', ',', 'RB', 'CD', 'NN', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNP', 'POS', 'NN', 'VBD', 'IN', 'JJ', 'JJ', 'NN', 'NNP', ',', 'IN', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'POS', 'NN', 'IN', 'VBG', 'NNS', 'VBD', 'RB', 'RB', 'IN', 'DT', 'JJ', 'NN', 'POS', 'NNS', 'IN', '$', 'CD', 'CD', ',', 'CC', 'CD', 'NNS', 'DT', 'NN', ',', 'CC', 'IN', 'NNS', 'RB', 'VBD', 'DT', '$', 'CD', 'CD', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'NNP', 'RB', 'VBN', 'TO', 'DT', 'NN', 'NN', 'POS', 'NN', ',', 'VBG', 'IN', '$', 'CD', ',', 'RB', '$', 'CD', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'VBD', 'DT', 'JJ', 'NNS', 'MD', 'VB', '``', 'JJ', 'TO', 'RB', "''", 'IN', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'IN', 'VBG', 'NNS', 'NN', 'IN', 'PRP$', 'CD', 'NN', ',', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'CC', 'NNS', 'IN', 'VBG', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'POS', 'JJ', 'CD', 'NN', 'IN', 'DT', 'JJ', 'NN', '.', '--s--', 'IN', 'NNP', ',', 'NNP', 'NNP', ',', 'NNP', 'NN', 'CC', 'JJ', 'NN', 'NN', ',', 'VBD', '``', 'NNP', 'POS', 'NN', 'VBZ', 'JJ', '.', '--s--', 'PRP$', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'VBD', 'CC', 'NNP', 'VBD', 'RB', 'JJ', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'PRP$', 'NNS', 'VBD', 'JJR', 'IN', 'DT', 'JJ', 'NN', ',', 'CC', 'PRP$', 'JJ', 'NN', 'VBD', 'CD', '.', "''", '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'NNP', 'VBD', 'JJ', 'IN', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', ',', 'RB', 'CD', 'NN', 'IN', '$', 'CD', 'CD', ',', 'CC', '$', 'CD', 'DT', 'NN', '.', '--s--', 'NNP', 'VBD', 'TO', '$', 'CD', 'CD', ',', 'RB', 'RB', 'IN', '$', 'CD', 'CD', '.', '--s--', 'NNP', 'NNP', 'CC', 'DT', 'NNP', 'NNP', 'NNP', 'VBD', 'NN', 'NN', '.', '--s--', 'IN', 'CD', 'NNS', 'IN', 'NN', ',', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', ',', 'VBD', 'VBN', 'RP', 'IN', 'PRP$', 'NN', 'IN', 'DT', 'NN', 'NNS', 'POS', 'NN', 'RB', 'IN', 'DT', 'NN', 'VBD', 'VBG', 'RP', '.', '--s--', 'IN', 'DT', 'JJ', 'NN', ',', 'CD', 'NNP', 'NNS', ',', 'CD', 'IN', 'WP', 'VBD', 'VBN', 'IN', 'PRP', 'IN', 'CD', 'CC', 'VBN', 'TO', 'NN', 'NN', ',', 'VBD', 'VBN', 'IN', 'PRP$', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', 'CC', 'IN', 'DT', 'NN', 'NNS', 'VBD', 'TO', 'PRP$', 'NNS', ',', 'DT', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', 'IN', 'CD', 'CC', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'VBZ', 'VBN', 'TO', 'DT', 'NNS', 'IN', 'DT', 'NN', 'POS', 'JJ', 'NNS', '.', '--s--', 'DT', 'JJ', 'NNP', 'NN', ',', 'IN', 'JJ', ',', 'JJ', 'CC', 'JJ', 'NNS', ',', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'POS', 'JJ', 'NN', ',', 'IN', 'IN', 'DT', 'NN', 'IN', 'NN', ',', 'NNS', 'VBD', 'DT', 'NNP', 'NNS', 'IN', 'PRP$', 'NNS', ',', 'NNS', 'CC', 'NNS', '.', '--s--', 'IN', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NNP', 'NNS', 'CC', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'POS', 'JJ', 'NN', ',', 'VBG', 'PRP', 'RB', ',', 'NNP', 'VBD', 'JJ', 'NNP', 'NNS', 'IN', 'PRP$', 'NN', 'CC', 'DT', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'VBD', 'IN', 'NNP', 'NNS', '.', '--s--', '``', 'PRP', 'VBP', 'JJ', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NNS', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'VBG', 'NNS', 'IN', 'DT', 'JJ', 'JJ', 'NN', '.', '--s--', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NN', 'VBD', '``', 'NNP', ',', 'NNP', ',', "''", 'DT', 'JJ', 'NN', 'VBD', 'PRP$', 'NNS', 'IN', 'DT', 'NN', '.', '--s--', '``', 'PRP', 'VBP', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNS', '.', "''", '--s--', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NN', ',', 'DT', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', ',', 'CC', 'PRP$', 'NNS', 'VBP', 'RB', 'IN', 'NNP', ',', 'NNP', '.', '--s--', 'CC', 'DT', 'JJ', 'NN', 'IN', 'DT', 'CD', 'NNS', ',', 'WP', 'RB', 'VBN', 'DT', 'JJ', 'CC', 'JJ', 'NN', 'IN', 'DT', 'NNP', ',', 'VBZ', 'DT', 'FW', 'FW', 'FW', 'IN', 'DT', 'NN', 'CC', 'DT', 'NN', 'IN', 'PRP$', 'JJ', 'NN', '.', '--s--', '``', 'DT', 'NN', 'MD', 'RB', 'VB', 'DT', 'NNP', 'RB', 'IN', 'DT', 'NN', 'RB', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'NNP', 'NNP', '.', '--s--', '``', 'NNS', 'VBP', 'VBN', 'RB', 'RB', 'IN', 'DT', 'NN', 'TO', 'VB', 'PRP', 'RB', '.', '--s--', 'EX', 'VBZ', 'RB', 'VBG', 'RP', '.', "''", '--s--', 'EX', 'VBD', 'RB', 'RB', 'VBG', 'DT', 'NN', 'IN', 'NNP', 'NN', 'JJ', 'NN', ',', 'WRB', 'NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', 'IN', 'WP', 'VBD', 'DT', 'JJ', 'NNP', 'NN', 'IN', 'DT', 'NN', 'IN', 'CD', 'NNS', '.', '--s--', 'NNP', 'NN', 'IN', '``', 'NNP', "''", 'CC', '``', 'FW', 'FW', 'FW', "''", 'VBD', 'DT', 'NN', 'IN', 'DT', 'CD', 'NN', 'NNS', 'VBD', 'IN', 'DT', 'NNP', 'MD', 'VB', 'PRP$', 'NN', 'IN', 'DT', 'NN', 'CC', 'DT', 'NNS', 'IN', 'JJ', 'NN', 'IN', 'DT', 'NNS', ',', 'VBG', 'DT', 'JJ', 'NN', '.', '--s--', 'CC', 'PRP', 'VBD', 'IN', 'DT', 'NN', 'TO', 'VB', 'NNP', 'NNP', ',', 'DT', 'NNP', 'POS', 'JJ', 'NN', ',', 'WP', 'VBD', 'VBN', 'IN', 'PRP', 'CC', 'VBZ', 'IN', 'NN', '.', '--s--', 'IN', 'PRP', ',', 'VBD', 'NNP', 'NNP', ',', 'DT', 'VBG', 'IN', 'DT', 'NNS', '``', 'VBZ', 'RB', 'DT', 'NN', '.', "''", '--s--', 'NNP', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'NNP', 'NNS', ':', 'RB', 'IN', 'CD', 'IN', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NNP', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', ':', 'IN', 'NN', 'IN', 'PRP$', 'NNS', 'TO', 'VB', 'DT', 'NN', 'IN', 'NN', 'CC', 'NN', 'IN', 'WDT', 'PRP$', 'NN', 'MD', 'VB', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'VBN', 'IN', 'VBG', 'NNS', 'DT', 'NN', 'IN', 'JJ', 'NN', '.', '--s--', 'CC', 'NNP', 'MD', 'RB', 'VB', 'VBG', 'DT', 'NN', 'IN', 'JJR', 'NN', 'CC', 'NN', 'IN', 'DT', 'RB', 'VBN', 'NN', '.', '--s--', 'IN', 'JJ', 'NN', 'NNS', ',', 'RB', 'NNP', 'CC', 'DT', 'NNP', 'NNP', ',', 'VBP', 'RB', 'VBN', ',', 'JJ', 'NNS', 'TO', 'VB', 'RP', 'NN', 'MD', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NN', 'WDT', 'VBZ', 'JJ', ',', 'IN', 'RB', 'JJ', ',', 'TO', 'VB', '.', '--s--', 'IN', 'DT', 'NNS', 'VBP', 'IN', ',', 'DT', 'JJ', 'JJ', 'NN', 'MD', 'VB', 'RB', 'RBR', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'RB', 'IN', 'DT', 'VBN', 'VBG', 'IN', 'DT', 'NNP', 'CC', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', '.', '--s--', 'IN', 'PRP', 'VBZ', 'RB', 'VB', 'IN', 'DT', 'NNS', ',', 'CC', 'RB', 'VB', 'VBG', 'RB', 'TO', 'DT', 'NNP', ',', 'DT', 'NNS', 'CC', 'VBP', 'VBN', 'IN', 'NN', 'POS', 'NNS', 'MD', 'VB', 'TO', 'VB', 'CC', 'NN', '.', '--s--', 'IN', 'PRP', 'VBZ', ',', 'DT', 'JJ', 'NN', 'IN', 'JJ', 'NNS', ',', 'WP', 'VBP', 'DT', 'NNS', 'TO', 'DT', 'JJ', 'NN', ',', 'MD', 'VB', 'RP', 'PRP$', 'NNS', 'CC', 'NNS', 'TO', 'VB', 'NNS', 'IN', 'PRP$', 'JJ', 'NNS', '.', '--s--', 'DT', 'RB', 'VBN', 'NNP', 'NNS', 'RB', 'MD', 'VB', 'IN', 'JJ', 'NN', '.', '--s--', 'DT', 'NN', 'VBZ', 'VBG', 'RB', 'TO', 'VB', 'IN', 'PRP$', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'JJ', 'NNS', 'CC', 'NN', ':', 'IN', 'PRP', 'VBZ', ',', 'NNP', 'MD', 'VB', 'DT', 'IN', 'DT', 'NN', 'TO', 'VB', 'NNP', 'NNP', 'IN', 'NNS', '.', '--s--', 'NNP', 'VBZ', 'RB', 'VBN', 'WRB', 'PRP', 'VBD', 'DT', 'VBN', 'TO', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', ':', 'IN', 'NN', 'CC', 'NN', 'TO', 'VB', 'DT', 'NN', '.', '--s--', 'IN', 'NN', ',', 'DT', 'NN', 'VBZ', 'VBG', 'IN', 'DT', 'NNS', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'CC', 'JJ', 'NNS', 'IN', 'DT', 'NNP', 'CC', 'IN', 'DT', 'RB', 'VBN', 'NNS', 'CC', 'DT', 'NNS', 'WP', 'VBP', 'VBN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NN', '.', '--s--', 'IN', 'NN', 'TO', 'VB', 'RP', 'DT', 'NNS', ',', 'NNP', 'NNP', ',', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NNS', 'IN', 'PRP', 'VBD', 'VBN', ',', 'VBD', 'PRP', 'TO', 'VB', 'TO', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'IN', 'RB', 'IN', 'JJ', '.', '--s--', 'DT', 'NNS', 'RB', 'MD', 'VB', 'VBN', 'IN', 'VBG', 'DT', 'NN', 'NN', 'IN', 'PRP', 'CC', 'DT', 'NN', 'POS', 'JJ', 'JJ', 'JJ', 'NNS', ',', 'DT', 'JJ', 'JJ', 'NNS', 'WP', 'VBP', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'NNS', 'IN', 'NN', '.', '--s--', 'VBZ', 'NNP', 'NNP', ',', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NNP', 'NNP', ':', '``', 'PRP', 'MD', 'VB', 'VBG', 'PRP', 'TO', 'VB', 'IN', 'NNS', 'IN', 'DT', 'NNP', '.', "''", '--s--', 'PRP', 'RB', 'VBN', 'PRP', 'TO', 'VB', 'NN', 'RB', '.', '--s--', 'IN', 'JJ', 'NN', 'POS', 'NN', ',', 'PRP', 'VBD', 'IN', 'PRP$', 'NNS', 'TO', 'VB', 'NN', ',', 'RB', 'VBN', ',', 'IN', 'PRP$', 'NN', 'TO', 'NN', '.', '--s--', '``', 'PRP', 'VB', 'NN', 'IN', 'PRP', 'VBP', 'IN', 'DT', 'NN', 'VBZ', 'RB', ',', 'RB', 'JJ', ',', "''", 'VBD', 'NNP', 'NNP', ',', 'DT', 'JJ', 'NN', 'NN', 'WP', 'VBZ', 'RB', 'CD', '.', '--s--', '``', 'PRP', 'VBP', 'TO', 'VB', 'NNP', 'NNP', 'CC', 'PDT', 'PRP$', 'NNS', 'RP', 'IN', 'NN', ',', 'CC', 'IN', 'PRP', 'VBP', 'RB', 'VBN', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'RB', 'IN', 'PRP', '.', '--s--']

In [53]:
prediction == prediction_array2

True